# *Torneio Alfabeta Jogo Focus*
## Introdução à Inteligência Artificial (2025-26)
### Avaliação Contínua 3


## O jogo Focus (Domination)

O [Focus (ou Domination)](https://en.wikipedia.org/wiki/Focus_(board_game)), foi concebido por Sid Sackson e publicado pela primeira vez em 1963. É um jogo estratégico de empilhar peças num tabuleiro especial de forma octogonal.
Cada jogador move pilhas de peças, controlando apenas as que têm a sua cor no topo.

As pilhas deslocam-se na ortogonal tantas casas quanto o seu tamanho, podendo capturar peças adversárias ou guardar as suas na reserva quando se fundem pilhas e o resultado da fusão ultrapassa a altura de 5 peças. **Para facilitar, neste projeto vamos limitar essa altura a três peças e o tabuleiro será mais pequeno do que no jogo original**.


Vence quem conseguir dominar o tabuleiro.

<img src="focus4x4.png" alt="Drawing" style="width: 300px;"/>

### Como funciona o jogo? 

O Focus pode ser jogado por 2 a 4 jogadores, mas neste caso vamos ter apenas dois jogadores. 


#### 1. Início do Jogo
O tabuleiro da nossa versão do Focus tem a forma de um quadrado 4x4 com 4 extensões laterais de duas peças, formando um octógono.
Cada casa válida contém uma única peça, e as peças são dispostas alternadamente entre as cores RED (vermelho) e GREEN (verde), conforme o padrão mostrado na imagem acima.

Cada jogador controla as peças da sua cor:
* O jogador RED começa a partida.
* O jogador GREEN joga em seguida, alternando os turnos.

Durante o jogo, cada jogador mantém dois contadores:
* Reserva – peças próprias retiradas de pilhas demasiado grandes (podem ser recolocadas mais tarde).
* Capturas – peças do adversário removidas do tabuleiro (são eliminadas definitivamente).

#### 2. Jogada

Em cada turno, o jogador cuja vez é de jogar tem duas opções:

(A) Mover uma pilha 
* Pode mover qualquer pilha cujo topo seja da sua cor.
	* A pilha desloca-se em linha reta (horizontal ou vertical) um número de casas igual à quantidade de peças na pilha.
	* Se a casa de destino já contiver uma pilha, ambas são fundidas, colocando a pilha que se está a mover no topo da que está na casa de destino.
	* Se a nova pilha ultrapassar **três peças**, as peças em excesso são removidas, da parte inferior da pilha:
	    * As peças do próprio jogador vão para a sua reserva.
	    * As peças do adversário são capturadas.

(B) Jogar uma peça da reserva
* Em vez de mover uma pilha, o jogador pode colocar uma peça da sua reserva em qualquer casa do tabuleiro (incluindo no topo de uma pilha cuja peça do topo não seja a sua).
* Esta ação é útil para recuperar controlo sobre pilhas dominadas pelo adversário.

NOTA: uma peça única (não empilhada) é por si só um caso particular de uma pilha, cujo topo é a cor dessa mesma peça.

#### 3. Fim do Jogo
O jogo termina quando um dos jogadores não tem mais jogadas possíveis, ou seja:
* não pode mover nenhuma pilha cujo topo seja da sua cor,
* e não tem peças na reserva para colocar.

O **vencedor** é o último jogador que ainda consegue realizar uma jogada válida.

* Como o jogo pode tornar-se de longa duração, vamos assumir que se um limite máximo de jogadas for atingido (por exemplo, 250 jogadas), o jogo termina. Será o vencedor aquele que tiver um maior número de pilhas com a sua cor. Será empate se nesse momento ambos controlarem o mesmo número de pilhas.

## Objetivos do projeto
Pretende-se que, dada uma formulação e implementação do jogo, os grupos de alunos:
<br><br>
Criem um jogador, na forma de uma função de avaliação a ser usada pelo algoritmo alfabeta para qualquer profundidade, par ou ímpar. Aconselhamos que comecem por desenvolver um jogador simples para depois ser progressivamente melhorado, de modo a que tenham um jogador disponível na data de entrega limite. Desenvolvam e comparem o desempenho de vários jogadores, para diferentes limites de profundidade, e depois selecionem o melhor deles para entrega. Cada grupo só pode participar no torneio com um único jogador.
<br><br>
O jogador de cada grupo participará em torneios de todos contra todos, i.e., cada jogador irá jogar vários jogos contra os jogadores de todos os outros grupos, em diferentes níveis de profundidade. O nosso jogador também participará, o <span style="color:orange"> ***Basicus***</span>.

<img src="basicus.png" alt="Drawing" style="width: 200px;"/>
<br>
(imagem gerada pelo GPT-5)



O Basicus sabe as regras do jogo e pouco mais. Quando imagina um tabuleiro que resultaria de uma possível jogada, sabe verificar se esse tabuleiro só tem pilhas cujo topo é da mesma cor, o que significaria uma vitória (se essas pilhas forem dele) ou uma derrota (se forem do adversário). Também sabe que, se não existirem apenas pilhas cujo topo é da sua cor, a *maximização* desse número também é importante. O desempenho do Basicus é a *baseline* que devem ultrapassar de forma a obterem uma nota superior a 10 valores.


## Formulação do Jogo Focus em Python

Vamos descrever de modo sumário como está modelado o jogo Focus, através das classes `EstadoFocus` e `JogoFocus` que estão em `focus.py`.

### O estado do jogo
Na classe `EstadoFocus` temos cinco atributos principais:
* `to_move`: Indica quem é o próximo a jogar (`'RED'` se for o jogador das peças vermelhas, `'GREEN'` se for o das peças verdes).
* `board`: Uma representação do tabuleiro sob a forma de um dicionário que guarda a informação sobre a localização das pilhas no tabuleiro. As chaves são as posições $(linha,coluna)$ das casas ocupadas (sendo a primeira linha a de cima e a primeira coluna a da esquerda) e os valores correspondem a listas com valores das cores dos jogadores, ordenadas da base para o topo. As casas livres não são representadas no board. 
* `reserve`: Indica quantas peças cada jogador tem na sua reserva. A representação é feita através de um dicionário cujas chaves são os jogadores (`'RED'` or `'GREEN'`) e os valores o número de peças correspondentes.
* `captured`: Indica quantas peças cada capturou do adversário. A representação é feita através de um dicionário cujas chaves são os jogadores (`'RED'` or `'GREEN'`) e os valores o número de peças correspondentes.
* `n_jogadas`: Indica quantas jogadas foram feitas desde o início do jogo. Serve para determinar o final do jogo que esteja a ser longo (depois de 250 jogadas contam-se o número de pilhas controladas pelos jogadores e vence o que controlar mais pilhas, ou, se ambos tiverem o mesmo número de pilhas controladas, há um empate).

Temos também os métodos: 

* **`valid_positions()`** (*static*): Função que devolve o conjunto de posições válidas do tabuleiro do Jogo Focus (de acordo com a imagem do tabuleiro acima).

* **`all_positions()`**: Devolve todas as posições possíveis do tabuleiro, sob a forma de lista de coordenadas `(x, y)`.

* **`top_piece(pos)`**: Devolve a cor da peça que está no topo da pilha na posição dada ou `None` no caso da casa se encontrar vazia.

* **`is_valid_position(pos)`**: Verifica se a posição faz parte do tabuleiro.

* **`possible_moves()`**: Devolve uma lista de tuplos com todas as ações possíveis para o jogador que está a jogar. As ações podem ser de dois tipos:
    - `((x, y), dir)` que representa a ação de mover uma pilha na posição `(x, y)` na direção `dir`. `dir` pode tomar os valores `'up'` (cima), `'down'` (baixo), `'left'` (esquerda) ou `'right'` (direita);
    - `('reserve', (x, y))` que representa a ação de colocar no tabuleiro uma peça da reserva do jogador. A peça pode ser colocada em qualquer lugar, mesmo em cima de uma pilha que não seja controlada pelo jogador.

* **`next_state(action)`**: Dada uma jogada (`action`), devolve o **novo estado do jogo** resultante.  
  - Se a jogada for de movimentação `((x, y), dir)`, a peça avança na direção `dir` tantas casas quantas peças estiverem na pilha.  
  - Se a jogada for de colocação de uma peça da reserva `('reserve', (x, y))`, a peça é colocada no topo da pilha da posição indicada.  
  Retorna um novo objeto `EstadoFocus` com o tabuleiro, a reserva e captura atualizados, o jogador seguinte e o número de jogadas incrementado.

* **`has_moves(player)`**: Verifica se o jogador `player` tem movimentos possíveis no tabuleiro atual.

* **`winner()`**: Verifica se algum jogador venceu.  
  Devolve `'RED'` ou `'GREEN'` se um dos jogadores tiver vencido o jogo, i.e., o oponente não tem jogadas possíveis. Caso contrário, devolve `None`.

* **`other()`**: Devolve o oponente do jogador `to_move`.

* **`who_dominate()`**: Devolve o jogador que domina todas as pilhas do tabuleiro, ou `None` caso nenhum domine todas.

* **`dominate_piles(player)`**: Devolve o número de pilhas que têm o topo da cor do jogador `player`.

* **`dominate_pieces(player)`**: Devolve o número total de peças controladas pelo jogador `player`, ou seja, todas as peças em pilhas cujo topo é do jogador.

* **`display()`** : Mostra o **tabuleiro do jogo**. Cada posição é representada pelos caracteres `[  ]`. Entre eles são apresentadas as peças da base para o topo, i.e., o primeiro elemento da é a peça da base. Cada peça é representada por um carácter (`R` para *Red* e `G` para *Green*). As casas vazias são representadas vazias. Inclui também índices de linhas e colunas para facilitar a leitura.

Vejamos agora alguns exemplos, após importarmos `focus.py`.

In [2]:
from focus import *

Criemos o estado inicial standard e observemos os seus atributos:

In [43]:
tabuleiro_inicial = criar_tabuleiro_inicial()
estado_inicial = EstadoFocus(to_move='RED', board=tabuleiro_inicial, reserve={'RED':0,'GREEN':0},
                           captured={'RED':0,'GREEN':0}, n_jogadas=0)
print("Próximo jogador:",estado_inicial.to_move)
print("Tabuleiro:",estado_inicial.board)
print("Peças Vermelhas na Reserva:",estado_inicial.reserve['RED'])
print("Peças Verdes na Reserva:",estado_inicial.reserve['GREEN'])
print("Peças Vermelhas Capturadas:",estado_inicial.reserve['RED'])
print("Peças Verdes Capturadas:",estado_inicial.reserve['GREEN'])
print("No.Jogadas:",estado_inicial.n_jogadas)

Próximo jogador: RED
Tabuleiro: {(0, 0): ['GREEN'], (0, 1): ['RED'], (0, 2): ['GREEN'], (0, 3): ['RED'], (1, 0): ['GREEN'], (1, 1): ['RED'], (1, 2): ['GREEN'], (1, 3): ['RED'], (2, 0): ['RED'], (2, 1): ['GREEN'], (2, 2): ['RED'], (2, 3): ['GREEN'], (3, 0): ['RED'], (3, 1): ['GREEN'], (3, 2): ['RED'], (3, 3): ['GREEN']}
Peças Vermelhas na Reserva: 0
Peças Verdes na Reserva: 0
Peças Vermelhas Capturadas: 0
Peças Verdes Capturadas: 0
No.Jogadas: 0


Agora observemos o tabuleiro em modo textual: 

In [44]:
estado_inicial.display()

-1 |                  [   ]  [   ]              
 0 |           [  G]  [  G]  [  R]  [  R]       
 1 |    [   ]  [  R]  [  R]  [  G]  [  G]  [   ]
 2 |    [   ]  [  G]  [  G]  [  R]  [  R]  [   ]
 3 |           [  R]  [  R]  [  G]  [  G]       
 4 |                  [   ]  [   ]              
        _____  _____  _____  _____  _____  _____
         -1      0      1      2      3      4


Vamos perguntar se temos um vencedor (obviamente que não):

In [45]:
print(estado_inicial.winner())

None


Vamos aplicar uma jogada e visualizar o tabuleiro resultante:

In [46]:
estado_inicial.next_state(((1,3), 'up')).display()

-1 |                  [   ]  [   ]              
 0 |           [  G]  [  G]  [  R]  [  R]       
 1 |    [   ]  [  R]  [  R]  [  G]  [  G]  [   ]
 2 |    [   ]  [  G]  [ GR]  [  R]  [  R]  [   ]
 3 |           [  R]  [   ]  [  G]  [  G]       
 4 |                  [   ]  [   ]              
        _____  _____  _____  _____  _____  _____
         -1      0      1      2      3      4


### A classe `JogoFocus`, subclasse de `Game`
A classe `JogoFocus` (em `focus.py`) é uma subclasse de `Game` (em `jogos.py`). 

O construtor gera o estado inicial do jogo com um tabuleiro resultado da função `criar_tabuleiro_inicial()` (ver imagem no início do enunciado), e o jogador 'RED' é o primeiro a jogar.

Temos também os métodos habituais:
* `actions`: Devolve a lista de ações possíveis (jogadas possíveis) para um determinado estado.
* `result`: Devolve o estado que resulta da aplicação de uma ação (jogada) a um outro estado.
* `terminal_test`: Verifica se o jogo acabou ou não.
* `utility`: Devolve +1 se ganhou o jogador, -1 se ganhou o adversário, 0 se empataram.
* `display`: Apresenta em modo de texto o estado do jogo, incluindo o tabuleiro e outras informações.

### Jogadores
Em `jogos.py` temos vários tipos de jogadores e funções para correr jogos.

#### O jogador aleatório
O jogador aleatório escolhe ao acaso uma das jogadas possíveis.
Vamos ver um jogo entre dois jogadores aleatórios:

In [3]:
from focus import *
from jogos import *
jogo = JogoFocus()
jogo.jogar(random_player,random_player)


   Jogadas feitas: 0
   Tabuleiro atual:
-1 |                  [   ]  [   ]              
 0 |           [  G]  [  G]  [  R]  [  R]       
 1 |    [   ]  [  R]  [  R]  [  G]  [  G]  [   ]
 2 |    [   ]  [  G]  [  G]  [  R]  [  R]  [   ]
 3 |           [  R]  [  R]  [  G]  [  G]       
 4 |                  [   ]  [   ]              
        _____  _____  _____  _____  _____  _____
         -1      0      1      2      3      4

  Próximo jogador: RED

  Peças na reserva: 0   (GREEN: 0)
  Peças capturadas: 0   (GREEN: 0)
  Pilhas dominadas: 8   (GREEN: 8)

   Jogadas feitas: 1
   Tabuleiro atual:
-1 |                  [   ]  [   ]              
 0 |           [  G]  [  G]  [  R]  [  R]       
 1 |    [   ]  [ RR]  [   ]  [  G]  [  G]  [   ]
 2 |    [   ]  [  G]  [  G]  [  R]  [  R]  [   ]
 3 |           [  R]  [  R]  [  G]  [  G]       
 4 |                  [   ]  [   ]              
        _____  _____  _____  _____  _____  _____
         -1      0      1      2      3      4

  Pró

1

O score final é +1(-1) se RED ganhou(perdeu), ou 0 se empataram.
Se quisermos correr um jogo vendo só os scores finais, podemos usar a função `jogar()` em modo não verboso:

In [4]:
jogo=JogoFocus()
jogo.jogar(random_player,random_player,verbose=False)

0

#### O jogador Alfabeta com profundidade limitada
No nosso torneio, os jogadores vão sempre usar o algoritmo Alfabeta. No jogo Focus, não é viável desenvolver a árvore até ao fim, por isso vamos usar o Alfabeta com profundidade limitada (função `alphabeta_cutoff_search_new` definida em `jogos.py`). Esta função recebe um estado, um jogo, a profundidade de procura, e uma função de avaliação. É a função de avaliação de cada jogador que vai determinar o seu desempenho.

Vamos equipar o jogador aleatório com uma função de avaliação. Ele continuará a escolher jogadas ao acaso, mas conseguirá reconhecer a situação em que o próximo jogador poderá ganhar o jogo (porque pode colocar/mover uma peça e dominar todas as pilhas). Quando é detetado o final iminente do jogo, a função de avaliação deve devolver `infinity`/`-infinity` para maximixar o número de cortes que a função `alphabeta_cutoff_search_new` faz.

In [5]:
def func_gameover(estado,jogador) :
    clone=copy.deepcopy(estado) #boa prática de programação, para não arriscarem estragar o estado
    winner = clone.winner()
    if winner != None:
        return infinity if winner==jogador else -infinity
    return 0 #em qualquer outra situação que não seja vitória ou derrota

def jogador_random_plus_1(jogo,estado) :
    return alphabeta_cutoff_search_new(estado,jogo,1,eval_fn=func_gameover)
    

Vejamos a vantagem que traz esta função de avaliação. Reparem que o `jogador_random_plus_1` joga com uma profundidade de procura 1. Quer isto dizer que a procura não vai além da jogada imediata; nem sequer olha para qualquer possível jogada do adversário. Vamos fazer 10 jogos entre o `jogador_random_plus_1` e o jogador aleatório que não reconhece o final do jogo. <br><br>
Reportamos os scores resultantes de cada jogo, e no final a soma de todos os scores, que nos dá a pontuação do primeiro jogador, o `jogador_random_plus_1`. Esta pontuação será positiva(negativa) se o primeiro(segundo) jogador ganhou a maioria dos jogos, e será 0 se ambos os jogadores ganharem o mesmo número de jogos (ou se empatarem todos os jogos).

In [11]:
pontuacao=0
for i in range(10):
    resultado = jogo.jogar(jogador_random_plus_1,random_player,verbose=False)
    print(resultado)
    pontuacao += resultado
print('Pontuação do jogador_random_plus_1:',pontuacao)

0
1
-1
0
0
1
1
1
0
0
Pontuação do jogador_random_plus_1: 3


Este jogador parece ser um bocadinho melhor do que o totalmente aleatório. Deverá ser ainda melhor com profundidades maiores. Vejamos os resultados finais obtido em 10 jogos a profundidades 1 a 5 (podem correr várias vezes, reparando como o tempo de execução aumenta com a profundidade):

In [8]:
def jogador_random_plus_p(jogo,estado) :
    return alphabeta_cutoff_search_new(estado,jogo,p,eval_fn=func_gameover)
    
for p in range(1,6):
    def jogador_random_plus_p(jogo,estado) :
        return alphabeta_cutoff_search_new(estado,jogo,p,eval_fn=func_gameover)
    
    print('Profundidade', p)
    quem_ganhou=0
    for i in range(10):
        resultado = jogo.jogar(jogador_random_plus_p,random_player,verbose=False)
        #print(resultado)
        quem_ganhou += resultado
    print(quem_ganhou)


Profundidade 1
1
Profundidade 2
1
Profundidade 3
0
Profundidade 4
4
Profundidade 5
2


### O Basicus
Vamos agora definir a função de avaliação do nosso jogador <span style="color:orange"> ***Basicus***</span>, chamada `func_basicus`. Como dito acima, o Basicus sabe reconhecer o final do jogo e sabe é importante dominar o máximo de pilhas.

In [6]:
def func_basicus(estado,jogador) :
    clone = copy.deepcopy(estado)
    winner = clone.winner()
    
    # --- Caso terminal ---
    if winner is not None:
        return infinity if winner == jogador else -infinity

    my_pilhas = clone.dominate_piles(jogador)
    opponent = 'GREEN' if jogador == 'RED' else 'RED'
    opp_pilhas = clone.dominate_piles(opponent)
    return my_pilhas - opp_pilhas


Vamos verificar como é que esta função de avaliação avalia alguns estados: 

In [7]:
board_1 = {
    (0, 0): ['RED'],
    (1, 0): ['GREEN'],
    (0, 1): ['GREEN'],
    (1, 1): ['RED'],
}

board_2 = {
    (0, 0): ['RED', 'RED'],
    (1, 0): ['RED'],
    (0, 1): ['RED'],
    (1, 1): ['RED'],
}

board_3 = {
    (0, 0): ['RED'],
    (1, 0): ['RED', 'GREEN', 'RED'],   
    (0, 1): ['GREEN'],
    (1, 1): ['RED'],
}

board_4 = {
    (0, 0): ['GREEN'],
    (1, 0): ['GREEN', 'RED', 'GREEN'],  
    (0, 1): ['RED'],
    (1, 1): ['GREEN'],
}

#para este efeito só interessa inicializar o coerentemente o tabuleiro
est1=EstadoFocus('RED',board_1, {'RED':0, 'GREEN':0}, {'RED':0, 'GREEN':0}, 10) 
est2=EstadoFocus('RED',board_2,{'RED':0, 'GREEN':0}, {'RED':0, 'GREEN':0}, 10)
est3=EstadoFocus('RED',board_3,{'RED':0, 'GREEN':0}, {'RED':0, 'GREEN':0}, 10)
est4=EstadoFocus('RED',board_4,{'RED':0, 'GREEN':0}, {'RED':0, 'GREEN':0}, 10)

jogo=JogoFocus()

# Aqui ninguem ganhou e ambos dominam o mesmo número pilhas
est1.display()
print('Avaliação segundo',est1.to_move,': ',func_basicus(est1,est1.to_move))
print('Avaliação segundo',est1.other(),': ',func_basicus(est1,est1.other()),'\n\n')

# Aqui o RED ganhou
est2.display()
print('Avaliação segundo',est2.to_move,': ',func_basicus(est2,est2.to_move))
print('Avaliação segundo',est2.other(),': ',func_basicus(est2,est2.other()),'\n\n')

# Aqui o RED domina mais pilhas do que o GREEN
est3.display()
print('Avaliação segundo',est3.to_move,': ',func_basicus(est3,est3.to_move))
print('Avaliação segundo',est3.other(),': ',func_basicus(est3,est3.other()),'\n\n')

# Aqui o GREEN domina mais pilhas do que o RED
est4.display()
print('Avaliação segundo',est4.to_move,': ',func_basicus(est4,est4.to_move))
print('Avaliação segundo',est4.other(),': ',func_basicus(est4,est4.other()),'\n\n')


-1 |                  [   ]  [   ]              
 0 |           [  R]  [  G]  [   ]  [   ]       
 1 |    [   ]  [  G]  [  R]  [   ]  [   ]  [   ]
 2 |    [   ]  [   ]  [   ]  [   ]  [   ]  [   ]
 3 |           [   ]  [   ]  [   ]  [   ]       
 4 |                  [   ]  [   ]              
        _____  _____  _____  _____  _____  _____
         -1      0      1      2      3      4
Avaliação segundo RED :  0
Avaliação segundo GREEN :  0 


-1 |                  [   ]  [   ]              
 0 |           [ RR]  [  R]  [   ]  [   ]       
 1 |    [   ]  [  R]  [  R]  [   ]  [   ]  [   ]
 2 |    [   ]  [   ]  [   ]  [   ]  [   ]  [   ]
 3 |           [   ]  [   ]  [   ]  [   ]       
 4 |                  [   ]  [   ]              
        _____  _____  _____  _____  _____  _____
         -1      0      1      2      3      4
Avaliação segundo RED :  inf
Avaliação segundo GREEN :  -inf 


-1 |                  [   ]  [   ]              
 0 |           [  R]  [RGR]  [   ]  [   ]       

Vamos definir o jogador <span style="color:orange"> ***Basicus***</span> e ver qual o seu desempenho ao jogar com o jogador random_plus com profundidades de procura 1, 2, 3 e 4. Fazemos só 10 jogos com cada profundidade, para não demorar muito. Mostramos também o resultado de cada jogo, para além da pontuação final.

In [8]:
jogo=JogoFocus()

def jogador_random_plus_p(jogo,estado) :
    return alphabeta_cutoff_search_new(estado,jogo,p,eval_fn=func_gameover)

def jogador_basicus_p(jogo,estado) :
    return alphabeta_cutoff_search_new(estado,jogo,p,eval_fn=func_basicus)
    

In [9]:
for p in range(1,5):
    def jogador_random_plus_p(jogo,estado) :
        return alphabeta_cutoff_search_new(estado,jogo,p,eval_fn=func_gameover)
    def jogador_basicus_p(jogo,estado) :
        return alphabeta_cutoff_search_new(estado,jogo,p,eval_fn=func_basicus)
    
    # print('\nProfundidade:',p)
    # pontuacao=0
    # for i in range(10):
    #     resultado = jogo.jogar(jogador_basicus_p,jogador_random_plus_p,verbose=False)
    #     # print(resultado)
    #     pontuacao += resultado
    # print('Pontuação do Basicus:',pontuacao)

Curiosamente, a estratégia do Basicus parece funcionar melhor a profundidades mais baixas e às vezes chega mesmo a perder contra o jogador aleatório. Podemos também correr um jogo com o utilizador humano. Atenção: o jogador humano deve indicar sempre uma das jogadas possíveis, caso contrário, o jogo não é válido. Tentem ganhar ao Basicus!

In [54]:
jogo=JogoFocus()
p=1
# jogo.jogar(query_player,jogador_basicus_p) # descomentar para jogar

### Funções de avaliação compostas
Obviamente que o Basicus, mesmo sendo melhor (em média) do que o jogador aleatório, não é lá grande jogador. Maximar o número de pilhas dominadas não lhe serve de muito se o adversário tiver liberdade para cobrir as suas pilhas na jogada seguinte. Para obterem um jogador melhor, poderão ter que combinar vários critérios na mesma função de avaliação! 

### N Pares de jogos
Vejamos agora uma função que realiza $N$ pares de jogos entre dois jogadores - $N$ jogos com um deles a jogar primeiro, $N$ jogos com o outro a jogar primeiro. Devolve um tuplo com 4 elementos: o número de vitórias de cada um e de empates quando um deles joga primeiro; o número de vitórias de cada um e de empates quando o outro joga primeiro; o número total de vitórias de cada um e de empates; e finalmente o score de cada um. A pontuação depende da tabela de `scores`, que neste caso indica que a vitória vale $3$, a derrota $0$ e o empate $1$. É a mesma escala que iremos utilizar no torneio.

In [10]:
def jogador_basicus_1(jogo,estado) :
    return alphabeta_cutoff_search_new(estado,jogo,1,eval_fn=func_basicus)

In [11]:
# Dicionário global de pontuação
SCORES = {'Vitoria': 3, 'Empate': 1}

def traduz_pontos(tabela):
    """
    Converte uma tabela de resultados (vitórias, empates) em pontuação.
    """
    empates = tabela['Empate']
    return {
        jogador: SCORES['Vitoria'] * vitorias + SCORES['Empate'] * empates
        for jogador, vitorias in tabela.items() if jogador != 'Empate'
    }

def joga_n_pares(jogo, n, jog1, jog2):
    """
    Executa n pares de jogos entre jog1 e jog2 (ida e volta).
    Retorna:
      - tabelaPrim: resultados da primeira sequência (jog1 vs jog2)
      - tabelaSeg: resultados da segunda sequência (jog2 vs jog1)
      - tabela: soma das duas
      - tabela_pontos: pontuação traduzida
    """
    name_jog1, name_jog2 = jog1.__name__, jog2.__name__
    inicial = {name_jog1: 0, name_jog2: 0, 'Empate': 0}
    tabela_prim, tabela_seg = inicial.copy(), inicial.copy()

    for _ in range(n):
        # Jogo 1: jog1 vs jog2
        vencedor = jogo.jogar(jog1, jog2, verbose=False)
        if vencedor > 0:
            tabela_prim[name_jog1] += 1
        elif vencedor < 0:
            tabela_prim[name_jog2] += 1
        else:
            tabela_prim['Empate'] += 1

        # Jogo 2: jog2 vs jog1 (inverso)
        vencedor = jogo.jogar(jog2, jog1, verbose=False)
        if vencedor > 0:
            tabela_seg[name_jog2] += 1
        elif vencedor < 0:
            tabela_seg[name_jog1] += 1
        else:
            tabela_seg['Empate'] += 1

    tabela_total = {
        chave: tabela_prim[chave] + tabela_seg[chave]
        for chave in tabela_prim
    }

    return tabela_prim, tabela_seg, tabela_total, traduz_pontos(tabela_total)

Façamos 5 pares de jogos entre o `jogador_basicus_1` e o `jogador_random_plus_1` e vejamos a pontuação final:

In [12]:
jogo=JogoFocus()
joga_n_pares(jogo, 5, jogador_basicus_1, jogador_random_plus_1)

({'jogador_basicus_1': 5, 'jogador_random_plus_1': 0, 'Empate': 0},
 {'jogador_basicus_1': 5, 'jogador_random_plus_1': 0, 'Empate': 0},
 {'jogador_basicus_1': 10, 'jogador_random_plus_1': 0, 'Empate': 0},
 {'jogador_basicus_1': 30, 'jogador_random_plus_1': 0})

### Torneio entre vários jogadores
Eis a função que realiza um torneio entre vários jogadores, em que cada um realiza um número $N$ de jogos contra todos os outros como primeiro jogador, e $N$ como segundo jogador:

In [13]:
def incorpora(tabela, nova_tabela):
    """
    Soma os valores de 'nova_tabela' à tabela principal.
    """
    for jogador, pontos in nova_tabela.items():
        tabela[jogador] = tabela.get(jogador, 0) + pontos


def torneio(n, jogadores, jogo):
    """
    Executa um torneio round-robin entre os jogadores fornecidos.

    Parâmetros:
        n (int): número de pares de jogos (ida e volta) entre cada dupla.
        jogadores (list): lista de funções/jogadores.
        jogo (obj): jogo (ex: JogoFocus).

    Retorna:
        dict: tabela final ordenada por pontuação.
    """
    tabela = {}

    for i, jog1 in enumerate(jogadores[:-1]):
        for jog2 in jogadores[i+1:]:
            print(f"{jog1.__name__} VS {jog2.__name__}")
            _, _, _, tabela_parcial = joga_n_pares(jogo, n, jog1, jog2)
            incorpora(tabela, tabela_parcial)

    tabela_final = dict(sorted(tabela.items(), key=lambda x: x[1], reverse=True))

    print("\n🏆 Classificação Final 🏆")
    for pos, (jogador, pontos) in enumerate(tabela_final.items(), start=1):
        print(f"{pos:>2}. {jogador:<15} {pontos:>4} pontos")

    return tabela_final

In [14]:
def jogador_grupo_28(jogo,estado) :
            return alphabeta_cutoff_search_new(estado,jogo,1,eval_fn=func_28)

Fazemos agora um torneio com três dos jogadores que definimos até agora, com 5 pares de jogos entre eles.<br> Os resultados são apresentados do melhor para o pior jogador:

In [15]:
torneio(5,[jogador_basicus_1, jogador_random_plus_1, random_player, jogador_grupo_28], JogoFocus())

jogador_basicus_1 VS jogador_random_plus_1
jogador_basicus_1 VS random_player
jogador_basicus_1 VS jogador_grupo_28
jogador_random_plus_1 VS random_player
jogador_random_plus_1 VS jogador_grupo_28
random_player VS jogador_grupo_28

🏆 Classificação Final 🏆
 1. jogador_basicus_1   81 pontos
 2. jogador_grupo_28   69 pontos
 3. random_player     17 pontos
 4. jogador_random_plus_1   11 pontos


{'jogador_basicus_1': 81,
 'jogador_grupo_28': 69,
 'random_player': 17,
 'jogador_random_plus_1': 11}

E pronto, agora divirtam-se a derrotar o Basicus!

<img src="basicus.png" alt="Drawing" style="width: 200px;"/>
<br>
(imagem gerada pelo GPT-5)

## Material a entregar
Devem entregar um ficheiro **Focus_proj_grupoXX.py** (em que XX é o número do grupo registado no moodle), com o código da vossa função de avaliação e todas as funções auxiliares que criarem.

<span style="color:red"> **Não alterem**</span> `focus.py`, `utils.py` nem `jogos.py` e **não devem submetê-los!**
<br><span style="color:red">**Não redefinam**</span> funções com o mesmo nome das já existentes nestes ficheiros.

A função de avaliação do vosso jogador deverá ter assinatura **`func_XX(estado,jogador)`** (substituindo XX pelo número do grupo) e deve devolver o valor estimado do `estado` na perspectiva do `jogador`. Não precisam de definir o vosso jogador, pois seremos nós a defini-lo com diferentes profundidades durante a execução do campeonato.

**Todas** as vossas funções devem ter o sufixo **_XX** (substituindo XX pelo número do grupo), para que não se partilhe nem se sobreponha código durante a avaliação.

## Avaliação
A nota do vosso projecto depende da pontuação final obtida no torneio *múltiplo*. 
   
**Torneios**: Cada jogador irá participar em torneios contra todos os outros, fazendo pelo menos dois jogos (um como primeiro jogador, outro como segundo jogador) contra cada um dos outros jogadores em cada torneio. Faremos torneios a diferentes profundidades, pelo menos profundidade 3 e profundidade 4. Se o tempo de execução o permitir, faremos mais jogos por torneio, e mais torneios a profundidades maiores. Caso o tempo de execução se torne mesmo problemático, poderemos ter de baixar o número máximo de jogadas para 100.

**Pontuação**: Nos torneios, vamos sempre incluir o nosso jogador <span style="color:orange"> ***Basicus***</span>. A pontuação obtida por cada grupo em cada torneio será a *soma dos pontos obtidos em todos os jogos* (em que cada vitória vale 3, cada empate vale 1, e cada derrota vale 0). A pontuação final será a soma das pontuações obtidas em todos os torneios.

**Nota**: Quem obtiver a mesma pontuação final que o <span style="color:orange"> ***Basicus***</span>, terá 10 valores. Quem tiver uma pontuação inferior ao Basicus terá nota menor do que 10. A nota será resultado da aplicação de uma função linear baseada na pontuação obtida, tanto acima do Basicus como para baixo. O grupo que ficar em primeiro lugar no torneio (naturalmente, acima do Basicus), terá 20 valores.

**Clones**: Qualquer jogador que use a mesma estratégia do <span style="color:orange"> ***Basicus***</span> ou de outro jogador definido no enunciado será desclassificado e a sua nota final será 0. Jogadores que sejam cópias uns dos outros também serão desclassificados. 

**Timeout**: Se um jogador ultrapassar o tempo limite para uma jogada, será também desclassificado. Usaremos bom senso para definir o que é o tempo considerado como limite: será o tempo acima do qual a execução do torneio fica comprometida.

**Ficheiros**: Se um grupo entregar ficheiros que não seguem as regras definidas acima (nomeadamente, no que respeita aos sufixos nos nomes das funções), sofrerá uma penalização na nota proporcional ao tempo necessário para resolver a situação. No limite, não será possível a sua participação no torneio e a nota será 0.

In [23]:
from focus import EstadoFocus, JogoFocus
from typing import Callable
# from itertools import take

import copy
import math

infinity = float('inf')

def func_28(estado: EstadoFocus, jogador: str) -> float:
    """
    Estado do jogo Focus:
    - 'board': dicionário {(x,y): [peças, de baixo para cima]}
    - 'reserve': {'RED': n, 'GREEN': n}
    - 'captured': {'RED': n, 'GREEN': n}
    
    Jogador retorna uma jogada/action/move que pode estar na formas:
    - ((x,y), direction) ex ((1,3), "up")   --  move da pos (1,3) para (1, 3 - n ) se a pilha tiver altura n
    ou 
    - ('reserve', (x,y))   -- tira da reserva de pecas e coloca na posicao (x,y)
    """
    clone = copy.deepcopy(estado)
    winner = clone.winner()
    
    # --- Caso terminal ---
    if winner is not None:
        return infinity if winner == jogador else -infinity

    opponent = "GREEN" if jogador == "RED" else "RED"

    res_val = reserve_two(estado, jogador, opponent)
    towers_val = towers_score(estado, jogador, opponent)
    captured_val = capture_score(estado, jogador, opponent)

    return (res_val + towers_val) * captured_val


def reserve_two(estado: EstadoFocus, jogador: str, opponent: str) -> float:
    res_plyr: int = estado.reserve[jogador]
    res_opp: int  = estado.reserve[opponent]
    res_diff: int = res_plyr - res_opp
    if res_diff >= 1:
        return 16 + math.log(res_diff, 2)
    else:
        return -10
    
def towers_score(estado: EstadoFocus, jogador: str, opponent: str) -> float:
    # Dominating more stacks is good
    plyr_doms = estado.dominate_piles(jogador)
    opp_doms = estado.dominate_piles(opponent)

    positions = [pos for pos in estado.board if len(estado.board[pos]) > 1 ]

    # for each tower
    plyr_tower_avg = 0
    opp_tower_avg = 0
    for pos in positions:

        # ideal dominated tower has top == jogador and every other piece opponents
        opp_pieces = len(list(filter(lambda x: x == opponent, pos)))
        if estado.top_piece(pos) == jogador:
            plyr_tower_avg += opp_pieces / (len(pos) - 1)

        # ideal non dominated tower has every piece == opponent
        if estado.top_piece(pos) == opponent:
            opp_tower_avg += opp_pieces / len(pos)

    # average score of each type of tower, max is 1 best case for jogador
    plyr_tower_avg = plyr_tower_avg / plyr_doms
    opp_tower_avg = opp_tower_avg / opp_doms

    # strategic value of each group of towers
    plyr_tower_value = plyr_doms * plyr_tower_avg
    opp_tower_value = opp_doms - (opp_doms * opp_tower_avg)

    return plyr_tower_value - opp_tower_value



def capture_score(estado: EstadoFocus, jogador: str, opponent: str) -> float:
    # pieces of plyrs color that have been captured by the opponent
    capt_plyr = estado.captured[opponent]
    # pieces of opponents color captured by player
    capt_opp = estado.captured[jogador]
    if capt_plyr == 0 and capt_opp == 0:
        return 1
    return capt_opp - capt_plyr

In [23]:
EARLYGAME = 0.37
MIDGAME = 0.38

EARLYRES = 0.10
EARLYPOS = 0.43
EARLYCAPT = 0.47

MIDRES = 0.43
MIDPOS = 0.57
MIDCAPT = 0.0

LATERES = 0.36
LATEPOS = 0.50
LATECAPT = 0.14

DOMWEIGHT = 0.7
BASEWEIGHT = 0.3

In [24]:
# run_tournament([37, 75,     26, 32, 42,     26, 29, 45,    38, 28, 35,     50, 50])
run_tournament()

Profundidade 1
-37
Profundidade 2


KeyboardInterrupt: 

In [19]:

def run_tournament():

    # EARLYGAME = input[0]/100
    # MIDGAME = input[1]/100

    # EARLYRES = input[2]/100
    # EARLYPOS = input[3]/100
    # EARLYCAPT = input[4]/100

    # MIDRES = input[5]/100
    # MIDPOS = input[6]/100
    # MIDCAPT =input[7]/100

    # LATERES = input[8]/100
    # LATEPOS = input[9]/100
    # LATECAPT = input[10]/100

    # DOMWEIGHT = input[11]/100
    # BASEWEIGHT = input[12]/100

    counter = 0
    for p in range(1,3):
        def jogador_basicus_p(jogo,estado) :
            return alphabeta_cutoff_search_new(estado,jogo,p,eval_fn=func_basicus)

        def jogador_grupo_28(jogo,estado) :
            return alphabeta_cutoff_search_new(estado,jogo,p,eval_fn=func_28)
    
        print('Profundidade', p)
        quem_ganhou=0
    
        for i in range(50):
            resultado = jogo.jogar(jogador_grupo_28,jogador_basicus_p,verbose=False)
            resultado1 = -jogo.jogar(jogador_basicus_p,jogador_grupo_28,verbose=False)
        
            quem_ganhou += resultado + resultado1
        counter += quem_ganhou
        print(quem_ganhou)
        
    print(counter)
    # return counter
    


In [40]:
import pygad

function_inputs = [0.25, 0.5, 0.2222222222222222, 0.5, 0.375, 0.125, 0.36, 0.32, 0.32, 0.7, 0.3, 0.7, 0.3]
desired_output = 96
    
def fitness_func(ga_instance, solution, solution_idx):
    output = run_tournament(solution)
    # fitness = 1 - (desired_output - output)
    print(solution, output)
    return output


def on_gen(ga_instance):
    print("Generation ", ga_instance.generations_completed)
    print("Fitness ", ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1])

fitness_function = fitness_func



num_generations = 200
num_parents_mating = 4

sol_per_pop = 8
num_genes = len(function_inputs)

init_range_low = 0
init_range_high = 100

parent_selection_type = "sss"
keep_parents = 1
save_best_solutions=True

crossover_type = "single_point"

mutation_type = "random"

gene_type = int


ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_probability=0.15,
                       mutation_type=mutation_type,
                       random_mutation_min_val=1,
                       random_mutation_max_val=15,
                       gene_type=gene_type,
                       on_generation=on_gen,
                       mutation_by_replacement=True)

ga_instance.run()
ga_instance.plot_fitness()



Profundidade 1
165
Profundidade 2
264
[18 63 57 15 27 90 73 62 82 20 91  9 16] 429
Profundidade 1
202
Profundidade 2
258
[87  7 45 87 44  1 52 89 36 53 27 31 47] 460
Profundidade 1
189
Profundidade 2
219
[80 37 17 68 84 10 55 82  0 31 93 43 68] 408
Profundidade 1
185
Profundidade 2
211
[48 49 17 83 28  3 48 26 30 48 99 87 27] 396
Profundidade 1
170
Profundidade 2
218
[70 56 33 88 86 33 28  9 36 34 85 33 20] 388
Profundidade 1
183
Profundidade 2
191
[88 71 88 23 10 52 16 36 25 77 13 93  8] 374
Profundidade 1
169
Profundidade 2
202
[72 71 40 60 86 55  2 60 37 49 12 12 66] 371
Profundidade 1
189
Profundidade 2
249
[82 88 75 65 71 58 11 68  4 25 17 36 31] 438
Profundidade 1
188
Profundidade 2
257
[87  7 45 87 44  1 52 89 13 25  5 36 31] 445
Profundidade 1
170
Profundidade 2
212
[82  8 75 65 71 90 73 62 82 20  2  9 16] 382
Profundidade 1
193
Profundidade 2
225
[18  8 10 15 27 90 73 62 82 31  6 43 68] 418
Profundidade 1
162
Profundidade 2
228
[80 37 17 68 84  1 52 89 36  3 27 31 13] 390
Prof

KeyboardInterrupt: 

In [ ]:
ga_instance.run()[18 47 65 98 83 85 62 69 79 61 27 93 34] 81
[72 99 75 38 42 63 90 23  4 63 42  4 57] 86
[44 34 75 73 92 55  3 24 67 17 79 67 59] 73
[62 48 42 81 89 75 90  9 63 45 75 81 76] 86
[37 34  1 33 16 13 12 51 20  2 13 24 21] 92
[13 53 70 92 12 38 79  3 69 90 97 34 48] 95
[91 60 56 29 92 67 43 63 30 38 12 43 44] 74
[67 99 81 44 74 11 61  4 16 83 30 38 19] 74
[71 32 95 86 20 74 82 39 61 40 80 68 34] 83
[26 90 72 23 10 40 67 91 25 54 70 32 52] 76
[39 45 44 43 45 68 48 33 81 61 70 72 71] 106
[64 90 95 17  7  0 85 89 91 32 35 98 18] 104
[12 24 28 82  1 74  0 52  3 95 14 28 62] 76
[78 20  4 95 92 24 97 81 99 11 53 40 21] 84
[38 69 78  7 48 91 70 38 19 49 37 82 64] 87
[12 54 55 17 51 44  3 28 84 69 89 64 69] 81
[ 9 45 12 13 45 68 48 89 91 32 35 98 13] 97
[13 53 70 92  5 38 12  3 69 90 97 34 48] 71
[37 34  1  2 16 13 12 51  6  2 13 24 21] 87
[37  5  1 33 16 13  8 51 20  5  3 82 64] 94
[38 69 78  7 48 10 90  9 63 45 75 81 76] 105
[ 2 48 42 81 89  4 90  9 63 45 75  4 57] 74
[72 99 75 38 42 63 90  1  4 63  7 40  4] 108
[10 20  4 95 92 24 97 33 81  8 70 72 71] 98
[39 45 44 43 45 68  7 33 14 32 35 98  3] 69
[64 90 95 17  7  0 85 89 69 90 97 11 48] 92
[ 2 53 70 92 12 38 79  3 69  1 13 24 21] 100
[37 34  1 11  6 13 12 38 19 49 37 82 64] 73
[38 10 78  7 48 91 90  9 63 45  1 81 76] 67
[62 48 42 81 89 75 90 23  9 63 42  4 57] 73
[72 20  4 95 92 24 97 81 99 11 53 40  9] 74
Generation  1
Fitness  108
[72 99  2 38 42 63 90  8  4 63  7  1 71] 87
[39 45  6 43 45 68 48 33 81  7 70 72 76] 83
[38 69 78  7 48 10 90 11 69  1 13 24 21] 87
[ 2 20  4 95 92 24 97 33 81 10 70 72 71] 103
[10 20  6 95 45  4 48 89 10 32 35 98 13] 101
[ 9 45 12  6 45 68 48 89 91 32  3 82 12] 105
[10  5  1 33 16 13  8 51 69 90 97 11 12] 93
[64 90 95 17 42 12 90  1  4 63  7 40  7] 75
[72 99 75 38 42 63 90  1  4 63 70 72 71] 85
[39 45 44 43 45 68 48 33 63 45  4 81 76] 86
[38 69 11  7 48 10 90  9 63 45 75 81 21] 113
[ 2 53 70 92 12 38 97 33 81  8 70 72 71] 100
[10 20  4 13 45 11 48  3 13  9  3 98 13] 76
[ 9 45 12 13 45 68 48 11 91  5  3 14  6] 75
[37  5  1 17  7  0 85 89 69 90 97 11 48] 89
Generation  2
Fitness  113
[38 69 11 38 42 63 90  1  4 63  7 40  4] 85
[72 99 75 38 45 68 48 89 91 10  3 82 12] 88
[ 9 20  4  2 92 24 97 33  7 10  5 72 71] 87
[ 2 20  4 95 92 24 97 33 11 32 35 98 13] 76
[ 3 20  6 95 12  7  5 33 81  8 70 72 71] 76
[ 2 53  6 92 12 38 97 10 69  6 97 11 12] 91
[10 11  1 33 16  0 85 89 69 90 97 11 48] 89
[37  5  1 13  7  0 85  9 12 45 75 81 21] 94
[38 69 11  9 48 10  4  1  4 63  7 40  4] 88
[72 99 75 38 42  5 90  1  4 63  7 82 12] 85
[ 2  2  4 95 92 24 97 33 14 10 70 12 71] 80
[ 2 20  2  5 92 24 97 33 81 32  9 98 13] 81
[ 2 53 70 92 12 10 97 33  1  8 70 11 12] 88
[37  5  1 17  7  2 85 89 69 90 97 11 48] 74
Generation  3
Fitness  113
[38 69 11  2 48 14 90  9 63  4 70 72 71] 68
[ 2 53 70 92 12 38 97 33 12 45 75 81 21] 83
[37  5  2 12  7  9 97 10 69  6 97 11 12] 78
[ 2 53  6 92 12 38 97 10 11 90 97 11 48] 112
[10 11  1 33 16  0  5 89 69 90 70 11  6] 92
[ 2 53 10  4  2 10 97 33  1 63  7 40 14] 78
[38 69 11  8 45 68 48 13 91 10  3 82 12] 87
[72 10 75 38 45 10 90  2 63  3 75 81 21] 86
[ 6 69 11  7 48 11 90  8 63  7 70 72 71] 82
[11 53  6 92 12 38 97 10 69  6 97 11 12] 92
[ 2 53  6 92 12 38 97 10 69 90 97 11 48] 78
[10 11  1  4 12 10  4  5  2 11  9 11 12] 72
[ 2 53 70 13 12 10  9 33  1  8 70 40  4] 79
[38 69 11  9 48 10  2  7 91 10  3 82 12] 81
Generation  4
Fitness  113
[38  1 10  7 48 10 97 10 11 90 97 11  7] 95
[ 2 53  6 92 12 38 97  1 11 90  4 11 21] 86
[37  5  1 13  3  0 85  9  3 45 75 11 13] 103
[11 53  6 92 16  3  5 89 69 90 70 11  6] 86
[10 11 11  8 45 68 48 13 91 12 11 82 12] 100
[72 10 75 38 45 11 90  2 63  3 75 81 21] 86
[72 10 75 38 45 38 97 33 12 45 12 81 21] 95
[ 2 53 70 92 12 38 97 33 12  4 75 81 21] 64
[38 69 11  7 48 10 90  9 11 90 97 11 48] 88
[14 53  6 92 12 14 97 12 11 90 97 81 13] 86
[37  5  1  7  7  6 85  9  5  5 97 11 12] 88
[ 7 53  6  9 12 38  5 89 69 90 70 11  6] 64
[10 11  1 33  9  0  5 13 91 10  3  8 12] 79
[38 69 11  8 45 10 90  2 63  3 75 81  9] 86
[72 10  7 38 45 10 97 33 12 45 75 81 21] 109
Generation  5
Fitness  113
[38 69  7 38 45 10 13 33 12 45 75 81 21] 68
[72  4  8 38 45 10 85  9  3 45 13 11 13] 92
[37  7  1 13  1 68 48 13 91 12 11 82 12] 81
[10 11 11  8 45 68 48 13  9 12 12 81 21] 79
[72 10 75 38 45 10 97 10 11 14 97 11  7] 82
[38  1 10 14  7  6 12  9  5  5 97 11 12] 81
[37  5  1  7  7  6 85  9  5 90 97 11 48] 80
[38 69 11  4  1 10 90  9 63 45 75 81 21] 88
[38 69  7  5 48 10 90  9 63 45 75 81 21] 97
[72 10 11 38 45 10 97  9  1 45 75 13 13] 76
[13  5  4  8  5 68 48  2 91 10 11 82 12] 85
[72 10 75 13  7 12 97 33 12 45 12 81 21] 46
[10 10 75 38 45 38 97 10 12 90 97 11  9] 79
[38  7  1  7  7  8 85  9  5  5 97 11 12] 88
[37  5  1  7 48 10 90  9 11 90  2 11  5] 91
Generation  6
Fitness  113
[ 2 69 11  7 48 10 90  3 63 45 75  8 21] 87
[38 69  7  5 48  1 90  9 63 45 13 11 13] 84
[72  4  8 38 45 10  5  9  3 90  2 11  5] 64
[37  7  1 14  7  8 85  9  5  5 97 11 12] 100
[38  7  1  7  7  5 85  9  5  5 97 81 21] 83
[38 69 11  4  1 68 48  2 91 10 11  6 12] 111
[11  4  4  8 45 10 97  8 11 14 97 11  7] 91
[72 10  4 11 45 10 90  9 63 45 75 81 21] 113
[38 69 11  7 48  4 90  9 63 10 75 81 11] 100
[72  4  8 13 45 10 85  9  3 45 13 11 13] 77
[72 10  8 14 45 10 85  9  3 45 13 11  5] 88
[37  5  4  9 48 10 90  9 13  5 97 11 12] 80
[38  7 10  4  1 10 90  9 63 45 75  5 21] 67
[38 69 11  4  1 68 48  2 91 10 11 82 12] 54
[72 10 75 38 45 10 97 10 11 14 11 11  7] 97
Generation  7
Fitness  113
[72 10  4 12 45 10 90  9 63 45  3  6 21] 79
[12 69  9  9  1 68 48  2 91 10 11  6 12] 59
[13 69 11  2 48  4  1  9 63 10  5 10 11] 77
[38 69  6  7  7  8 85  9  5  5 97 11 12] 99
[37  1  1 38 45 10 97 10 11 14 11 11  7] 63
[72 10 75 38 45  1 97 13 11 14 97 11  7] 82
[11  4  4  8 45 10 97  8  3 45  1 11  5] 83
[72 10  8 14 45 10 90  9 63 45 75 81 21] 88
[38 69 11  4  1 68  5  2 91 10 11  6 12] 97
[38 69 11  4 11 68 48  2 91 10 75 81  6] 95
[38 69 11  7 12  8 85  9  5  5 97 11 12] 84
[72 10 75 38 45 10 97  8  7 14 97 11  7] 89
[11  4  8 14 45 10 85  5  3 45 13 10  5] 93
Generation  8
Fitness  113
[ 2 69 14 11 45 10 90  9 63 45 75 81  5] 70
[72 10  3 11 45 10 85  9  5  5 97 11 12] 85
[38 69 75 38 45 10  1 10 11 14 11 11  7] 69
[72 10  7 38 10 10  9 10 11 14 11 11 12] 86
[38 69 11  4  2 68  5  2 91 10 11  5  6] 85
[38 69 11  4 11 13  6  2 91 45 13 10  5] 84
[11 10 75 38 13 10 97  8  7 14 97 11  3] 86
[72  6 75 38 45 11 97  9 63 45 75 81 21] 46
[38 69 11  3 48 10 90  9 63 45 75 81 21] 87
[72  1 12 11 45 10  8  6 63  5 97  8 11] 76
[38 69  6  7  1  8 85  9  4 14 11 11  7] 110
[72 69 11  4  1 68  5  2 91 10 11  6 12] 85
[38 69 11  4  6 68 48  2 91 10 75 81  6] 86
[38 69 11 10 11 14 48 10 91 10 14 10  8] 75
[11  4  8 14 45 10 97  8  7 14 97 11 11] 99
Generation  9
Fitness  113
[10 69 11  7 48 10 90  9  4 14 11 11  6] 99
[38 69  6  7  1  2 97  8  7 14 97 11 11] 98
[38  5 11 12 48 10 90  9 63 45 12 81 21] 86
[38 69 11  3 48 10 90  9 63 10  9  5  6] 86
[38 69 11  4  6 68 48  2  7 14 97 11  3] 91
[11 10 75  5 13 10  8  8 11  3 11 11 12] 55
[72 10  7 38 10  9  5  3 91 10 11  6 12] 71
[72 69  2  7 48 10 90  9 63 45 75 81 21] 86
[38 69 11  7  1  8 85  9  4 14 11 11  5] 71
[38 69  6  7  1  8 85  9  7 14 97 11 11] 91
[11  4  8 14 45 10 97  8  7 14 97 81 21] 87
[38 69 11  4  6  4 48  2 91  5 75 81  5] 66
[38 69  9 38 13 10  9  8  7 14  8 11  3] 95
[11 10 75 12 13 10 13  8 11 14  3 11 12] 114
[72 10  7 38 10  5 11 10 91 10 11  6 12] 83
Generation  10
Fitness  114
[11 10 75 12 13 10 11  7 63 45  3 81 11] 104
[38 69 11  7 48 10  9  9  4 14 11  9  6] 87
[ 7 13 11  7 48 10  2  9  4 14 11 11 11] 103
[38  7  8  7 14  2 97  8  7 14 97 11  3] 108
[38 69  9 13 14  8 85  9  7 14 97 11 11] 50
[38  9 11  4  6 68  5  2  7 14 97 11  3] 89
[38  9 11  4  6 68 97  8  1 14 97 81 21] 93
[ 9  8 75 12 13 10 13  8 11 14  3 11 12] 88
[11 10 75 13 13 10  1  8 11 45 75  4 21] 73
[38 69 11  7 48 11 90  9  4 14 11 11  6] 88
[10 69 11  7 48  2 97  8  3 14 10  7 11] 93
[38 69  6 13  1  2  9  8  7 14  8 12  3] 71
[13 69  9  7  1  8 10  9  7 14 97 11  4] 88
[38 69  6  7 12 68 48  2  7 14 97  2  3] 98
[38  6 11  4  6 68 48  2  7 14 97 81 21] 86
Generation  11
Fitness  114
[11 10 75  9 14  2  5  8  7 14 97 11  3] 88
[11  7  8  7 14  2  5  8  3 45  7 81 11] 82
[ 5 10 75  1 13 10 13  9  3 14 11 11 11] 105
[ 7  8  6 12 12 68 48  2  7 14  6  2  3] 79
[ 6 69 11  8 48  2 97  8  3 14 10  7 11] 70
[38  9 11  4  6 68 11  8  1 14 97 81 21] 101
[38  9 11  4  6  2 97  8  1 14 97  2  3] 96
[38  9 11  4  6 68  5  2  7 14 97 11 12] 95
[13 10  8  7  8  2 97  8  7 14 97 11 12] 68
[38 13  8  7  8 10 11  7 63 45  3 81  9] 85
[11 10 11  7 48 10  8  9  4 14 11  8 11] 75
[ 7 13 11  7 48 10 48  2  7  5 97  2  3] 83
[38 13  6  7 12 68 48  2  7 14 97  2 10] 100
[13 69 11  4  6 68  7  1  1 14 97 81 21] 73
[38  9 11  4  6 68  5  2  7  8  1 11  3] 93
Generation  12
Fitness  114
[11 10 75 12 13 10 13  9  3 14 11 11 11] 93
[ 5  9 11  4  6 68 11  8  1 14 12 81 21] 72
[38  9 11  4  6 68 11  8  7 14 97 14 10] 60
[38 13  6  7 12  4 48  2  7 14 97  2  3] 86
[38  9 11  4  6  2 97  2  7 14 97 11 12] 96
[38  9 11  4  2 68  5  2  7 14  1 11  3] 98
[13  9 11  4  6 68  5  8  7 14 97 11  9] 94
[11 10 75  9 14  2  5  5  7 14  3 11 12] 106
[11 10 75 12 13 11 13  9  3 14 11 11 11] 75
[ 5 10 75  1 13 10 13  9  3  9 11 81  8] 68
[38  9 11  4  6 68 11  8  1 14 97  4 10] 81
[12 13  6  7 12 68 48  2  1 14 97  2  3] 65
[38  9 11  4  6  2  5  2  7 14 97 11 12] 88
[38  9 11  4  6 68  5  2  7  8  7 11  3] 77
Generation  13
Fitness  114
[11 10 75  9 14  2  5  2  7 14  1 11 11] 89
[38  9 11  4 11  2 97  2  7 14 97 11 12] 91
[38  9 11  4  6  2  5  8  7 14 97 11  9] 78
[ 1  1 11  4  6  5  5  8  7  8  1 11  3] 88
[38  9 11  4  6 68  5  2  3  4  1 11 11] 74
[11 10 11 14 13 10  6  2  7 14 97 11 12] 99
[38  9 11  4  6  2 14  2  7 14 12 11 12] 90
[11 10 75 12 13 10  3  8 11 14  3 11 12] 129
[ 7 10 75  9 14  2  5  5  7 14  3 11  3] 97
[38  9 11  4  2 68  5  2  7 14  1 11 12] 74
[38  9 11  4  8  2 97  2  7 14 97 11  9] 104
[38  9 11  4  6 68  5  2  7  8  6 11  3] 88
[38 10 75  7 13 10 13  9  3 13 11 11 11] 83
[11  8 75  4  6  2  5  2  7 14 97  6 12] 101
Generation  14
Fitness  129
[12 10 75 12 13 10 13  8 11  2  6 11 12] 72
[11 10 75 12  9  2  5  5  7 14  3 11 12] 107
[11 10 75  9 14  2  5  5  7 14 97 11  9] 99
[11  8 75  4  6  2 13  2  7 14 97  6 12] 87
[11  8 11 14 13  5  6  2  7 14  7 11 12] 84
[11 10 11 14 13 10 12  2  7 14 97  3  3] 100
[13 10 75  9  9  2 10  2  7 14 10 11 12] 78
[ 2  9 11  4 13 10  3  8 11 13  3 11 12] 85
[11 10 75 12 13 10 13  8 11 14  1 11  9] 83
[11 10 75 12 13  9 13  5  7 14  3 11 12] 80
[ 6 10 11  9 14  2  5  5  7 14 97 11  9] 80
[38  9 11  4  8 11 97  2  7 14  8  6 12] 90
[11 10 11 14 13 10  6  2  7 14  7 11 12] 88
[11 10 11 14 13 11  6  2  7 14  3 11  3] 125
[ 7 10 11  9 11  2 97  2  7 14 97 13 12] 103
Generation  15
Fitness  129
[11 10 75 12 13 11  6  2  7 14  3 11  3] 74
[11 10 11 14 13 11  6  3  7 14  3 11 12] 86
[11 10 11  9 11 10 97  2  7 14 97 13 12] 93
[ 7 10 11  9 11  2 97  2  7 14 97  3  3] 95
[11 10 11 14 13 10  5  5  7 14 97 11  9] 87
[11 10  5  9 14  2 97  2  7 14  8  6 12] 117
[38  9 11  4  8 11  6  2 14 14  7 11 12] 91
[11 10 11 14 13 10  3  8 11 14  3 11 12] 86
[ 6  6 75 12 13 10  3  8 11 14 11 11  3] 81
[ 9 10 11 12  9  2  5  7  3 14  3 11 12] 81
[ 1 10 11  9  2 10 12 10  5 14 97  3  3] 72
[11 10 11 14 13 10 12  2  5 14 97 11  9] 90
[11 10  1  9  8 11 97 12  7 14  8  6 12] 76
[38  9 11  4  8 11 97  2  7 14  7 11 12] 95
Generation  16
Fitness  129
[11 10 75 12 13  5  3  8 11 14  3  3 10] 81
[11  7  5  9  6  2 13  2  7  8  8 13 12] 61
[ 7 10 11  9 11  2 97  2  7  6 10 11 12] 72
[38  9  6  4  8 11 97  5  7 14  7 11  3] 97
[ 7 10 11  9 11  7 97  2  7  7 97 13 12] 78
[11 10 11  9 11 10 97  2  7 14  7 11 13] 62
[38  9 11 14 13 10 12  2  5 14  9 11  9] 98
[11 10 11 14 13 10 12  2  5 14  2 11 12] 77
[11 10 75 12 13 12 97  2  7 14  4  6 12] 90
[11 10 11 14 11  2 97  2  7 14 14 13 12] 71
[ 7  3 11  9 11  2 97  2  7 12 14 11 12] 107
[38  9 11  9 11  2 97 14  7 14 97  3  1] 70
[ 7 10 11  9 11 10 97  2  7 14 97 13  7] 98
[11 10 11  9 11 11  6  2 14 14  7 11 12] 88
[38 10 11 14 13 10 12  2  5  5 97  3 14] 75
Generation  17
Fitness  129
[11 10 14 12 13  6  3  8 11 14 14 11 12] 82
[ 7 10 11  9 11 10 97  2  7 14 13 13  2] 82
[ 7 10 11  9 11 10 12  2  5 14  9 11  9] 94
[38  9 11 14 13 10 12  5  7  1  7 11  3] 106
[38  9  6  4  8 11 97  5  7 14  4  6 12] 82
[11 12 75 12 11 11  6  2  9 14  7 11  1] 82
[11 12 75 12 13  9  3  8 11 14  3  3 10] 78
[11  2 75 12 13  5  3  8 11 14  3 11 12] 85
[11 10 75 12 13 10 97  2  7 12 13 11 12] 85
[ 7  3 11  9 10 11 97  2  7 12 97 13  7] 84
[ 7 10 11 14 13 10 12  2  5  3  9 12  9] 93
[38  9 11 14 13 10 97  5  7 14  7 11  5] 84
[12  9  6  4  8 11  6  5  7 14  7 11 12] 94
[11 10 11  9 11 11  2  2 14 14  7 11 10] 84
Generation  18
Fitness  129
[11 10 75 12 13 10 12  5  6  1  7 11  3] 99
[38  9 11  2 13 10 12  5  7  1  7 11  2] 95
[12  9  6  4 11 10  2  2 14 14  9 11  9] 91
[ 7  1 11  5 11 10 12  6  5  3  9  7  9] 68
[ 7 10 11 14 13 10 12  2  5  3  9 12 12] 71
[ 3 10 75 12  8  9 97  2  7 12 13 11 12] 67
[11 10 75 12 13  5  3  8 11 14  3 11 12] 106
[14  2 75 14 13 10  3  8 11 14  3 11 12] 78
[ 3 10 75 12  1 10 12  8 11  1  7 11 14] 79
[38  9 11 14 13  7  9  5  7  7  7 11 11] 79
[12  9  6  4  6 11  6  5  7 14  9 11  9] 94
[ 7 10 11 14 13  8  1  2  5  3  9 12  9] 76
[ 7 10 14  9 11 14 10  2 14 14  7 11 12] 87
[11 10 75 12 13  5  3  8 11 14  3 11 12] 77
Generation  19
Fitness  129
[11 10 12 12 13 10  3  8 11 14  3  9 12] 80
[11 10 75 13  9  5  3  8 11  1 12 11  3] 90
[10 10 75  2 13 10 12  5  7  1  7 11  2] 115
[38  9 11  2 13 10 12  5  6 14  9 13  9] 86
[12  9  6  4 11 10  2  2 14 14 10 11  9] 77
[12  9  6  4 11 10  2  2 14 14  9 11 14] 84
[11 10 11  1 11  1 10  2  1 14  7 11 12] 87
[ 7 10 75 12 13 10  3  8 11 10  3 11 12] 103
[11 10 75 12  8 10 12  5  6  1  7 11 11] 81
[11 10 75  4 13 10  9  5  7  9  7  4  2] 86
[38  9 11  5  6 11  6  5  7 14  9 11  9] 97
[ 3  9  6  4 11 10  2  2 14 14  9 11  9] 75
[12  9  1  4 11 10  6  8 14 14  7 11  9] 73
[11 10 11  9 11 13  6  2 14  6 11 11 12] 110
Generation  20
Fitness  129
[ 9 10 75  2 13 10 12  5  4  1  7 11  2] 74
[10  3 11  9 11 10  6  2 14  6 11 11 12] 106
[11 10 11  9 11 10  3  8 11 10  3 11 12] 80
[ 7 10 75 12 13 11  6  5 10 14 10 11  9] 82
[38  9 11  5  9  5  3  8 11  1 12 11  3] 86
[11 10 75  1 11  5 10  6  1 14 13 11 12] 88
[11 10  5  1 11  1  6  2  1 14  3 11 12] 99
[11 10 75  7 13 10  3  8 11 14  4 11 12] 73
[11  9 75 12 13 10 12  5  6  1  7 11  5] 88
[10 10 75  2 11 13  6  2 14  9 11 11 12] 88
[11 10 11  5 13 10  3  8 11 10  3 11 12] 79
[ 6 10 75 12  2 14  6  5  7  2  9  6  3] 57
[38 10  6 13  9  5  3  8 11  1 12 11  3] 84
[11 10 11  1  2  1 10  2  1 14  7 11 12] 71
Generation  21
Fitness  129
[11 10 75 12 13 10  3  9 14  6 11 11 12] 90
[ 4  4 12  9  8 11  6  2  1 14  3 11 12] 101
[11 10  5  2 11 13  6  2 14  9 11 11 11] 66
[10 10 75  2 11 10 12  5  6  1  7 11  5] 80
[11  9 75 12 13 10 12  5  6  1  7 11 12] 92
[11 10 75  1  5  5  3  8 11  1 12 11  3] 97
[38  9 11  5  9  5  3  8 11  1  3 11 12] 89
[11 14 75 12 13 10  3  8 11 14  3 11 12] 92
[11 10 75 12 13 10  6  2 14  6 11 11 12] 93
[11 10  5  1 11  1  6  2  1 14  3  2 12] 66
[11 10  5  1 11 13  6  2 14  9 11 11  1] 82
[10 10 75  2 11 13  6  2 14  9 11 11  5] 93
[11  9 75 12 11  5 10  6  1 14 13 11 12] 92
[13  4 75  1 11  6 10  6  1 14 12 11  3] 65
Generation  22
Fitness  129
[11 10 75 12 13 10  3  8  4 14  3 11 12] 68
[11  9 75 12 13 10 14  8 11 14  3 11  8] 57
[ 4  4 12 10  6  5  3  8 14 11 10 14  3] 90
[11 10 75  1  5  5  6  4 14  9 11 11  5] 81
[ 8 10 75  3 13 10  6  6 14  6 11 11 12] 87
[11 10 75 12 13 10  3  2  1 14 13 11 12] 70
[11  9 75 12 13  2  3  8 11 14  3  4 12] 90
[11  1 75 12 10 10  3  8 11 14  3 11 12] 109
[11 10 75 12  7 10  3  8 11 14  3 11 12] 73
[11 10 75 12 13 10  6  2  1 14  3 11 12] 103
[10  4 12  2  8  5  3  8 11  1 12 11  3] 82
[10  2 75 13  5  5  6  2 14  9 11 11  5] 96
[10 10 75  7 13 10  6  1  2  6 11 11 12] 91
[11 10 75  8 13 10 10  6  1 14  6 11 12] 95
[11  9 75 12 11  5 10  6  1 14 13 11 14] 88
Generation  23
Fitness  129
[11 10 75 12 11 10  1  8 11 14  3 11  8] 98
[11  1 75 12 13 10  6  2  1 11  3 11 12] 85
[11 10 75 12 13 10  6  2 14  9 11 11  5] 87
[10  2 75 13  5  5  6  2 14  9 11  9 12] 99
[11 10 75  9 13 10 10  6 13  6  5 11 12] 87
[12  9 75 12 13  2 11  8 11 14  3  4 12] 71
[ 4  4 12 10  6  5  3 11 14 11  7  1  3] 84
[ 4  4 13 10  6 10  3  8 11 14  3 11 12] 68
[ 7 10 75 12 13 10  3  8 11  8  3 11 12] 97
[11  1  8 12 13 10  6  1  8 14  3 11 12] 66
[11 10  7 12 13 10  6  2 14  9 11  5  7] 87
[10  2 75 13  5  5 10  6  1 14  6  8 12] 92
[ 2 10 75  8 13 10 10  6 13 14  6 11 12] 77
[ 5 10 75  6 13  2  3  8 11 14  3  4 12] 75
[11  9 75 11  3  2  3  8 11 11 10 14  3] 95
Generation  24
Fitness  129
[ 9 10 75 13  5  5  6  2 14  9 11  9 12] 72
[10  2 75 13  5 10  1  8 11 14  3 11  8] 68
[11 10 75 12 11 10  3  8 11  8  3 11 12] 81
[ 9  9 75 11  3  2  3  8 11 11  9 14  3] 122
[11 14 75 11  3  2  3  8 11  1  9  8 12] 65
[10  2 75 13  6  5 10  6 14  9 11  5  7] 91
[11 10  7 12 13 10  6  6 13  6  5 11 12] 89
[12 10 75  9  3 10  3 13  5 14  3 12  3] 87
[11 10 75 12 13 10  3  2 14  9 11  9 11] 78
[10  2 75 12 11 10  1  8 11 14 12 11  8] 88
[11 10 75 12 11 10  1  8 11 14  3 11 12] 82
[ 7 10 75 12 12 10  3  8 11 11 10 14  3] 92
[11  3 75  2  5  5 10  6  1 14  6  8 12] 98
[10  2 11 13  5  5  7  2 14  9 11 14  7] 81
[11 10 11 12 13 10  6  4 13 13  5 11 13] 106
Generation  25
Fitness  129
[11 10 75 12  6 10  3  8 11 14  3  2 14] 72
[ 9  3 75 11  3  2  3  4 10  9 13  5 13] 79
[11 10 11  1 13 10  6  4 13 14  6  8 12] 80
[ 8 10 75 12  1 10  3 11  4 11 10 14  3] 76
[ 9 10 75 12 12  5 10  6  2  9 11  4  7] 84
[10  2 75 13  6  5 10  6 14  5  5 11 12] 113
[11 10  7 12 13 10  6  6 12 12 12 11  8] 72
[10  2 75 13 14 10  5 11 11 14 12  8 12] 96
[11 10 75 10 13 10  3  8 11 14  3 14  3] 73
[ 9  9 75 11  3  2  3  8 13 13  5  3  9] 78
[11 10  9 12 13 10  8  4 13 13  5 11 12] 78
[11  9 75 12 12 10  3  8 11  1 10 14  9] 79
[ 6 10 75  2  6  5 10  6 14  9 11  5  6] 85
[10  2 75 13  6 10  6  6 13  6  5 11 12] 103
[11 10  2 12 13 10  6  6 13  6 12 11  8] 87
Generation  26
Fitness  129
[11  6  1 12 12 10  3 12 11  5  5 11 12] 59
[10  2 75 13  5  1 10  6 13  6  2 11 12] 82
[10  1 75 13 14 10  5 11 11 14 12  8 12] 101
[10  2 75 13 14 10  6  6 13  6 12 11  8] 67
[13 10  2  5  6  5 10  6 14  9 11  5  6] 103
[ 6 10 75  2  6  5 11  2  2  9 11  4  7] 96
[ 9 10 11  1 10 10  6  4 13 14  6  8 13] 81
[11 10 12  1 11 10 10  4 13 14  6  2 12] 99
[10  2 75 13  6  5 10  9 14  5  7 11 12] 102
[ 4 14 75 13  6  5 10  6 13 13  5 11 12] 81
[10  2 75 13 14 10  5 11 11 14 12  8  8] 92
[10  2  2 12 13 10  6  6 13  6 12 11  8] 78
[ 6  7 75  2  6  5 10 14 14  9 11  5  6] 98
[ 6 10 75  2  6  5 10  4 14  9 11 13  7] 81
[ 9 13 75 12 12  5 10  4 13 14  6  8 12] 88
Generation  27
Fitness  129
[11 10 75 12 13  5 10  6 14  9 11  5  6] 80
[13  4  2  5  6  9 10  9 14  5  7 11 12] 89
[10  1 75 13 14 10  5 11 11  5 12  8 12] 72
[10  1 75 13 14 10  5 11 11  2 12  1 12] 87
[11 10 12 13 11  6 10  4 13  9 11  5  6] 92
[ 6  7 75  2  6  5 10 14  5  9 11  4  7] 91
[ 6  2 75 13 14 10  5 11 11 14  5  8  8] 70
[10  2 75  6 14 10  5 11 11 14 12  8  7] 74
[12 10 75  5  6  5 10  6  2  9 11  5 12] 100
[ 4 10  2  5  6  5 10  9 14  4  7 11  5] 86
[10  2 75  5  6  8 10 11 11 14 12  8 12] 86
[12  1 12  1 11 10 10  8 13 14 13  2 12] 75
[11 10 12  1 11 10 10 14 14 13 11  5  6] 77
[ 6 10 75  2  6  5 11  2  2 12 11  4  7] 86
[ 6 10 75  2  6  5 11 13 11 14 12  8  8] 75
Generation  28
Fitness  129
[11 10 75 12 13 10 10  5  2  3 11  5 12] 84
[12 10 75 13 11  6 10  4 13  9 11  5  6] 99
[11 10 12 13  6  5 10 14  5  9 11  4  7] 116
[13  4  2  5  6  9 10  9 14  2 12  1 12] 100
[10  1 75 13  9 10 11  2  1 12 11  4  7] 97
[ 8 10 75  5  6  8 10 11  4 14 12  8 12] 96
[10  2 75 10  6  8  3  8 11 14  3 11 12] 86
[12 10 75  5  6  8 10  6  4  9 11  5 12] 80
[12 10 75  5 11  3 10  4 13  9 11  5  6] 88
[11 10 10 13 11  6 10  4  5  1 11  4  7] 91
[ 6  7 75  2  6  5 10  7 14  5  7  4 12] 75
[13  4 75 13 14 10  5 11 11  2 12  1 12] 76
[10  1 75  9 14 10  5 11 11  2 11  4  7] 103
[ 6 10 75  2  6  5 11  2  2  2 11  8 12] 76
Generation  29
Fitness  129
[11  4 75 12 13 10  3 14  5  9 11  4  7] 93
[10  4 75 12  6  9 10  9 14  2 12  1 12] 76
[13  4  2  5  6  9 10  4 13  9 11  5  6] 91
[12 10 75 13 11  6 10  4 13 12 11  8  7] 78
[10  1  6  8  9 10 11  2  1 12 12  8 12] 67
[ 8 10  9 10 11  6 10  4  5  1 11  4  7] 84
[13 10 13 12 13 10  3  8 11 14  3 11 12] 44
[11 10  8 12 13 10  3  8 11  9 11  4  7] 83
[11 12 12 13  6 10  5 11 11  2 11  4  7] 70
[10  1 75  9 12 10  5 11  9  2  7  4 12] 76
[13  4  2  5  6  9 10  9 13  9 11  5  6] 77
[12 10 75 13 11  6 10  4  8  9 11  5  4] 87
[10  1 75  5  6  9 10 11  4 14 12  8 12] 98
[ 8 10 75  5  6  8 10 11  4 14 11  7 14] 82
Generation  30
Fitness  129
[11 10 12 12 13 10  3  8 11 14 13 13  7] 68
[11 10 12 13 14  5 10  2  5  9 11  4 12] 96
[10  1 75 13 13 10  3 12  5  9 11  4  7] 100
[11  4 75 12 13 11  3 14 11  9 11  5  6] 86
[12 10 75 13 11  6 10  4  8  2 11  5  4] 77
[12 10 75 13 11  6  5  4  5  1 11  4  7] 101
[11 10  4 12 13 10  3  7 11  9 11  4  7] 83
[11 10  8 12 13 10  3  8 11 14  3 11 12] 75
[11 10 75 12 13 10  9  8 11 14  3  4  7] 77
[ 9 10 12 13  6 10 10 14  5  9 11  8 12] 89
[11  4 75  3 13  8  3 14  5  9 11  4  7] 75
[11  4 75 12 13 10 10  4 13  9 11  5  6] 95
[13 10  2  5  9  2 10  4  8  9  5  5  4] 86
[12 10 75 13 11  6  8  4  5  1 11  4  7] 76
[ 8 10  9 10 11 10 11 14 11 13  6  4  7] 85
Generation  31
Fitness  129
[11 10 75 12  3 10  3  8  5  1  8  4  7] 85
[12 10 75 13 11  6  5  4  5 13 11  4  7] 89
[ 8  9 75 13 13 10  3 12  5  4 11  4 12] 95
[11 10 12 13 14  5 10  1  5  9 11  4  6] 101
[11  4 75 12 13 10  2 14  5  9 11  8 12] 104
[ 7 10 12 13  6 10 10  3  8  9  5  5  4] 90
[13  2  2  5  9  2 10  4  8  9  7  5  6] 72
[ 2  4  8 12 13 11  3 14 11  9  3 11 12] 100
[11 14 75 11 11  6  8  4  5  1 11  4  7] 90
[12 10 75 13 11 10  3 12  5  9 11  4  7] 61
[10  1  3 13  4 10  3 12 10  2 11  4 12] 94
[11 10 12 13 14  7 10  4 13  9 11  5  6] 104
[11  4 75 12 13 10 10  4 13 14 11  4 12] 92
[ 9 10 12  7  6 10  1 14  5  9  5  5  4] 88
[13 10  2  2  9 11 13 14 11  9 11  5  7] 87
Generation  32
Fitness  129
[ 1 11 75 12 13 10  2 14  5  9 11  6 12] 73
[11  4 75 12 13 10  2  1  6  9  9  4  6] 74
[11 10 12 13 14  5 10  1  5  9 11  4 12] 75
[11  5  8 14 13 10  3 12  5  9 11  4 12] 80
[ 8  9 75 13 13  6  3 12 10  2 11  4 12] 86
[10  1  3 13  4 10  3 12 13 14 11 11 12] 67
[11  4 75 12 13 10  3  8 11 14  3 11 11] 94
[11 10 75 12 13 10  3  8 11 14  3 11  6] 113
[11 10 12 13 14  7  2 14  5  9 11  8  8] 91
[11  8  8  7 13  1  3 14 11  9  3 11 12] 96
[ 2  4  8 12 13  8  3 12  5  4 11  4 12] 79
[ 8  9 75 13 13 10  3 12 10  2 11  4 12] 81
[10  1  3 13  4 10  3 12 10  2  3  4 12] 88
Generation  33
Fitness  129
[11 10 75 12 13 10  3  8 11  3  3 11  6] 62
[11 10 75 13  7  1 10  4  1  9 11  5  6] 78
[11 10 12 13 14  5 10  1  5  9 11 10 14] 92
[11 14 12 13 14  5 10  1  5  9  3 11 12] 84
[11  4 10 12 13 10  3  8 11 14  3 11 11] 106
[11 13  5 12 13 10  3  8 12 14 11  8  7] 81
[11 10 12 13 14  7  2  1  7  2 10  4 12] 103
[ 9 10 75 12 13 10  3  8 11  8  3 11  2] 88
[11 10 10 12 14  7 10  4 13  9 11  5  6] 106
[11 10 12 13 14  7 10  4 13  9 11  4  6] 91
[11 10 12 13 14 14 10 12 11  9  3 11 12] 76
[11  8  8 12 13 10  3  8 11 14  3 11 11] 96
[11  4  9 12 13 10  3  8 11 14  3 11  8] 84
[11 10 10 13 14  7  2  4  5  9 11  8 12] 93
Generation  34
Fitness  129
[11  4 75 12 14  7 10  1 13 14 11  5  6] 74
[11 10 10 12 14  7 10  4 13  9 11 10 11] 79
[11  4 10 13 14  7  2  1  7 14 10  4 12] 85
[11 10 12 13 13 10  3  8 11 14  3  4 11] 75
[ 8  8  8 12  3 10  3  8 11 14 11  5 12] 87
[11 10  1 13 14  5 10  1  5  9 11 10 14] 66
[11  6  6 13 14  5 10  3 13  9 11  4  6] 86
[11 10 75 10 13 10  3  8 11 14  3 11 12] 105
[11 10 75 12 14  7 10  4 13  9  9 11  6] 88
[11 10  6 12 14  7  4  4 13 14  3 11 11] 92
[11  4 10 13 14  6  2  1  7  2 10  4 12] 90
[11 10  7 13 14 14 14  1  7  2 10 11 11] 76
[ 1  8  8 12 13 10  3 12  5  9 11  9 12] 64
[11 10 10 13 14  5 10  1  5  9 11 10 14] 90
[11 10 12 13 14  5  5  4 13  6 11  4  6] 90
Generation  35
Fitness  129
[ 2 10 75 10 13 11  3  8 11 14  3 11 12] 73
[11 13 75 10 13 10  3  4 13 14  3 14 11] 111
[11 10 12 13 14  5  5  4 12  6 11  4  6] 80
[11 10 12  6 14  5  1  4  1  6 11  4 14] 83
[11 10 10 13  3 11 10  1  5  9  6  2 12] 65
[11 11 10 13 10  6  2  3 13  6  9 11  6] 79
[11 10 75 12 14  7  3  8 11 14  5  3 12] 88
[ 8  8  8 12  3 10  3  8 11 14 11 11 12] 67
[11 10 75 12 13 10  9  8 11 14 14 11 12] 94
[11 10  6 12 14  7 14  4 13 14  3 11 12] 96
[11 10  6 12 14  4  4  4  8  6 11  4  4] 97
[11 10 14 13 14  5 10  1  5  9  7 10 14] 95
[11  4 10 13 14  6  2 10  7  2 10  1 12] 102
[11  4  1 13 14  6  2  1  7  2 10  4  6] 102
[11 10  8 13  3 10  3 11 11 14 11  6 12] 92
Generation  36
Fitness  129
[11 10 75  4 13 10  3  4 13  2  3 14 11] 80
[11  1 75 13 13  6  2  1  7  2 10  4  6] 84
[11  4 11 13 14  6  2  8  7  2 10  7 12] 76
[11  4 10 13 14  6  4  4  8  6 11  4  4] 92
[11 10 11  4 14  7 14  4 13 14  7 12 12] 70
[ 8 10  6 12 14  7 14  4 13  9  7 10 14] 89
[ 6 10 75 13 13 10  9  8 11 14 14 11 12] 96
[11 10 75 12 10 10  9  8 11  8 14 11 12] 79
[11 10 75 12  3 10  3  4 13 14  3 14 11] 92
[11 13 75 10 13 10  3  4  2 14 10  4  6] 67
[11  4  1 13 14  6  2 10  7  4 10  1 12] 70
[11  4 10 13 14  6  2 10  8  6 11  4  4] 78
[11  1  6 12 14  4 14  4  5 14  3 11 12] 84
[11 10  6 12 14  7 10 10  5  9  7 10 14] 81
[ 5 10  2 13 14  5  9  8 11 14  9 11 12] 50
Generation  37
Fitness  129
[11 10 75 12 13 10  3  8 11 14 14 11  7] 100
[ 6 10 75 13  9 10  9  4 11 14  3 14 11] 83
[11 10 75  8 14  6  4  5  8  6 11  4  4] 66
[ 8 10  1 12 14  7 14  4 13  9 13 10 14] 84
[11  1  6 12 12  4 14  4  5 14  3  4 12] 90
[11  1  6 13  4  4  2 14  7  7 10  4  6] 72
[ 6  1 75 13 13  6  2  1  7  2 10  4 14] 101
[ 9 10  6 12 14  7 10 10  5 14  3 11 12] 80
[11 10 75 12 13 10  3  8 11 14 14 11 12] 95
[ 6 11 75 13 13  3  9  4 13 14  3 14 11] 108
[11 10 13 14  3 10  4  4  7  6  7  4  4] 85
[11 10 10 12 14  7 14  6 13  9  7 10  7] 70
[ 8 10  6  5 14  4 14  4  5 14  3 11 12] 77
[11  1  6 12 14  4 14  4  5 14  3 11  8] 97
[11  1 75 13 13 13  2  1  7  9 10 13 14] 61
Generation  38
Fitness  129
[11 10 75 14 13  3  9  4 13 14  8 14  6] 81
[ 6 11 75 13 13  3  9  4 13 11 10  4 14] 79
[ 6  1 75 13 13 10  5 10 11 10 14 11  7] 100
[11 10 75 12 10 10 14  4  3 14  3 11  8] 84
[11  1  6 12 13 10  3  8 11 14 14 11 12] 101
[ 5 10 75 12 13 14  3  6 11 14  3  4 12] 88
[11  1  6 12 12  4 14  4  5 14  3  4  4] 86
[11  9 75 12 13 10  3  8  2 14 12 11 12] 101
[ 6 11 75 13 13  3  9  4  5 14  3 14 11] 66
[ 6  1  7  5 13  6  2  1  7  2 10  4 14] 75
[ 6  1 75 13 13  6  2  8 11 14 14 11  7] 97
[11 10  8 12 13  7  3  8 11 14  3 11  8] 65
[11  1  6 12 14  1 14  4 11 14 14 11 12] 90
[11 10 75 12  5 10  3  8  8 14 14  4 12] 65
[11  1  6 12 12 10  4 13  7  6  7  4  4] 85
Generation  39
Fitness  129
[11 10 75 12 13  6  3  6 12 14 12  6 12] 106
[11  9 75 12 13 10  3 13  2 14 14 11  6] 69
[11  1 14 12 13 10  3  8 11  6 14 11  7] 72
[ 9  1 75 13 13  7  5  8 11 14  4 11  7] 77
[ 6  1 75 13 13  6  2  8 11 14 14 11 12] 87
[11  1  6 12 14  1 13  6 11  4  3  4 12] 89
[11  1  6 12 12  4  8  4  5 14  3  4  4] 58
[11  1  6 12 13 10  3  8 11 14  3  3 12] 104
[11 10 75 12 13 10  3  8  2 14 12 11 12] 112
[11  9 75 12 13 10  3  8 11 14 14 11 12] 72
[12  1  7 12 13  6  3  8 11 14 14 11  7] 84
[ 6  1 75 13 13 10  5  8 11  3 14 11  7] 81
[ 6  8  4 13 14 12 14  6 11 14 11  9 12] 81
[ 5 10 75 12 13 14  3  6 11 14 14  2 12] 101
[ 5 10 75 12 13 14  3  4  3 10  3  4  4] 83
Generation  40
Fitness  129
[ 3 10 75  1 13 10  3  8  2  4  6 11 14] 83
[11 10 75 12 13 10  4  8  2 14 12  6  9] 92
[11  1  6 12 13 10  3  8 11 14 10  3 12] 98
[11  1  6 12 13 14  3  6 11 14 14  2 12] 85
[ 5 10  6 12  8  1 13  6  8  4  3  4 12] 80
[11  1  8 12 14  6  2  8 11 14 14 11 12] 56
[ 6  1 75 13 13  6  3  8 11 14 14  3  7] 73
[12  1  7 12 13  6  3  8 11 14 14 10 12] 65
[11 10 75 12 13 10  3  8  2 14 12  3  6] 94
[ 3 10 75 12 13 10  3  6 12 14  5  6 12] 101
[ 5 10 75 12 13  6  3  8 11 14  3  5 12] 89
[ 5 10 13 12 13 14  3  6  2 14 14  2 12] 82
[ 5 10 75 12 13 14  3  6 11  1  5  4 12] 76
[11  5  6 12 14  1 13  2 11  5 14 11 12] 96
[ 6  1 75 13 13  6  3  8 11  5 14 11  7] 87
Generation  41
Fitness  129
[11 10 75 12 13 10  3  8 11 14  5  6 12] 78
[ 3 10 75 12 13 10  3  8 13 14 10  3 12] 91
[11  1  6  7  4 10  3  8 11 14 10 11 10] 104
[11  5  6 12 14  1 13  2 10 14 12  5  6] 116
[11 10 75 12 13 10  4  8  2 14  2  6  9] 91
[14 10 75  8 13  6  3  4 11 14  3  5 12] 83
[ 4 10 75 12 13  6  3  8 11 14  3 11  7] 76
[ 6  1  4 13 13  6  3  8 11 14  3  9 11] 89
[11 10 75 12 13 10  2  6 12 14  5  6 12] 68
[ 3 10  6 13 13 10  3  8 11 14 12  3  1] 71
[11  1  9 12 11 10  3  8 11 14 10  3  6] 89
[11 10 75 12 13 10  5  8  2 14 12  3  6] 73
[11  1 75 12 13 10  3  8  2  6 12  6 10] 83
[11 10 75 12 13 12  3  8 11 14  3  5  8] 98
[ 5 10 75 12 13  6  3  8 11 12 14 11  7] 59
Generation  42
Fitness  129
[11 10 75 12 13  8  3  8 10 14 12  5  6] 84
[11  5  6 12  4 10  3  8 11 11 10 11 10] 119
[11  1  6  7  4 10  9  8 11 14  3  5  8] 69
[11 10 75 12 13 12  3  8 11 14 11 13  9] 93
[ 3 10 75 12 13 10  3  7 13 14 10  8 12] 86
[ 3  9 12 12 11 10  3  8  8 14 10  3  6] 87
[11 14  4 13 13  6  3  8 11 14  3  9 11] 99
[ 6  1  4  9 13 12  8  8 11 14  3 11 12] 90
[11 10 75 12 11 13 13  9 10 14 12  5  6] 94
[11  5  3 12 14  8  3  8 11 14 10 11  3] 74
[ 7  1  6  7  4 10  3  8 11  9  3  5 11] 88
[11 10 75  2 13 12  3  8  2 14  2  6  3] 95
[ 1 10 75  2 13  2  3  1 13 14 10  3 12] 91
[ 3 10 75 12 11 10  3  8 11 14 10  3  6] 94
[11  1  9 12 11 14  3  8 11 14 11  9 11] 71
Generation  43
Fitness  129
[11 10 75 12  4 10  3  4 11 11 10 11 13] 101
[ 4  5  6 12 14  7  3 14 11 14  3  9 11] 85
[11  2  4 13 13  6  3  8  2 14  2  6  3] 83
[11 10 12  2 13 12  3  8 11 14 10  3  6] 71
[ 3  6 75 12 11 10  3  8 11 14 10  5  6] 80
[11 10 75 12 13 12  3  8 11 14 10 13  9] 79
[ 1 10 75  2 13  2 13  1 13 14 10  3 12] 64
[ 1 10 75  2 13  2  3  8 11 14  3 11  5] 77
[11 10 75  7 13  8  3  8  4  7  3 11 10] 102
[11  5  6 12  4 10  3  8 11 11 10  9 11] 84
[11 14 75  2 13 12  3  8  2 14  2  6  3] 100
[11 10 75  1 13 12  3  8  2 14  2  6  6] 76
[ 3 10 75 12 11 10  3  8 11 14 10  5  6] 86
[11 10 75 12 11 13  9  9  2 14 12  5  9] 88
[11 14 75 12 13 12  3  8  9 14 11  3 12] 105
Generation  44
Fitness  129
[11 10 14 12 10 10  3  8  9 14 11  3 12] 82
[11 14 75 12 13 12  3  8  9  7  3 11 10] 76
[12 10 75 12  4 10  3  4 11 11 10 11 13] 85
[11 10 75 12  4 10  3  4  3 14  5  6  3] 91
[11 14  2  2 13 12  3  9  2 14 12  5  9] 93
[11 10 75 12 11 10  3  8  3 14 10  5  6] 82
[ 3 10 11 12 11 10  3 10 11 14  3  9  1] 82
[ 2  5  6 12 14 10  3  8 11 14  3 11 12] 93
[11 10 75 12 13 10  3  8 11 14 11  3 12] 93
[11 14 75 10 13  8  3  8  4  7  3 11 10] 92
[11 10 75  1  4 10  3  4 11 11 10 10 13] 89
[11 10 75 12 13 12  3  8  2 14  2  6  3] 93
[11 14  5  2 13 12  3  8  2 14 12  5  9] 98
[11 10 75 12 11 10 10  8  5 14 10  5  6] 87
[ 4  5  6 12  5  7  3 14 11 14  3  9 11] 86
Generation  45
Fitness  129
[11 10 75 12 13 10 10  8 11 14  3 12  9] 84
[11 14  4  5 13 12  3  8  2 14  2  9  3] 98
[ 2  1 75 12 13 10  3  8 11 14 11  3 12] 77
[11 10 75 12 13 10  3  8 11 14 13 11 12] 88
[ 2  5  6 12 14 10  3  8 11 14  3  5  5] 74
[11 14  2  8  9  3  3  4  4  8  3 11 10] 95
[11 14 75 12  4 10  8  4  3 14  5  6  3] 87
[11 10 75 12 13  9  3  8 11 14  3 11 12] 97
[11 10  5  2 13 12  5  8  2 14 12  5  9] 90
[11 14 12  2 13 12  3  8  2 14  2  6  1] 87
[11  7 75 12 13 12  3  8  2 14 11  3 12] 88
[11  5  6 12 14 10  3  8 11 14  3 11 12] 84
[11 14  2  2 13 12  3  1  2 14 12  5  9] 102
[14 14  2  2  3  6  3  9  2 14 12 13 10] 76
[ 6 14 75 10  4 10  3  2  3 14  2  6  3] 92
Generation  46
Fitness  129
[11 10 75 12 13  4  3  1  1 14 12  5  9] 103
[11 14  2  2 13 11  3  8  2 12  2 11  3] 101
[11 14  4  5 13 12  3  8  6 14  3 11 12] 97
[11 10 75 12 13  9 12  8 13  8  3 11 10] 85
[11 14  9  8  9  3  5  4  4  5  2  6 10] 87
[ 6 14 75 10  4 10 12  2  3 14  2  6  9] 82
[11 10  5  2 13  3  3  8  3 14 11  3 12] 71
[11  7 75 12 13 12  3  8 11 14  3 11 12] 41
[11  5  2  1 13 12  3  1  2  1 12  9  9] 80
[11 14  2  2 11 12  3  1  2  6  2  9  3] 90
[11 14  4  5 13 12  3  8  2 14  2 11 12] 115
[11  7 75 12 13  9  3  6  4  2  3 11 10] 77
[11 14  2  1  9  3  3  7  4 14  2  6  3] 76
[ 8 14 75 10  4 10  5  8  2 14 12  5  9] 79
[11  3  5  2 13 12  5  9  2 14  6  3 12] 97
Generation  47
Fitness  129
[11 10 75 12 13 12  3  8  2 14  2 11 12] 86
[ 8 10 75 12 13  4  1  1 10 14 12  6  9] 106
[11 10 75 12 13  4  3  8  2 12  2 11  3] 102
[14 14  2  2 13 11  3  9  2 14  6  3 12] 92
[ 5  3 12  2 13 12  3  8  6 14  3 12 12] 92
[11 14  4  5 13 12  3  4  6  4  3 11  3] 93
[11 14  2  3 11 12  3  1  2  6 12  6 10] 96
[11 10 75 12 13 10  3 12 11 14  9 11 12] 87
[11 10 75 12 13 10  3  8 11 11  7 11 12] 95
[11 10 75 12  7  4  3  1  1 14 12  3  9] 86
[11 10  2  2 13 11  3  3  2 12  2 11  3] 71
[11 14  2  2 13 11  3  8  2 12  2  3 10] 89
[11  3  3 10 13 12  5  9  2  9  6 11 12] 92
[11 14  4  5 13 12  3  8  6  6  2 11  3] 88
[ 5 14  4  2 11 12  3  5  2  6  2  6 10] 96
Generation  48
Fitness  129
[11 10 75 12 12  5  3  8 11 14  7  6  9] 98
[ 7 10 75 12 13  4  2  1 10 14  2 11  3] 87
[11 10 75 12 11 12  3  5  2  6  2  6  3] 80
[ 5 14  4  2 11 12  3  5  6  6 12  6 10] 105
[11 14  2 12 13 10  3  8 11 11  7 11 12] 91
[ 9 10 75 12 13 10  3  4  6  4  3 11  3] 82
[11 14  4  5 13 12  5  9  2  9 11 11 12] 76
[11 10 75 14 13 10 10  9 11 13  3 11 12] 102
[11 10 75 12 13 10  3  8 11 14  9  6  9] 64
[ 8 10 75 13 13  4  3  8  2 12  2 12  3] 72
[11 10 75 12  2  4  3  8  2  6  2  6 10] 79
[ 5 14  2  3 11  4  3  1  7  6  1 14 10] 76
[11 11 75 12 13 10  3  8  7 11  7 11 12] 98
[11 10  4  5  9 12  3  4  6  4  3  4  3] 66
[11 14  4  5 13 12  3  4  6 11  3 11 12] 95
Generation  49
Fitness  129
[ 5 14  4  2 11 12  3  5  1 13  1 11 12] 88
[11 11 75 12 13 10  3  8  7  6  7  6  9] 99
[11 10 75 12 13  5  3  8 11 14  7 11  2] 81
[11 14  4  5 13 10  3  8  4 11 10 11 12] 89
[11 14 75 12  3  4  2  1  2 14  2 11 10] 74
[ 7 10 75 12 13  4 11  8 10 14  3 11 12] 72
[11 10 75  9 13 12  3  5  6  6 12  6 10] 84
[ 9 10 75 14 13 10 10 11 11 13  3 11 12] 68
[11 10 75 14 13 10 10  9  8 13  3 11 12] 89
[11 11 75 12 13  3  8  8 11 14  7  6  9] 76
[11  1 75  7 12  5  3 13 11 14  3  2 12] 89
[11 14  4  9 13 12  3  4  9 11  2 11 12] 98
[11 14  2 12 13 10  3  1 10 14  2 11  3] 90
Generation  50
Fitness  129
[11 10 75 12 13 10  3  8  4 14 12  6  9] 99
[ 5 14  4  2 11 12  3  9 11 13  3 11  7] 98
[11 10  4 12 13  2  3  8  7  6  7 14  9] 108
[11  2  4 12 13  4  3  4  9 11  2 11 12] 88
[11 14  2 12 13 13  3  1 10 14 14 11  3] 89
[11  2  2  7 12  5  3 13 11 14  3  2 12] 93
[14 14 75  7 12  5  3 13  4 14  2  2 12] 97
[11 10  5 12 13 10  3  8 11 14  3 11 12] 101
[11 10 75 12 13 10  3 12  6  2 12  6 10] 73
[11 10 75 14 13  3 10  9 11 11  7  6  9] 102
[11 11 75 12 13 10  3  8  7  6  7 11 12] 87
[11 14  4  5 13  2  3  4  9 11  2 11  3] 80
[11 14  2  7 12  5  8 13 11 14  3  2 12] 97
[11  1 75  7 12  5  3 13 11 14  7 11 12] 93
Generation  51
Fitness  129
[11 10 75 12 13 10  3  8 11  6  6 14  9] 102
[11 10  6 12 13 10  9  8  7  6  7 14  9] 96
[11 10 75 14 13  4 10  5 11 13  3 11 12] 86
[11 10  8 14 13 10 10  9 11 13  3 11 12] 103
[11 10  5 12 13 10  3  8 11 14 12  6  6] 89
[11 10  3 12 13 10  3  9 11 13  3 11  7] 78
[ 5 14  4  2 11  5  3  1 11 14  3  2 12] 54
[11 14  2 12 14 10  3  8 11 14  3 11 12] 80
[11 10 75 12 13  7  3  8  7  6 12 14  9] 74
[11 10 75 14 13  3  5 13 11 11  7  6  9] 100
[11  4 75 14 13  3 10  5 11  8  7  8 12] 78
[11 10 75  6 13 10 10  2 11 13  3 11 12] 93
[11 10  5 12 13 10  3  8 11 14  3 11  9] 97
[11 10 75 12 13 12  3  9 11 13  3 11 14] 91
[14 14  4  2 11 12  3 13 11 14  3  2 12] 80
Generation  52
Fitness  129
[ 1 10 11 12 13 10  3  8 11  5  3 14 12] 71
[11 10  8 14  1 10 10  9 11 13  3 11  5] 79
[ 4 10 75 12 13  3  5 13 11 10  7  6  9] 67
[11 10 75 14 13  3  5 13 11 11  7 11  9] 68
[12 10  5 12 13 10  2  8  4  6  7 14  9] 80
[11 10 14 12 13 10  9  8  7  6  3 11 12] 73
[11 10 75  6 10 10 10  2 11 13  3  9 14] 87
[13 10 75 12 13 13  3  9 11 13  3 11 12] 89
[11 10  8 14 13  7 13 12 11 13  3 11 12] 97
[ 8 10  2  9 14 10 10  8 11  6  6  6  9] 71
[11 10 75 14 13  3  5  8 11 14  9 11  9] 87
[ 7  6  5 12 13 10  9  8  7  6  7 14  9] 104
[11 10  6 12 13 10  9  8  7 13  3 11 12] 87
[11 10 75 12 13 12  3  9  2 13  3 11 14] 100
Generation  53
Fitness  129
[11 10  4 12 13 10  3  8 11 14  3 11  9] 114
[ 7  6  5 12 13 10  3  8 11  6  6 14  9] 93
[11 10 75 12 13 10  3  8 11  6  6 14 14] 94
[11 10 75 12 13  5 13 12 11  4  3 11 12] 67
[11 10 12 12 13 13  3 13 11  3  3 11 12] 87
[13 10  6 12 13 10  9  8  7 13  3 11 12] 107
[ 3 10  6 12 13 10  5  8  9 14  9 11  9] 107
[11 10 75 12 13 10  9  8  7  6  7 14  9] 94
[ 6 10 75 12 13 10  3  8 11  6  6 14  9] 64
[11 10 75 12 13 12  3  9  2 13  3 11  8] 78
[11 10 75 12 13 12 13 12 11 13  3 11 12] 111
[11 10  8  5 13  5 13 12 11 13  3 11 12] 75
[13 10 75  8 13  3  3  9 11 13  3 11 12] 81
[11 10  6 12  1 10  9  8 11 13  9 12  9] 101
Generation  54
Fitness  129
[11 10 75 12 13 10  3  8 11 14  3 10 12] 82
[11 10 75 14  4 10  3  7 11 14  3 11  9] 87
[11 10  6 12 13  9  3  8 11 14  3 11 12] 92
[11 10 75 12 13 10  5  8  9 14  9 11  9] 83
[ 3 10  6 12 13 10  9  8  7 13  3 11 12] 70
[13 10  6 12 13 10  9  8  7 13  9 12  4] 94
[11 10  6 12 13 10  9  8  7  6  7  4  9] 98
[ 4 10 75 12 13 10  9  8 11  8  3 11 12] 91
[11 10 11 12 13 14  3  8 11 14  3 11 12] 83
[11 10 75 12 13 10  3  8 11 11  3 11  9] 97
[11 10 11 12 13 12 13 12 11 13  3  1 12] 98
[11  4  8 12 13 12 13 12 11 13  3  1  9] 69
[ 3 10  1 12 13 10  9  8  7 13  3 11 12] 78
[13 10  6 12 13  4 12  8 11 13  9 12  9] 102
[11 10  6 12 13 10  9  8  7 11  7 14  9] 84
Generation  55
Fitness  129
[11 10 75 12 13 10  3  8 11 13  9 12  9] 87
[ 4 12  6 12 13  4 12 12 11 13  3  1 12] 90
[11  7 11 12 13 12 13 12 11 13  7  4  9] 82
[11 10 75  1 13 10  9  8  7 13  9 12  4] 95
[11  8  6 12 11  9  3  8 11  7  3 11 12] 72
[11 10  6 12  9 10  9  8 11  8  3 11 12] 90
[ 4 10 75  9 13 10  9  8 11  8  2 11 12] 91
[11 10 75 12 13 10  5  8  6 13  9 12  9] 78
[13 10  6 12 13 12 14 12 10 13  3  1 12] 93
[11 10 11 12 13 12 13 12 11  6 14  4  9] 83
[11 10  6 12 13 10  9  8  7 11 13 11  9] 83
[11 10 75 12  7 10  3  8 11 13  9 12  4] 96
[13 10  6 12 13 10  9  8  7 14 14 11 12] 83
[11 10  6 12 13  9  3  7 11 14  3 11 12] 93
Generation  56
Fitness  129
[11 11 75  1 13  3  3  8 11 11  3 11  9] 66
[11 10 75  5 13 10  3  8 11  2  2 12  4] 95
[11 10 75 12 13  6  9 10  7 13  9 12  4] 94
[11 10 75  1 13 10  9  7 11 14  3 11 12] 120
[11 10  6 12  1 12 14 12  7 13  3  2 12] 78
[13 10  6 12 13 12  9 10 11  8 10 11  1] 102
[ 4 10 75  9 13 10  9  8 11  8  3 13 12] 112
[11 10  6 12 13 10  3  8 11 14  3  2 12] 76
[11 10 75  8  8 10  3  8 11 11  3 11 13] 84
[11 10 75 12 13 10  3  8 11 13  9 12  4] 55
[ 1 10 75 12  7 10  9  8  7 13  9 12  4] 86
[11 10 75  1  1 10  9  8  7 14  3 11 12] 72
[11 10  1 12 13  9 13  7 11 14  3 11 12] 82
[ 4 10  2 11 13 10  9 13 11  7  3 14 12] 100
Generation  57
Fitness  129
[11 10 75 12 13 10  9  7 11 14  3 11 12] 72
[11 10 12  9 13 10  9  8 11  8  3 13 12] 87
[ 4 10 75 12 13 12  9 11 11  8 10  6  1] 84
[13 10  6 12 13 12  9 10 11  8  3 14  8] 97
[ 4 10  2 11 13 10  3 13 11  7  3 12  4] 57
[11 10 75  5 13 10  1  8  7 13  9 12  4] 79
[11 10 75  9 13 10  9  8 11  8  2  7 12] 87
[ 5 10 75  9 13 10  9  8 11  8  3 11 12] 76
[11 10 75 12 13 10  9  7 11 14  3 11 12] 87
[11 12 75  1 13 10  9  7 11 14  3 13 12] 81
[ 4 13 12  9 13 10  9  8 11  8 10 11  1] 73
[13  7  6 12 10 12  9 13 11  7  3 14 12] 95
[ 4 10  2  4 13 10  9  2  9  4  2 12  4] 71
[11 10 75  5 13 10  3 10  7 13  9 12  4] 81
[11 10 75 12 13 10  9  8  8  8  2 11 12] 87
Generation  58
Fitness  129
[11 10 75 12 13 11  3  8 11  8  3 14  8] 89
[13  2  6 12 10 12  9 13 11  7  3 10 12] 88
[10  7  6 12 13 10  9  8  8  8  2 11 11] 83
[11 10 75 12 13 10  9  8 11 14  3 11 12] 79
[11  3 75  9 13 10  9  8 11  8  2  7 12] 67
[10 10 75  9 13 10  9  8 11  8  3 13 12] 96
[ 1 10 12  9 13 10  9  8 11 10 10  6  1] 86
[ 4 10 75 12 13 12  9 11 11 14  3  4  1] 95
[11 10 75 12 13 10  3  8 11 14  3 11  8] 79
[13 10  7 12 10 12  9  6 11  7  3 14 12] 94
[13 10 75 12 13  5  9  8  8  8  2  3 12] 103
[ 7 10 75 12 13  6  9  9 11 14  3 11 12] 94
[ 4 10 75  9 13 10  9  8 11  8  3 13 12] 98
[11 10 75 12 13 12  9 13  9  8 10  6  1] 80
Generation  59
Fitness  129
[11 10 75 12 13 10  9  8  8  8  2  3 12] 87
[13 10  7 12 13  5  9  8  8  8 11  7 12] 87
[ 4 10 75  9 13 10  9 12 11  8  3 13 12] 76
[10 10 75  9 13 10  9  7 11 14  3  4  1] 104
[ 3 10 75  6 13 12  9  9 11 14  3 11 10] 101
[ 7 10  7 12 10 12  9 11 12  7  3 14 12] 67
[13  9  7 12 10 12  9  6 11  8  3  7  8] 66
[11 10 12 12 13 11  3  8  2 14  3  4 12] 96
[11  8 75 12 13 10  3  8 11 14 11  3 12] 83
[ 3 10 13  2 13  5 10  8  8  8  3 13 12] 101
[ 9  1 75  9 13 10  9  8 11  8  3 13 12] 86
[10 10 75  9 13 12  9 11 11 14  3  4  1] 103
[ 4 10 75 12 11 12  9 11 11 14  3 11 12] 80
[ 7  6  7 12 13  6  8  6 11  7  4 14 12] 80
[12 10  5 12 10 12  6 12 11  7  3 14  8] 89
Generation  60
Fitness  129
[11  6 75 12 13 10  9  7 11 14  3  4  1] 89
[ 6 10 75  9 13 12  9 11 11 11  3  4  1] 69
[ 6 10 75  9 13 12 13  8  8  8  3 13 12] 101
[ 3 10 13  2 13 12  9  9 11 14 11 11 10] 89
[ 3 10 12 12 13 11  3  8  2 14  3  4 12] 106
[11 10 12 12 13 11  3 12 11  7  3 14  8] 81
[12 10  5 12 13  5  9  8  8  8 11  7 12] 90
[13 10  3 12 13  5  3  8 11 14  3 11 12] 83
[11 10 75 12 13 10  3  8 11 14  3  4  1] 99
[10 10 75  6 13 13  9 11 11 14  3  4  1] 110
[10  6 75  9 13  5 10  8  8  8  3 13 12] 119
[ 1 10 13  2 13  5 10  8 11 14  3  6 10] 96
[ 3 10  8  6 13 12  9  9 11 14  3  4 12] 101
[11 10 10 12  6 12  6 12 11  7  3 14  8] 90
[13 10  7 12 13  5  3  8  8  8 11  2 12] 101
Generation  61
Fitness  129
[ 4 10 75  4  3 10 14  8 11  8  3  8 12] 72
[10 10 75 13 13 13  9 11  4 14  3  4  1] 80
[ 3 10 13 12 13 11  3  8  2  7  3  4 12] 61
[ 3 10 12 12  4 11  2  8  2 14  3  4 12] 79
[ 8 10  8  6 13 12  9  9 11 14  3  4 12] 76
[ 3 10  9  6 12 12  9  9  8  8  3 13 12] 82
[ 6 10 75  9 13  8 12  8 11 14  3  4  1] 87
[11 11 14 12 13 10  3  8  2 14  3 11 12] 87
[11  9 75  9 13  5 10  8  8  8  3 13 12] 71
[10  6  8  9 13  5  9 11 11  3  3  4  1] 92
[10 10 75  6 13  2  3  8  2 14  3  4 12] 76
[ 3 10 12 11 13 11 11 10  8  8 11  2  6] 80
[13  1  7 12  2  5  3  4  8  8  3  4 12] 93
[ 3 10  8  6 13 12 13  8  8  8  3 12  6] 97
[ 6 10 75 12 13 10  3  8 11 14  7  4  1] 95
Generation  62
Fitness  129
[ 4  8 75 12 13 10  3  8  8  8  3 12  6] 66
[ 3 10  8  6 13 12 13  8  8  8  7  4  1] 72
[ 6 10  7 12  2  5  3  4  8  8  3  5 12] 91
[13  4  7 12 13  5  9 11 11  3  3  2  1] 75
[10  6 14 12 13 10  3  8  1 14  3 11 12] 63
[11 10 75  9  9  8 12  8 11 14 11  4  1] 95
[ 6 10 75  9 13  8 12  8 11 14  3  4 12] 99
[ 3 10 75 12 13 10  3  8 11 14  8 11 12] 71
[11 10  5 12 13 10  3  8 11 14  3 12  6] 97
[ 6 10  9 12 13 10  3  8 11 13  7  4  1] 94
[ 6 10 75 12  2  5  3  4  8  8  9  4 12] 100
[13  1  7 12  2 10  9 11 11  3  3  7  1] 102
[10  6  8  9 13  5  7 11  1  3  3 11 10] 88
[11 11 14 10 13 10  3  8  2 14  3 11  1] 82
[ 6 10 75  9 13  8  9  7  8  8  3 13 12] 82
Generation  63
Fitness  129
[11 10 75 12 13 10  3  5 10  3  3  7  1] 92
[ 6 10 75 12  2 13  3 10  8  8  9  4 12] 63
[ 6 10 75 12  2  5  3  4  8 14  3  4 12] 92
[ 6 10 75  9 13  8 12  7 11 14  3 12  6] 65
[11 10  5  9  9  8 12  8 11 14 11 14  4] 85
[11 10 75  9  9  3 12  8 11 14 11  4  1] 101
[ 6 10  8 12  2  5  3  4  8  8  3  5 12] 94
[11 10 75 12 13 10  3  8 11  9  3 11  9] 83
[11 10  4 12 13 10  9 11  8 10  3  7  1] 76
[ 6 10 75 12  2  5  3  4  8  8  9  4  5] 87
[ 7 10 75 12  2  5  3  4 11 14 13  4 12] 52
[ 6 10 75  9 13 14  3  8 11 14  3 12  6] 71
[11 10  5 12 13 10  3  8 11 14 11  4  1] 78
[11 10 75  9  9  8 12  8  3 14 11  4  1] 87
[ 6 10  9 12 13  4  3  4  8 12  3  6 12] 85
Generation  64
Fitness  129
[13 10 75 12 13 10  3 14  4 14 11  4  1] 63
[ 2 10 75  9  9  3 12 12 11  8  3 11 12] 83
[ 6 10  7 12  2  5  3  4  8 14  3  7 12] 87
[ 6 10 75 12  2  5  3 14 10  3  3  7  1] 91
[11 10 75 12 13  8 12  8  3 14 11  4  5] 84
[14 10 75 12  2  5  3  4  8  8  9  4  5] 81
[ 6 10  9 12  2  5  3  4  8  8  9  1 12] 99
[ 6 10 75 12 13 10  3  8  9 14  7 11 12] 81
[11 10 75 12 13 10  3  6 11 14 11  4  1] 96
[11 10 75  9  9  3 12  8 11 14  3  5 12] 68
[ 6 10  8 12  2  5  9 14  8  8  3  4 11] 87
[ 4 10 75 12  2 12  3  5 10 11  3  7  1] 81
[11 10 75  9  9  8 12  8  9 14 11  4  1] 93
[11  4  1  9  9  4  7  4  8  9  9  9  5] 96
[ 5 10 75 12  2  5  3  4  8 12  3 13 12] 75
Generation  65
Fitness  129
[11 10  9 12  4  5  3  4  8  8  9  1  2] 87
[ 6 10  9 12 14  5  3  4  8  8  9  9  5] 97
[11 12 75 12 13 10  3  6 11 14  8  4  1] 76
[11 10 75 12  1  3  3  6 11 14 11  4  1] 68
[11 10 75 12  2  5  3 14 10  3 10  7  1] 68
[ 6 10 75 12  6  5  3 14 10  9  3  1 11] 84
[ 6 10  8 12  2  5  3  4  8  2  3  7 12] 80
[ 6 11  7 11  2  5  3  8 11 14  3  1 12] 70
[11 10  9 12  2  5 11  1  8  8 13  1 12] 100
[ 6 10  9 12  2  5  3  4 11  8  9  7  5] 79
[ 5  4  1  9 11 10  3  6 11 14 11  4  6] 91
[11  6 75 12 13 10  3 10 11 14 11  4  1] 82
[11  2 75  9  9  8 12  8  7 14 11  7  1] 106
[ 6 10 75 12  2  5  9 14  8  8  3  4 11] 73
[ 6 10  8 12  2  5 11 14  8  8  3  7 12] 79
Generation  66
Fitness  129
[11 10 75 12 13  5  3  8  7 14 11  7  1] 97
[11  2 75  9  9  8 12  5  7 14 11  1 12] 97
[11 10  9 12  2  5 11  1  8  8 13  1  5] 62
[ 6 10  9 12 14  5 14  6  8 13 11  4  6] 77
[11 10  9 12  4  9  2  4  8  8  5  1  7] 88
[ 6 11 75 12  6  5  3 14 10  9  3  1  1] 96
[11  1  8 12 13 10  3  8 11 14  5 11 12] 71
[11 10 75 12 13 10  3  8 11  6 11 10  1] 79
[11  2 75  9  9  8  6  8  8  5 13  6 12] 91
[11  1  9 12  2  5 11  4  8  8  9  9  5] 78
[ 6  1 14 12 14 11  3  4 11 14 11  4  6] 84
[ 5  4  1 12  4  5  3  4  8  8 13  1  2] 80
[11 10  9 12  4  5  3  4  8  8  3  1 11] 92
[11  6 75 12 13 10  3 10 11  4 11  4  1] 88
Generation  67
Fitness  129
[ 9 10 75 12 13 10  3  8  1  9  3 11 12] 92
[11  2 75  9  4  8 12  5  7 14 11  7  1] 53
[11 10 75 12 13  5  3  1 10  6  3  1  1] 100
[ 6 11 75 12  6  5  3  4  8  8  8  1 11] 80
[11 10 75  9  9  8  6  8  8  5 13  6 12] 103
[11  2  3  9 13  7  3 10  8  4 11  4  1] 79
[11  6  9 12  4  9  9  4  8  8  5  1  8] 75
[11 10  9 12  4 12  2  4  8  8  5  4 12] 85
[10 10 75  9  8  8 13  6  7 14 11  1 12] 81
[11  2 75 12 13  3  3  8  7 14 11  7  1] 60
[11  5 75 12  6  5  3 14 10  9  3  1  1] 82
[10 11  9 12  6  5  1 14 10  9  3  1 11] 54
[11  2 75  9  9  8  6  6  8  5 13  2 12] 64
[11  5 75  9  9  8  6  8 11 13 11  4  9] 68
[11 10  9 12  4  9  2  4  8  8  5 10  7] 76
Generation  68
Fitness  129
[11 10 75 12  9  8  6  8  8  5 13  6 12] 84
[11 10 13  9  9  8  6 11  3  6  3  1 11] 76
[ 9 10 75  7 13 10  3  4  3  8  5  4 12] 98
[11 10  9 12  4 12  2  3  8  8  5  4  1] 99
[11  5  8  4  8  8 13  6  7 14 11  1 12] 83
[10 10 13 12  6 10  3  4  8  8  8  1 11] 72
[ 6 11 75 12  6  5  3 14 11 14  4 11 12] 104
[11 10 75 12 13 10  9  8  1  5 13  6 12] 91
[11  7 75 12 13  3  3  1 10  6  8  1  1] 67
[11 10 75 12 13 10  3  8  1  9  8 11 12] 68
[11  4  9 12  4 12  2  4  8  8  8  4 12] 94
[11 10  9 12  4  6  2  4  8  8  5  1  1] 84
[11  2 75 12  8  8 13  6  7  9  7  1 12] 87
[10 10 75  9  8 14 13  6  7 14  8  3  7] 94
Generation  69
Fitness  129
[11 10 75 12  6  5  3 14 11 14  4 11 12] 73
[ 6 11 75 12  4 12  2  3  8  8  5  4  1] 90
[11 10  9 12  4 12  2  4  3  8  5  4 12] 73
[ 9  1 75  7 13 10 13  6  7 14  8  3  7] 74
[10 10 75  2  8 14 13  6 12  8  2  4  8] 75
[11  4  9 13  4 12  2  4  1  9  3  9  3] 62
[ 9 10 75 12 13 10  9  8  1  5 13  6 12] 82
[11 10 75 12 13 10  9  8  1 14  3 11  8] 94
[11 11 75 12  6  5  3 14 11 14 12 11 12] 86
[ 6 11 75 12  4 12  2  3  8  8 10  4  3] 80
[ 1  4 75  7 13 12  3  4  3  8  5  4 12] 88
[10 10 75  9  8 14 13  6  7 14  4  3  7] 78
[10 10 12  9  8 10 13  4  9  8  8  4 12] 95
[11 14 75 12 13 10  3  8  1  9  4 11 12] 70
[ 9  6 75 12 13 10  3  8  1  5 13  2 12] 80
Generation  70
Fitness  129
[11 10 75 12  9 10 13  4 11  8  8  4 12] 63
[10 10 12  9  8 10 13  4  9  8  3 11  8] 114
[ 2 10 75 12  4 12  2  3  8  8  5  4  1] 65
[ 1  4  3  7 13 12 10  4  3  8  5  4 12] 75
[ 1  4 75  7 13 12  3 13  3  8  5  6 12] 90
[11  7 75 13 13 10  9  8  1  5  6  6 12] 72
[ 9 10 75 12 13  3  9  8  1  5 13  6  8] 86
[ 9  6 75 12 13 10  3  8 11 14  3 11 12] 67
[11 10 75 12 13 10  3  8 11 12  3 11 12] 82
[10 10 12  9  8 10 13  2  9  8  9  4 12] 85
[11 13 75 12 13 14  2  3  8 10  5  7  1] 84
[ 6  5 75 12  4 12  2 12  3  8  5  4 12] 73
[ 1  4 75  7 13 12  3 14 11 14 12 11 12] 98
[11 11 75 12 13 10  9  8  1  5 13  6 12] 91
[ 9 10 75 14 13 10  3  8  3  5 13  2  6] 86
Generation  71
Fitness  129
[11  5  9 12 13 10  3  8 11 14  3 11  8] 73
[10 10 12  6 13 12  3 14  1 14 12 11 12] 86
[ 1 14 75 10 13 11  3 14 11 14 13  6 12] 69
[11 11 75 12 13  9  9  8  1  5  5  6 12] 76
[ 1  4 75 14 13 10  3  8  1 14 13  2  6] 75
[ 9 10 75 14 13 10  9  8  1  5  1  9  8] 82
[ 9 11 75  3  8 10 13  2  9  8  9  4 12] 105
[10 10 12  9 13 10  3  8 11 14  3 11 12] 71
[11  8 75 12 13 10 13  4  9  6  3 11  8] 70
[ 1  4 75  7  3 12  3 14  7 14 12 11 12] 100
[ 1  1 75  7 14 10  9  8  1  5  5  6 12] 80
[11  5 75 12 13 10  9  8  1  5 13  6 12] 100
[10  6 75  7  2  3  3 10  3  5 11  4  6] 82
[ 9 10 75 14 13 10  3  8  3  5 13  2  8] 83
[ 9 10  2 12 13  3 10  8  9  8  9  4 12] 79
Generation  72
Fitness  129
[11 10 75 12 13 10  3  8 11 14  3  4 12] 64
[ 9 11 75  3  8 10 13  2 11  8  5  4 12] 84
[11  5 75 12 13 10  9  8  1  5 12 11 12] 87
[ 1 13 75  7 13  3  3 14  1  1  6 11 12] 81
[10 10  2  6 13 12  3 14  1 14 13  2  8] 80
[ 9 12 75 14 13 10  3 10  3  5 11  4  6] 93
[10  6 13  7  2  3  3 10  3  5 11  4 10] 81
[11 10 75 12 13 10 10  8 11 14  3 11 12] 105
[11 10 75 12 13 10 13  2  9  8  9  4 10] 88
[ 9 11 75  3  8 10 13  2  9  5 13  6  3] 88
[11  5 75 12 13 10  9 14  7 14 12 11  5] 64
[ 6  4 75  7  3 12  3 14  7 14 12 11 12] 101
[10 10  3  6  5 12  3  8  1  5  6  2  8] 85
[ 9 10 75  9 13  3  3 10  3  5 11  4  6] 69
[10  6 75  3  2  3  3 10  3  5 11  9  8] 89
Generation  73
Fitness  129
[ 8 10 75 12 13 10 10 11 11 14  3 11  1] 81
[13 10 75 12 13 11  3 14  7 14 12 11 14] 80
[ 6  4 75  7 13 10  3 10  4 13 11  4  4] 95
[ 9 12 75 14 13  3  3 10  3  5 11  9  8] 94
[10  6  5  9  8 10 13  2  9  5 13  6  3] 80
[ 9 11 75  6  8  5 13  2  9  5 13  6 10] 97
[11 10 13 12 13  8  9  8  1 13 12 11 12] 112
[11 10 75 12 13 10  3  8  4 14  3 11 12] 84
[11 10 75 12 13 10 10  8 11  5  3 11 12] 85
[ 1 10 75 12 13 10 10  8 11 14  3 11 12] 81
[ 6  4 75 14 13 10  3 10  3  5 11  4  6] 107
[ 9  7 75 14 13 10  3 10  3  5 11  9  8] 74
[ 9 11 75  3  8 10 13  2  9  5 13  6 11] 87
[ 9 11 75  3  8 10  5  2  7  8  9  4 10] 99
[11 10 75 12 13 10 13  2  9  8  9 11 12] 97
Generation  74
Fitness  129
[11  1 75 12  2 10  3 13 11 14  3 11 12] 89
[11 10 13 12 13  7  9  8 12 13 12 11  6] 67
[10 11  6  3  8 10  5  2  7  8  9  4 10] 76
[ 9 11 75  3  8 10  5  2 10  8  9 11  4] 84
[11 10  9 12 13 10 13  2  9  5 13  6  7] 92
[ 9 11 75  6  8  5 13  2 10  5 13  4  4] 88
[ 6  4 75  7  7 10  3  2  3  5 11  9  8] 94
[ 9 12  3 14 13  3  3 10  3  5  3 11 12] 88
[ 2 10 75 12 13 10  3  8 11 14  3 11 12] 82
[11 10 13 12 13  8  9  8  1 13 12 11  2] 99
[ 6  4  3 14 13  2  5  2  4  8  9  4 10] 95
[ 9 11 75  3  8 10  5  2  7  8  9 11 12] 98
[11 10 75 12 13 10 13  2  6  8 13  6 10] 66
[ 9 11 75  6 10  5 13  2  9  5 13  4  4] 95
[ 6  6 14  7 13 10  3  6  3  5 14  9  8] 78
Generation  75
Fitness  129
[11 10 75 12 13 10  3  8 11 13 12 10  2] 71
[11 10 13  5 13  9  9  8  7  8  1  8 12] 89
[ 9 11 75 14  4  2  5  3  4  8  9  4  2] 86
[ 6  4  3 14 13  2  5  8  4  8  9  4  8] 81
[ 6  4 75  2  7 10  3  2  3  5 13  6  7] 97
[10  1 75 12  2 10  3 13 11 14  3 11 12] 99
[11 10 75 12 13 10  3 10 11 14  3  6 12] 73
[11 10 10 12 13 10  3  8  4 13 12 11  2] 101
[11 10 13 12  5  8  5  9  7  8  9 11 12] 81
[12 11 11  3  8 10  5  2  9  6 13  4  4] 83
[ 9 11 75  6 10  5  1  2  4  8  9  4 10] 83
[ 6  4  6  1  7 10  3  2  3  5 11  9  8] 73
[ 6  4 75  7 13 10 13  2  9  5 13  6  5] 60
[ 5 10  9 12 13  5 13  2  1  6  3 11 12] 55
Generation  76
Fitness  129
[11 10 75 12 13 10  3  8 11 14  1 11  1] 96
[11 10 10 12 13  7  3  8 12 13 12 11 12] 84
[10  4 75  2  7 10  3  7  3  5 13  6  7] 68
[ 6  4 75  2  9 10  3  2  3  5 13  4  4] 89
[ 9 11 14  4 13  9  9 12  7  8  1  8  3] 86
[11 10 13 14  4  2  5  1  4  8  9  4  2] 74
[ 9 11 75 14  4  2  5  3  3  8  9  4 10] 95
[ 9 11 75  6 10  5  1  2  4  8  9  4 12] 76
[11 10 75 12 13  2  3  8  1 14  3 11  2] 89
[11 10 10 12 13 10  3  8 12 14  3 11 12] 59
[10  1 14  2  2 10  3  2  3  5 13  6  7] 99
[12  4 75  6 10  5 13  2  9  5 13  4  4] 97
[11 10 13  5 13 11  9  8  7  8  1  8  4] 80
[ 9 11 75 14  4  2  5  3  4  8  7  4  2] 72
[ 9  5 75  5  4  2  5 13  4  3  8  4 10] 71
Generation  77
Fitness  129
[11 10 75 12 13 10  3  8 11 14 13  6  7] 103
[10  4 14  6 10  5 13  2  9 11 13  4  4] 69
[12  4 75  6 10 11  3  8 11 12  1 11 13] 95
[13 10 75 12  4  2 14  3  3  8  9  4 13] 109
[ 9 11 75 14  7  2  5  8  1 14  3 11  2] 85
[11 10 75  2  9 10  3  2  3  5 13  4 13] 102
[14  4 75  2  9 10  3 12  7  8  1  8  3] 68
[ 9  6 14  4 13 12  9  8  7  6  1 11  1] 89
[10  6  3 12  2  2 13  2  3  5 13  6  1] 85
[10  1 14  2  7 10  3  2  3  5 13  4  4] 78
[ 8  4 75 12 13 10  3  8 11 14  1 11  1] 74
[11 10 75 12 13 10  3  8 11 14  1 11  7] 100
[ 9 11  2 12  9  2  3  8  1 14  3 11  2] 93
[11 10 75 12 13 10  3  2  3  5 13  4  4] 100
[ 6  4 75  2  9  2  3 11  7  8  1  7  3] 92
Generation  78
Fitness  129
[11 10 75 12  4  2 14  3  3  8  9  4 13] 95
[14 10 10  8  4  2 14  3  3  8  7  4  7] 74
[11 10 75 12 13 10  3  8 11 14 13  2 13] 94
[ 1 10 75  2  9 10  3  2 11  5 13  4  4] 110
[11 10 75 12 13  9  3  8  9 14  1 11  7] 105
[11 10 75  6 13 11  3  8 11 12 12 11 13] 97
[12  4 75 12  9  2  3  8  1 14 13 11  2] 105
[ 9  3  2 12 13 10  3  8  3 14  3  6 12] 79
[11 10 75 12 10 11  8  3  3  8 13  4 13] 98
[13 10 75 12  4  2 14  3  3  8  9  6  7] 80
[11 10 75 12 13 10  2  2  3  5 13  4  4] 99
[11 10 13 12 13 10  3 13 11 14  1 11  7] 86
[11  4 75  6 10 11  3  8 11 11  1 11 13] 78
[12  4  9 13 10 11  3  8  1 14  3 11  2] 66
Generation  79
Fitness  129
[11 10 75 12 13 10  3  8 11 14  3  9  4] 88
[ 1  6 75 12  3  2  3  8  1 14 13 11  2] 83
[ 1 10 13 12 13  9  3  8 10 10 13 11  7] 55
[11 10 75  7  9  2  3  2  3  5 13  4 13] 80
[11 14 75  2  9 10  3  2  3  5 13  4  4] 86
[ 3 10 75 12 13 11  8  3  3  8 13  4 13] 80
[11 10  8 12 10 11  3  8 11  2 12 11 13] 82
[11 10 75  6 13 11  3 11 11 14  3 11  5] 106
[12 10 75  2  9 10  3  2 11  5 13  4  4] 93
[12  4 13 12  9  2  4  9  1  5 13 11  2] 104
[12  2  2 12  9  9  3  2  9 14  1 11  7] 75
[11 10 75 12  5 11 11  8  9  6  1  4 13] 86
[11 10 75 14  9 13  3 13  6  5 13  5  8] 69
[11 10 14 12 10 11  8  3  3  8 13  4 13] 85
[ 8 10 75  6 13 11  3  8 11 12 12 11 13] 66
Generation  80
Fitness  129
[ 2 10  6  3 13 10  3 11 11 14  8 11  5] 99
[11 10 11  6 13  7  5 11 11 14  3  6  2] 68
[12 14 13 12  9  2  4  2 14  5 13  4  4] 83
[12 10 75 12 13 10 13  8 11 14  3  9  4] 72
[11 10 75 12 13 10  3  8 11  6  1  4 13] 97
[11 14 75  7  9  5  3  2  3  5 13  4  4] 75
[11 14 75 14  9 10  3  2  3  5 10  4 13] 92
[11 10 14  9 10 11  5  3  2  1 13 11 12] 66
[11 12  5  6 13 11  1  4 11  5  3 11  5] 98
[11 10 75 12 14  2  7  9  1  8 13 11  2] 93
[12  4 13 12  9  2  3  9  1  5 13  4  4] 104
[12 10 75  2  9 10  3 12 11 14 12 11  4] 99
[11 10 75 12 13 10  3  8  9  2  5  4 13] 74
[11 10 75 12  5 14  3  2  3  5  6  4  4] 85
[11 14  2  2  9 10  3  2  3  7 13  4 13] 82
Generation  81
Fitness  129
[11  4 13 12  9  2  3  9  1  5 13  4  4] 87
[12  4 13 12  9  2  1  9  1 14 12 11  4] 78
[12 10 75  2  9 10  3 11 11 14  8 11  5] 91
[ 5 10  6  3 13 11  1  4 11  5  3 11  5] 92
[ 6  9  5  6 13 10  2  8 11  6  1  4 13] 77
[11 10 75 12 13 10  8  8 11  6  1 11  2] 104
[11 10  9 12 14  2  7  2  3  8 10 12 13] 75
[ 1  2 75 14  9 10  3  8 11 14  3 11 12] 61
[11 10 75 12 13 10  3  8 11 14  3 11  4] 95
[12 10 75  2  9 10  3 12 11 14 12  6 12] 93
[ 2 10  6  3 13 10  3 11 13 14  8 11  5] 83
[ 2 10  6  3 13 10  7 11 11  5  3 11  5] 79
[11 12  5  6 13 11  3  8  1  6  1  4 13] 72
[11 10 75  9 13 10  3  8 11  6 13 11  2] 66
[ 1  1  2 12 14  2  7  9  1  8 13  4  2] 63
Generation  82
Fitness  129
[11  9 75 12 13 10  8  8  5  6  1 11  2] 86
[11 10 75 12 13 11  3  8  3 14  3 11  4] 100
[11 10 75  1  9 10  3 12 11 14 12  6 12] 68
[ 7 10 75  2  9 10 10 12 11  5  3  7  5] 85
[11 10  6  3 13 11  1  4 11 14  8 11  5] 94
[12 10 75  2  9 10  3 11 11 14  8 11  4] 66
[11  4  7 12 13 10  3 11 13 14  8 11  6] 91
[ 2 10  6  3 13 10  3 11  3 14  8 11  2] 83
[11 10 75 12 13 10  8  6 10  6  1 11  2] 87
[ 9 10 75 12 13 10  3  8 11 14  3 11 12] 81
[12 10 75  2  9 10  3  9 11  5  3 11  5] 64
[ 5 10  6  3 13 11  1 11 11 14  8 11  5] 88
[12 10 75  2  9 10  3 11  3  5 13  4  4] 107
[11  4  4  6  9  2  3  9  1 10  8 11  5] 108
Generation  83
Fitness  129
[11 10 75  6  9  2  3  9  1 10  1 11  5] 102
[11  1 12  2  9 10  3 11  3  5 13  4  4] 89
[12 10  3  2  9 10  3 11  3 14  3 11  4] 102
[ 7 10 75 12 13 10  3  8 11 14  3 11  3] 101
[ 7 10 75 12 13 10  3  4 11 14 12 11  5] 88
[11 11  6 12  7 10  3 11 13 14  8 12  6] 97
[11  2  7 10 13 10  3 11 11 14 12 11  5] 90
[ 5 10 11  5 13 12  1  5 11 14  3  2 12] 87
[11 10 75 12 13 10  3  8 14 14  8 11  5] 76
[11  4  4  8  9 10  3 11  3  5  2  4  3] 68
[12 10 75  2  2 10  3 13  3  5 13 11  4] 73
[11 14 75 12 13 10  2  8 11 14  3 11  4] 107
[11 10 75 12 13 10  1  4 11  3  8 11  5] 123
[14 10  6 12 13 10  3 11 13 14  8  8  6] 74
[11  4  7 14 13 10  1 11 11 14  8 11  5] 88
Generation  84
Fitness  129
[11 10 75 12 13 10  3  8 12  5  3 11  6] 87
[ 4 14 75 12 13 10  1  4 11 14 13 11  4] 87
[14 10  3  2  9 10  3 11  3 14  2 11  4] 91
[12 10 75  6  9  2  3  9  1 10  1 11  5] 74
[11 10 75 12 13 10  3  8  2 14  3 11  3] 64
[ 7 10 75 12  5 10  3  8 11 14  1 12  6] 75
[11 11  6 10 13 10  3 11 11 14 12 11  5] 67
[13  2  7 10 13 10  3 14  5 14 12  2 12] 93
[11  7 75 12 13 10  1  4 11  3  8  4  5] 107
[11 10 75 12 13 10  1  8 12 14  3 11  4] 66
[11 14 75  2  9 10  3  1  3  9  3 11  4] 70
[12  2  3  2  9 10  3  9  8 10  1  3  5] 79
[11 10 75  6  9 12  3  9  1  9  1 11  7] 104
[ 7  5 75 12 13 10  3  8 14  4  3 11  6] 97
[11  2  7 10 13 10  3  7 11 14 12 11  5] 77
Generation  85
Fitness  129
[11 10 75  9 13 10  1  4 11  3  8 14  5] 88
[11  7 75 12 13 10  1  4 10  3  1 11  7] 94
[11 10 75  6  9 12 10  8 14  4  3 11  6] 90
[ 7  5 14  2 13 10  3  8  5  9 12  2 12] 92
[13  7 13  2  9 10  3 11  3 14  2 11  4] 89
[14 10 12  2 13 10  1  4 11 11 13 11  4] 106
[ 4 10 75 12 13 13  3  9 12  5  3  5  6] 79
[11 10  3 12 13 10  3  8 11 14  3 11  4] 76
[11 10  9 12 13 10 11  8 11 14  3 11  5] 95
[11  7 75 12 13 10  1  4  2  3  1 11  7] 104
[11 10 12  6  9 12 13 14 14  4  3 11  6] 82
[ 7  5 75 12 13 10 11 14  5 14 12  2 13] 83
[13  2  7 10 13 10  3 14  5 14  2 11  4] 80
[14 10 75 12 12 10  1  4 11 14 13 11  6] 63
[ 4  3 75 12 13 10  3  8 12  5  3 11  6] 75
Generation  86
Fitness  129
[11 10 75 12 13 10  1  4 11 11 13  9  4] 91
[11 10  2 12  5 10 11  8 11 12  3  7  5] 94
[11 10  9 12 13 10 11  4 10  3  1 11  7] 92
[11  7 75 12 13  2  1  4 10  3  1  2  9] 98
[ 7  5 14  2 13 13  3  8 12  8  3 11  6] 70
[11  7 13  2  9 10  3 11  9 14  2 11  1] 66
[13  7 13  2 14 10 12 11  3 14  3 11  9] 92
[11  8 11  1 13 10  3  8 11 14  3 11  4] 97
[14 11 75 12 13 10  1  4  2  3  1 11  7] 76
[11  7 75 12 13 10  8  4  2  3  1  5  5] 93
[11  2 14  2  5 10  3  8  5  9 12  2 12] 79
[ 9  5 14  2 13 10 10  8 14  5  3 11  6] 76
[11 10 13  2  9 10  3 11  6  3  2 11  4] 96
Generation  87
Fitness  129
[11 10  6 12 13 12 10  4  2  3  1  9  7] 82
[11  7 75 12 13 10  1  4  2 11  1 11  5] 102
[11  7  7 12 13 10  4  8 11 14  3 11  4] 80
[11  8 11  1 13 10  3 11  6  3  2 11  4] 86
[11 10 13  8  9 10  3 11  6  3  2 11  7] 95
[11  7 75 12 13 10  9  5 10  3  1 11  5] 99
[11 10  1 12  5 10 11  8  5  1  1  5  5] 87
[11  7  4 12 13 10  3  8  2 14  3 11 12] 115
[11 10 75 12 13 10  3  8  2  3  1 11  1] 66
[11  7 75 12 13 13  1  4 10  3  1 11  4] 85
[11  8 11  1 13 10  3  8 11 14  3  2  4] 87
[11 10 13  2 10 10  3 11  6  3  1 11  7] 85
[ 7  7 75 12  5 10 11  8 11 12  3  7  2] 78
[11 10 75 12 13 10  8  4  2  3  1  9  5] 99
Generation  88
Fitness  129
[11 10 75 12 13 10  3  8  2 14  3 11  1] 100
[11  7  4 12 13 10 13  8  2 14  3 11  5] 100
[11  7 75  7 13 10  8  4  2  3  1  9  5] 105
[11 10 75 12 13 12  9  5  9  3  1 11  4] 82
[11  7 75 12 13 10  9  6 10 13  1  7  7] 80
[ 5  7 13  8  9 10  3 11  6  3  2 11  7] 88
[ 9  1 11  1 13 10  3 12 11 14  3  2  4] 111
[11  8 11  5 13 10  3  8 11 14  7 11 12] 87
[11 10 75 12 13 10 13  8  2 14  3 11 12] 108
[11  7  1 12 13 10  1  4  1  2  1 11  5] 75
[11  7 75  7 13 10  8  4  2  3  1  9  5] 95
[13 10 75 12 13 10  8  5 10 11  1 11  5] 96
[11  7  9 12 13 10  9  5 10 11  1  2  9] 97
[11  7 75 12 13  2  1  4 10  3  1 10  7] 101
[ 4  8 11  1 13 10  3  8  4 14  3 14  4] 77
Generation  89
Fitness  129
[14 10 11  1 13 10  3 12 11 14  3  2  4] 78
[ 9  1 11  1  1 10  7 12 11 14  3  2 12] 87
[11 10 75 12  6 10  8  4  2  3  6  9  5] 114
[11  7 75  7 13 10  1  5 10  3  1 11  7] 97
[11  7 75 12 13  2  1  4 10  3  1 11  5] 90
[11  7 75 12 13 10  3  8  2 14  3  8  1] 63
[13 10 75 12 13 10  3  8  2 11  1  2  9] 86
[11  7  9 10 13 10  3 11  6 14  3 11 12] 86
[11 10 14 12  2 10 14  9  7 14  3  2  4] 96
[ 9  1 11  1 13 10 14  8  2 14  3 11 12] 88
[11 10  9 12 13 10 13  8  2 14  3  9  5] 73
[11  7 75  7 13 10  8  4  2  3  1 10  7] 83
[11  7  4 12 13  1 13  8  2 14  9 14  5] 72
[11  7  4 12 13 10 13  8  2 14  3 11  1] 94
[11 10  9 12 13 10  3  8  2 14  3 11  9] 63
Generation  90
Fitness  129
[11 10 75 12 13 10  3  8 11 14  3  9  2] 85
[11 10 75 10 13 10  1  9 10  3  1 11  7] 79
[11  6 75  7 13 10  1  9  7 11  3  7  4] 84
[11 10 14 12  2 10 14  9  3 14  3 11  1] 98
[11  7  4 12 13 10 13  8 10  3  1 11  3] 86
[11  7 11  1 13  6 14  8  2 14  2 11 12] 83
[ 9  1 11  1  1 10  7 12 11 14  3  2  8] 100
[ 9  1 11  1 13 10  3  8 11 14  3 11 12] 69
[11 10  4 12 12 10  8  8  2  3  6  9  5] 91
[11 10  3 10  6 10  1  5 10  3  1 11  7] 88
[11  7 75  7 13 10  1  5  7 14 11  7  4] 81
[11 10 14 12  2 10 14  9  7  7  3  2  1] 73
[11  7  4 12 13 10 11  4  4  3  1 11  3] 88
[ 4  7 14 12 13  7  1  8  2 14  3 11 12] 95
[ 5 11 12 14 13 10 14 11  2 14  3  2 12] 66
Generation  91
Fitness  129
[11 10 75 12 13 10  3  8 11 14  3 11  8] 76
[ 9  1 11  1  1 10  7 12 11 13  3  3  1] 104
[11 10 14 12 13  7  1  8  2 14  3 11 12] 90
[ 2  7 14 12 13  7  1  8  2 14  8 11  5] 69
[11  7  4 12 13 11 11  4  4  1 12 11  3] 81
[ 3  1  3 10  6 10  1  5 10  3  1 11  7] 85
[11 10  3 12 13 10 13  8 10  3  6 11  4] 83
[11  7  3 12 13 10 13  8 10  3  3 11 12] 101
[11 10 75  1  1 10  7  3 11 14 10  2  8] 83
[ 9  1 11  1  1 10  7 12  3 14  3 11  1] 88
[11 10 14 12  2 11 14  9  3 14  3 11 12] 61
[ 4 10  4 12 13 10  8  1  2  3 10  9  5] 95
[11 10  4 12 13 10 11  4  4  9  1 11  3] 92
[11  7  4 12 13 10  1  5 10  8  1 11  7] 95
[11 10  3 10  6 10 13  7 10  3  1 11  3] 78
Generation  92
Fitness  129
[12 10 75 12  1 10  7 12  9  8  3  3  1] 87
[ 9  1 11  1  1  9 13  8 10  3  3 11 12] 91
[11  7  3 12 13 10 10  5 10  8  1 11  7] 74
[ 4 10  4 12 13  7  8  1  2  3  9  9  5] 74
[ 4 10  4 12 13 10  8  1  2  7  1 11  3] 83
[11 10 14 12 13  7  1  2  2 14  3  9 12] 85
[11 10 11  1  1 10  7 12  3 14  3 11  1] 71
[ 9  1  3 12 13  4  3  8 11  5  3 11 12] 68
[11  1 11  1  1 10  7 12 11 13  3  3  1] 81
[ 9  1 11 12 13  4 13  8 10  3  3  2 14] 71
[11  7 10 11 13 10  1 12 10  8  1 11  7] 80
[11  7  4 12 13 10  1  5  2  3 10  9  5] 83
[ 4 10  4 12 13 10  8  4 13  9  6 12  3] 72
[11 13  4 12 13  7  1  8  2 14 13 11 12] 80
[11 10 11  1  1 10  7 12  3 14  3  6  7] 100
Generation  93
Fitness  129
[11 10 75 12  1 10  7  8 11 14  3  6  7] 83
[11  9 11  9 12 10  7 12  3 14  3  6 12] 85
[ 9  1 11  1  1 10  7  2  9  8  3 12  1] 72
[12 10 14 12 13  7  1  2  2 14  3  9 12] 94
[11 10 14 12  5 10  1  5  2  3 10  9  5] 82
[11  7  4 12  6 10  8  1  2  7  1 11  3] 90
[ 4 10  4 12 13 10  8  1 11  3  3  7  1] 108
[11  1 11  1 13  7  3  8 11 14  3 14  8] 92
[11 10  8 12 13 10  7 12  3 14  3  6  7] 79
[ 7 10 11  1  1  9 13  8 10  3  4 11 12] 95
[ 9  1 11  1  5 10  7 12  9  8  3  3 10] 90
[12 10 75  6  6  7  1  2  7 14  3  9 12] 99
[11  7 12 12 13  9  1  5  2  3 10  9  5] 73
[11  7  4  1 13 10  8  1  8  7  1 11  3] 73
[ 4  9  4 12 13 10  8 12  5 13  3  3  1] 112
Generation  94
Fitness  129
[ 4  9  4 12 13 10  8 12  5  3  3  7  1] 83
[ 7 10  4 12 13  7  1  2  7 14  3  9  2] 89
[12 10 12  6  6  7  1 10  7  3  4 11 12] 79
[ 7 10 11  1  1  7  1  2  2 14  3  9 12] 88
[11  1 11  9 13  7  3  8 11 10  3 14  8] 81
[11  1 11  1  5 10  7 12  9  8  3  3 10] 95
[ 9  1  6  1  9 10  7 12  6 14  3 11 12] 93
[11 10 10 12 13 10  8 12  5 13  3  3  1] 112
[ 4  9  4 12 13 10  8  6 11  3  3  7  1] 90
[ 4  5  4  6  6  7  1  2  7 14 14  9 12] 92
[12 10 75  1  1  9  7  8 10  3  4 11 12] 83
[ 6 12 11  1  1 13 13  8 10  3  4  9 12] 80
[ 9  1 11  1 13  7  3  8 11 11  3 14  8] 134
[11  1 11  8 13  7  7 12  9  8  3  3 10] 100
Generation  95
Fitness  134
[ 9  1 11  1 13  7  3  1 11 14  3 11 12] 89
[11 10  1 12 13 10  3  8  7 14  3  3  1] 91
[13  9  1 12 13 10  8 12  5 13 12  3  1] 85
[11  1 10  8 13  7  7 12  9  8  4  3 10] 76
[ 3  1 11  1  5 10  7  5  9  8  3  3 11] 84
[11 12 11  1  5 10  7 12  9  8  3 11 12] 82
[ 9  5  4  6 14  7  1  2  7 14 14 11 12] 87
[ 4  5  4  6  6  7  1  2 11 11  3 14  8] 101
[ 9  1  3  1 13 10  3  8 10 14  3 11 12] 87
[11 10 75 12 13 10  3  8 11 13  3  3  1] 71
[11 10 10 12 13 10  8 12  5 13  3  3 14] 85
[ 4  9  4 12 13 10  8 12  5  8  3  3 10] 75
[11  1 11  1  5 10  7 12  9  8  3  3  6] 78
[11  1 11  1  5 10  7 12  9 14  3 11 12] 93
[ 9  7  6  1  9 10  1  3 12 13 14  1 12] 73
Generation  96
Fitness  134
[ 4  5  4  6 11 10  7 12  9 14  3 11 12] 100
[11  1 11  1  5 10  7 12  9 14  3  7  1] 82
[11 10  1 12  9  2  3  8  7 14  3 11 12] 93
[ 9  1  8  1 13  7  3  8 10 14  3 11 12] 74
[ 9  1  3  1 13 10  3  8 10 14 11 11 12] 98
[ 9  5  4  3 14  7  1 12  5 13  3  3 14] 72
[ 2 10 10 12 13 10  3  8 11 11 14 14  8] 81
[ 9  1 11  6  6  7 13  2 11 11 10 14  8] 80
[ 4  1 11  1  5 10  7 12  9 14  3 11 12] 92
[11  1 11  8 13 10  3  8  7 14  3  3  1] 93
[11 10 11  1 13  7  3  1 14 14  3 11  6] 91
[ 9  9 11  6 13 10  3  8 10 14  3 12  4] 106
[ 5  1  3  9 10 10  1  2  7 14 14 11 12] 92
[ 9  5  4  9 13 10  8 12  5 13  3  3 14] 86
Generation  97
Fitness  134
[ 9  1 11  1 13  7  3 10 10  4  2 12  4] 74
[ 7  5  9  6  6  7  1  2 11 11  6 14  8] 104
[ 4 11  4  6 11  9  7  8 10 14 11 11 12] 84
[ 9  1  3  4 13 10  3  8  7 14  3  3  1] 68
[11  1 11  8 13  2  3  8  7  9  3 13 12] 79
[11 10  1 12  9  2  3  5 12 14 14 11  7] 98
[ 5  1  3  9  2 10  1  2  7 14  7 14  8] 85
[ 9  1 11  1 13  7  5  8 11 11  3 12  4] 80
[ 9  9 11  6  6  7  1  2 11  3  3 14  9] 102
[ 3  5  4  6  6  7  6  2 11 11  3 11 12] 62
[ 4  5 10  6 11 10  7 12  9 14 11 11 12] 82
[ 9  1  3  1 13 10  3  8 10 14  3  3  1] 75
[11  1 14  8 13 10 11  8  7 14  3 11 12] 100
[11 10  1  9  9  1  4  8  7 14  3 11 12] 85
Generation  98
Fitness  134
[ 9  1 11  1 13  7  3  8 11 11  6 14  8] 75
[ 7 14  9  6  6  7  1  2 11 11  6 14  9] 101
[ 9  9 11  6  6  7  1  2 11 14 12 11 12] 85
[ 2  1 14  8 13 10 11 12  9 14  3 11 12] 72
[ 4  5  4  6 11  2  3  5 12 14 14 11  7] 106
[11  9  1 12  9  2  3  8  7 14  3 11 12] 73
[ 2 10  4  9  9  1  4  8  7 14  3 11  8] 79
[ 5  1  3  9  2 10  1  2  7  8  7 14  8] 92
[ 8  5  5  6  6  9  1  2 11 11  6 14  8] 92
[ 7  5  9  6  6  7  1  2 11  3  3 14  9] 89
[ 9 12 14  8 13 10 11  8 10 14  3 10 11] 74
[11  1 14  8 13  7  7  7  7 14  3 11 12] 82
[ 4 10  1 12  9  2  3 12 12 14 14 11  7] 76
[11 10  1  9  9  8  4  8  7 14  3 11 12] 87
[11 10  1  9  2 10  1 11  7 14  7 14  8] 85
Generation  99
Fitness  134
[ 9  1 11  1 13  7  3  5 12 14 14 11  7] 87
[ 4  5  4  6 11  7  1  2 11 11  6 14  9] 88
[ 7 14  9  6  5  7  1  2 11 11 14 14  8] 116
[ 2  5  5  6  6  9  6  2 11 11  7 14  7] 90
[ 5  1  3  9  3  5  1  2 14  3  3 14  9] 80
[ 4  5  9  6  6  8  4  6  7 14  3  1 12] 58
[11 10  1  7  2 10  1 11  7 14 10 14  8] 83
[ 5  1  4  6 11  2  3  5 12 14 14 14  7] 77
[ 4  5  4  5  6  7  1  2 11 11  6 14  9] 80
[ 7 14  5  6 11  8  1  2 11 11  7 14  5] 82
[ 8  5  5  6  2 10 14  2 13  8  7 14  8] 100
[ 5  8  3  9  2 10  1  2  7  8  7 14  9] 90
[ 7  5  9  3  7  8  4  8  7 14  3 11  7] 77
[11 10  1  9  2 10  1 11  5 14  2 14  8] 79
Generation  100
Fitness  134
[ 9  1 11  1 13  7  3  8 11 11 14  6  8] 100
[ 7 14  9  6  5  7  1  2 11  8 14 14  2] 80
[ 8  3  5  6  4 10  6  2 13  8  7 14  9] 96
[ 5  8  3  3  6 12  6  2 11 11  7 14  7] 78
[ 2  5  5  6 11  7  1  2 11 11  6 14  9] 77
[ 4  5  4  6 11  7  3  5  4 14 14 11  7] 94
[ 9 13 11  1 13  2  3 11  7 14  7 14  8] 99
[11 10 11  1  6  7  3  8 11 11  6 14  8] 86
[ 7 14  9  6  9  7  1  2 11  3 14 14  2] 82
[ 7 14  9  6  8  9  1  8 11 11  8 14  8] 78
[10  5  5  8  2 10 14  2  7  8 13 14  9] 88
[12  8  3  9  2  4  1  2  7  8  7 13  7] 77
[ 4  5  4  6 11  7  1  2 11 11  3 14  9] 82
[13  5  4  6 11  7  1 14 11 11  6  8  7] 82
[ 9  1 11  9  2 10  4 11  7 14  7 14  8] 75
Generation  101
Fitness  134
[ 9  1 11  1 13  7  3  8 11 11 14  3  8] 98
[ 9  3 11  1 12  7  3  8 11 11  1 14 10] 105
[ 9 13 11  1 13  2 10 11  7 14  7  7  9] 82
[ 8  3  5  6  4 10  1  5  4 14 14 11  7] 82
[ 4  5  4  1  2 10  2  2  7  8 13 14  9] 78
[10  5  5  8  2 10 14  2  7 11  6 14  8] 72
[11  5  4  6 11  7  1 14 11 11  6  8  7] 91
[13  7 11 11  4  7  3  8 11 11  3 14  8] 84
[ 9  1 11  1 10  7  3  8  7 14 11 14  8] 93
[ 9 13 11  1  4 10  6  2 13  8  7 14  9] 80
[ 8  3  5 10  4 10  6  2 13  8  7 14  7] 93
[ 4 12  4  6 11  7  3 11  7  8 13 14 10] 95
[10  5  5  1  6  7  6 11 11 11  6 14  8] 85
[11 10 11  1  6  7  7  8 11 12  6  8  7] 91
Generation  102
Fitness  134
[ 9  3 11  1 12  7  3  8 11 11  1 14  7] 87
[ 9  3 11  4 13  7 12  8 11 14 14  8 13] 91
[ 9  1 11 14  6  7  3  8 11 11 14  6  8] 77
[13  1 11  1 13  7  3  8  7  8 13 14 10] 95
[ 4 12  4  6 10  7  3 11 13  8  4 14  7] 71
[ 8  8 11  1 10  7  3  8  7 14 11 14  8] 78
[ 9  1 11  1  6  7  7  8 11 12  6  8  7] 83
[11 10 13  1  6  7  7  8  8 12  6 10 14] 78
[ 9  1 11  1 12  7  3  8 11 11  1  4 10] 97
[ 9  3 11  1 12  7  3  8 11 11 14  6  8] 94
[ 9 11 11  1  6  7  3  8 11 11  7  3  4] 61
[ 9  1  7  1 13  7  3  8 11  2 14  3 10] 102
[ 8  3  5 10  4 10  6  2 13  8 14 14  2] 96
[ 8  3  5  1 10  8 11  8 10 14 11 14  8] 95
[10  1 11  1 10  3  3  8 11 12  6  8  7] 89
Generation  103
Fitness  134
[ 9  1 11  7 13  7  2  8 11  2 14  3 10] 84
[ 9  1  7  1 13  7  3  8  1  2  1  4  4] 60
[ 9  1 11  1 12  8  6  2 13  8 14 14  1] 84
[ 8  3  5 10  4 10  6  2  6 14 11  5  8] 81
[13  5 11  1 13  7  3  8  7  8 13  6  8] 84
[ 1  3 11  1 12  7  3  8 11 14 14 10 13] 85
[ 9  3 11 12 13  7 12 10 11 11  3  1  8] 102
[12  3  2  1 13  7  3  8 11  2 14  3 10] 97
[ 9  1  7  1 13 14  3  8 11 11  1  4 10] 97
[ 9  1 11  1 12  7  3  8 11 11  1 14  2] 92
[13  3  5  1 10  8 11  8 10 14 11 14  8] 93
[ 8  3  5  1 10  6 11  8  7  8 13 14 10] 93
[13  1 11 10 12  7  3  8 11 11 14  6  8] 94
[ 9  3 11  1 12  7  9  8 11 11 14  8 13] 85
Generation  104
Fitness  134
[ 5  1  2 13 13  7  3  8 11 10  1  1  8] 97
[ 9  3 11  7 10 14  3  7 11 11  1  4 10] 94
[12  3  2  1 13  7  3  8 11  7 14  3 10] 85
[14  6  2  1 14  7  3  8 11  2 13  3  9] 74
[13  1 11  1 13  7  3  8 11 11 14  6  8] 84
[13 13 11  1 10  6 11  8  8  8 13  9 10] 81
[ 8  3  5  1 13  8 11  8  5 14 11 14  8] 73
[ 2  1  5  1 10  7  3  8  7 11  3 12  8] 79
[ 7  1 12  1 13  7  3  9 11 11  3  1  8] 88
[ 9  3 11 10 13  7 12 10 11 11 12  4 10] 95
[ 9 11  7  1 13  7  3  8 11  7  4  4 10] 98
[ 4  3  2  1 13  7  5  8  7  8 13 14 10] 84
[13  1 11  1 13  7  3  8 11 11 14  6  8] 80
[13  1 11 10 12  7  3  8 12  8 13 14 10] 70
[ 8  3  5  8 10  6 11  8  7  8 11 14  8] 63
Generation  105
Fitness  134
[ 9  1  7  1 13  7  3  8 11  7  4  4 10] 71
[ 9  1  2 13 13  7  1  8 11 10  1  1  8] 101
[ 4  1  2 13 13  7  3  8 11 10  5  1 10] 90
[ 7 10 11  7 10 14  3  7 11 11  1  4 10] 80
[ 9  3 11  7 10 14  3  9 11 11  3  1  8] 90
[ 7 14 12  1  7  7  3  9 11  7 14  3 10] 93
[ 4  3  2  1 13  7  5  8  7  8 13  4 10] 99
[ 4  3  2  2 12  1  6  8 11 11  3 14  8] 90
[ 9  1 11  1 13  7  3  8  9  7  4  4 10] 83
[ 9 11  7  1 13  7  3  8 11 10  1  1  8] 104
[ 7  1  2 13 13  7  3  8 11 11 12  4 10] 89
[ 9  3 11 12 13  7 12 10 11 11  1  4 10] 78
[14  3 11  1 13  2  6  9 11 11  1  1  8] 71
[ 7  1 12  1 13  7  3  8  2  7 14  3 10] 101
[12  3  5  1 13  7  5  8  7  8 13 14 10] 75
Generation  106
Fitness  134
[ 9 11  7  1 13  7 11  8 11  3  1  1  1] 68
[ 3 11  7  1  6  7  3  8 11 10  1  1 10] 75
[ 7  1 12  1 13  7  3  8  2  7  1  1  8] 106
[14  3 12  1  7  7  3  9 11  7 14  3 10] 77
[13 14 12  2 12  1  6 13 11 13  3 14  8] 104
[ 4  3  2  7 10 14  3  9 11 11 11  1  8] 79
[ 9  3 11  7  4 14  3  8 11 11  3 14  8] 76
[ 9  9  2  1 13  7  3  8  1 10 13  1  8] 111
[ 9  1 12  1 13  7  3  8  2  7 14  6 10] 62
[ 7  1 12  1 13  7  3  9  2 10  1  1  8] 59
[ 9  1  2 13 13  7  1  8 11  3 13  4 10] 64
[ 4  3  2  1 13  7  1  1  7  8 14  3 10] 79
[ 7 14 12  1 12  1  6 11 11 11  3 14  8] 96
[ 4  9  2  2 12  6  6  8 11  6  6  1  8] 103
Generation  107
Fitness  134
[ 9  1 12  1 13  7  3 10  2  7  1  1  8] 80
[ 7  1 12  2  1  1  6 13 11 13  3 14  8] 68
[13 14 12  2 12  1  6  8 11  6  6  1  8] 83
[ 4  3 11  1 13  6  5  8  7  8 13  4 10] 90
[ 4  6  2  1 13  7  7 11 11 11  3 14  8] 70
[ 7 14 12  1 12  1  6  1  7  8 14  3 10] 82
[ 4  3  2  1 13  7  3  8 11 11  3 11  8] 86
[ 9  1  4  2 13 13  3  8  1 10 13 10  8] 76
[12  9  2  1 13 13  3  8  2  7  1  7  5] 90
[ 7  1 12  1 13  1  6  1 11 13  3 14  8] 105
[13 14  2  2 12  6  6  8 11  6  6  1  8] 91
[ 4  9  2  2 12  3  5  8  7  8 13  3  9] 75
[ 4 14  2  1 12  1  6 11 11  8  3 14  8] 82
[ 7  1  6  1 12  1  6 11 11 11  3  3 10] 87
Generation  108
Fitness  134
[12  1 11  1 13  7  3  3 11 11 13  1  8] 112
[ 9  1 12  1 13  1  6  1 11 13  3 14 12] 95
[ 7  1 12  6 13  1  6  1 11  6  6  1  3] 105
[12  9  2  1  4 13  3  8  2  7  1  7  5] 79
[12  9  2  1 13  8  3  8  2  7 13  4 10] 76
[ 4  3 11 12 12  1  6 11  1 12  3  3 14] 93
[ 7  1  6  1 13  7  3  8  4 11  3 11  8] 85
[ 4  3  2  1 13  7  1  8 11 11  3 14  8] 98
[ 9  1 11  1 13  7  3  4 11 11 13  1  8] 88
[ 9  6 14  1 13  1  6  1 11 13  3  9  8] 87
[ 7 14  2  2 12 13 12  5 11  6  6  1  8] 94
[13 14  2  2 12  6  6  8  2 12  1  7  6] 85
[14  3  5  1 13  6  5  8  7  8 13  4  8] 75
[ 7  1  6  1 12  1  6 11 11 11  3  3  5] 84
[ 7  1  6  1 13  7  3  8 11 11  3 11  8] 85
Generation  109
Fitness  134
[12  1 11  1 11  7  3  3 11 11 13 11  8] 94
[12  1 11  1 13  7  3  3 13 11  6  1  6] 68
[ 7  1 12  2 11  1  6  1 11  8  8 14  8] 89
[ 4  3  2 11 13  7  1  8 11 13  3 14 12] 95
[ 9  1 12  1 14 13  6  1 11  6  6  1  3] 83
[ 7 14 12  2 12  1  6 11  1 12  3  3 14] 90
[ 4  3 11 12 12  1  6 11  1 11 13  1  8] 83
[ 9  1 11  6 13  7  3  2 11 11  3 14  8] 87
[ 9  1 11  1 13  7 13  9 11 11  3  1  8] 92
[12  1 12  6 10  1  6  1 11  6  6  1  3] 90
[ 7  1 12 12 13  1  1  8 11 11  6  3  8] 73
[ 4  3  2  1 13  7  1  8 11 11  3  2  7] 62
[ 9 14  2  2 12 13  8  5 11  6  6  1  3] 77
[ 7 14  2 12 12  1  6 11 13 12  3  3 14] 83
[ 9  1 11  1 13  7  3 10 11  7 13  1  8] 86
Generation  110
Fitness  134
[ 9  3  2 11 13  7  1  8 11 13  3 11 12] 93
[ 4  3  2 11 13  7  1  1 11 13  3 14  8] 82
[12  1 11  1 13  7 13  9 11 11  3  1  8] 78
[ 9  1 12  6 10 12  4  1 11  6  1  1 11] 71
[12  1 12  6 10  1  6  1 11  6  3  3 14] 75
[ 7 14 12  2 11  1  6  1 11  8  8 14  8] 63
[ 7  1 12  2 13  7  3  2 11 11  3 14  8] 106
[ 9  1 11  6 13  7  3  2 11 11  3  7  8] 64
[ 9  1 11  1 13  7  3  8 11 13  3 14 12] 86
[12  1  1 12  4  7  3 12 11 11 13 11  8] 87
[12  1 11  1 11  7  3  9 11 11  3  1  2] 89
[ 9  1 11  6 10  1  6  1 11  6  6  1  3] 82
[12  1  5  6 10  1  6  1 11  6  3  3 14] 90
[ 7 14 12  2 11  1  6  2 11  8  8 14  8] 98
[ 7  1 12  2 11  1  3  2 11 11  3 14  8] 77
Generation  111
Fitness  134
[ 9  1 11  1 13  7  3  8 11 11  3  4  8] 102
[ 7  1 12  2 11  1  6  2 11  8  8 14  3] 65
[ 7 14 12  2 11  1  5  8 11 13  3 11 12] 83
[ 9  3  2 11 13  7 10  8 11 13  3 11 14] 57
[12  1 11  6 11  7  3  9 11 11  3  1  1] 83
[12  1  7 13 11  7  5  9 14 11 13 11  8] 85
[12  3 11  1 13  7  1  8 11 11  3 14 12] 93
[ 9  1 11  9 13  2 11  8 11 11  3 14  8] 78
[ 9  1 11  1 13  7  3  8 14 11  3 14  8] 82
[ 7  1 12  2 13 12  3  2 11  8  8 14  8] 68
[ 7 14 12  2 11  1  6  8 11 13  3  2 12] 71
[ 9  3  2 11 13  1  5  1 11  6  3  3 14] 101
[12 13  5  6 10  1  3  9 11 11  3  1  2] 96
[12  1  1 12  9  7  3 12 11 11 13 11  8] 102
[12  1  1 12  4  7  7 12 11 11 13 14 12] 105
Generation  112
Fitness  134
[12 14  1 12  4  3  7 12 11 10 14 11  8] 82
[ 3  1 11  1 13  7  3  8 11 11  3  4  8] 70
[ 9  1  9 11 13  1  5  1 11  6  3  3 14] 103
[ 9  3  7  2 10  3  3  9 11 11  3  2  2] 92
[12 13  5  6  9  1 11  9  2 11 14 14 12] 77
[12  3 11 10 13  7  3  8  6 12 13 11  8] 91
[13  1  7 13 11  7  7  9  4 11  3 14  8] 90
[ 9  1 11  1 13  7  7 12  6 11 13  4 12] 94
[12  1  1 12  5  7  2 12 11 11 13 11  8] 75
[ 9  1  3  7 13  7  3  8 11 11  3  3 14] 71
[ 2  5  2 11  3  1  3  9 11 11  2  1  2] 98
[12 13  5  6  7  6  1  8 11 11  3 14 12] 72
[12  3 11  1 13 13  1  9  4 11 13 11  8] 79
Generation  113
Fitness  134
[12  1  1 12  4  7  7 12 11  1 13 12 14] 75
[ 9  1  9 11 13  1  3  8 11 11  3  4  8] 101
[ 9  1 11  1 13  7  7  8 11 11  3  4  2] 82
[ 2  5  2 11 13  7  7 12  6 11 13  1 12] 90
[ 9  3  7  2  7  3 11  9 11 11  3  2  2] 76
[ 9  3 11 10 13  7 13  8  6 12 13 14  8] 96
[12  6 11 10 13  7  3  8 11 11  3 14  8] 75
[ 9  1 11 12 13  4  7 12 11 11 13 14 11] 76
[12  1  1  9  4  7  7 12 11  6  3  3 14] 84
[ 9  1  9  1 13  7  3  8 11 11  3  4  8] 72
[ 9  6 11  7 13  7  3 13 11 11  3  4  2] 77
[ 8  5  2  1 13  7  7 12  6 11 13  4 12] 87
[ 7  1 11  1 13  7  7  8  6 11 13  4  2] 85
[ 9  3 11 10 13  7  3  8  6 12 11 11  8] 97
Generation  114
Fitness  134
[ 9  1 11  1 13  7  3  8  3 11  3 14 12] 73
[12  1  1 12  4  7  8 14 11 11  3  4  7] 101
[ 9  1  9 11 13  3  3  8 11 11 11 11  8] 67
[ 9  9 11  5  8  7 13  8  6 12 13 14  8] 69
[ 9  3 12 10 13  7  7 12  6 11 13  1 12] 107
[ 2  5  2  5 13  7  7 12  6 11 13  4 12] 102
[ 1  5  2  1  7  7  7  8  6 11 14  4  2] 95
[ 7  1 11 14 13  7  7 11 13 11 13  6  8] 90
[ 9  1 11  1 13  7  3 12 11 11  9 14 12] 94
[ 9  1  9 11 13  1  3  8  6 12 11 11  8] 79
[ 9  3 11 10 10  7 13  8  6 12 13 14  8] 99
[ 8  3  1 10 13  7 13 12  6  7 13  1 12] 80
[10  1 11  1 10  4  4  8  6 11 13  4  2] 84
Generation  115
Fitness  134
[ 9  1 11  1 13  7  3  8 10 11 13  1 12] 52
[ 9  3  6 10 13  7  7  9  6 11 13  4 12] 95
[12  5  2  7  8  7  7 12 11 11  3  4  8] 99
[ 9  1  1 12  4  7  8 14 11 11  3  4  7] 114
[12  1 11 10 10  7 13  8 12 12 13 14  8] 86
[ 9  3  7 10 10  7 13  8  6 11 14  4  2] 58
[14  5  2  1  7  7  3 12 11 11  6 13 12] 76
[ 9  1 12 10 13  7  7 12  6 11 13  1 11] 58
[ 9 13 12 10 13  7  7  8  6 11 13  4  8] 99
[ 1  5  9 11 13 12  3  8 11  5  3  4  8] 103
[ 9  1  9 11 13  1  3  8 11  2  3  4  7] 89
[12  1  1 12  4  7 13  3  6 12 13 14  8] 92
[ 1  1  2  1  7  7  7  8  6 11 14  4  2] 85
[ 1  5  2  1  7  7  7 12 11 11  9 14  5] 90
Generation  116
Fitness  134
[ 5  1 11  6 13  7  4  8 11 11  3 14  8] 83
[ 9  1 11  1 13  7  3  8  4 11  3  4  7] 83
[ 9  1  9 11  7 12  9  8 11  5  3  4  8] 81
[ 1  5 12 11  7 12  7  1  6 11 13  4  8] 65
[ 9 13 13 10 13  7  7  8 11 11  3  4  8] 91
[12  5  2  7  8  7  7  5 11 11  2  4 12] 97
[ 9  3  6 10 13  3 13  9  6 11 13 14  3] 83
[12  1 11  1 13  7  3  8 11 11  3 14 12] 79
[ 9  1 11  1 13 13  3  8 11 11  3 14  8] 87
[ 9  1  7  4 13  7  3  8 11 11  3  4  7] 83
[ 9  1  1 12  4  1  3  8 11 13  3  8  8] 95
[ 1  5 11 11 13 12  3  8 11  9  3  4  8] 87
[11 13 12 10 13 12  7  8  6 11 13  4  8] 109
[12  8  2  7  8  7  7 11  6 11 13  4 12] 100
[ 9  3  6 10 13  7 13  9  6 12  9 14 11] 101
Generation  117
Fitness  134
[ 9  1 11  9 13  2  7 11  4 11 13  4  8] 85
[11 13 12 10 13  7 13  9  6 12  9 14 11] 74
[ 9  3  6 10  8  7  7 11  6 11 13  4 12] 103
[12  8  2  7  8  7  7  5 11 11  2  4 12] 57
[12  1  1 12  4  1  3  8 11  9  3 12  8] 99
[ 9  1  1 12  4  1  3  8 11 13  3  4  8] 99
[ 9  3 13 11 13 12 12  3  6  9  3  4  8] 126
[ 1  5 11 11 13  2  3  8 11 11  3 14  8] 87
[ 9  1 11  1 13 12  7 10  6 11 13  4  8] 125
[11  9 12 10 13  7 13  9  5 12  9  7 11] 71
[ 9  3  6  7  8  7  6 11  6 11  8  4 12] 65
[12  8  2  7  8  7  7  5 11 11  2  4 12] 91
[12  5 10  7  4  1  1  8 11 13  3  8  8] 84
[ 9  7 13 10 13  7  4  8 11 11  3  4  8] 112
[ 9 12  3 10 13  7  7 12 11 11  3 12  8] 119
Generation  118
Fitness  134
[ 9  3 13  5 13 12 12  3  6  9  3  4  8] 100
[ 9  3 13 13 13 12  7 10  6 11 13  4  8] 81
[ 9  1  3 10 12  7  7 12  5 11  3 12  8] 65
[ 9  8  3 10 13  4  7 12  8  3  5  4  8] 95
[14  7 11 10 13  7  4  8 11 11 13  4 12] 80
[ 9  3  6  4  8  5  3  8 11 13 13  4  8] 97
[ 9  1  1 12  4  1  3  8 11  9  3  7  8] 97
[12  3 11  1  5  7  3  8 11  2  3 14  8] 95
[12  3 11  1 13 12  4 10  6 11 13  4  8] 92
[ 9  1 11  3 13  7  7 12 11 11  3 12  8] 80
[ 9 12  3 10 13  7  7 12 11 11  3  4 13] 62
[ 9  7 13  3  7  7  9  8 11 11 13  4 12] 88
[ 9  3  3 10  8  7  7 11  1 11 13  4  8] 83
[ 9  1  1 12  2  1  3  8 11 13  3  4  8] 95
Generation  119
Fitness  134
[ 9  1 11  1 13  7  3  8  6  9  3  4  8] 84
[ 9  3  7  4 13 12 12  3  6  9  3  5  8] 94
[ 9 11 13  5 13 12 12  3 11  9  3  7  8] 106
[ 1  1  1 12  4  1  3  8  3 13 13  4  8] 71
[ 9  3  6  6  8  5  3  8 11 13 13  4  8] 96
[ 9  1  1  2  2  8 11  8 11  2  6 14  8] 60
[12  3 11  1  5  7  3  8 11  2  5  4  8] 73
[ 9  1 11  1 13  7  3  8 10 11  3 14  8] 106
[ 9  1 11 10 11 13  3  8  6  9  3  4  8] 83
[ 9 10 13  5 13 12 12  6  6  9  3  4  8] 80
[ 9  3 13  5 13 12 12  3 11  9  3  7  8] 91
[ 9  1  1 12  4  1  3  8 11 13 13  7  7] 70
[ 9  3  1 12  2  1  3  8  6 13  3  4  5] 82
[ 9  1  1 12  2  1  3  6 11 13  3 14 13] 66
[12  3  1  1 13  4  7 12  8  4  5  4  8] 105
Generation  120
Fitness  134
[ 5  1 14  1 13  4  3  8 11 11  9 14  8] 85
[ 9  1 11  1  7  7  3  8 10 11  3  7  8] 94
[ 9 11 13 14 13 12  7  1 10  4  5  4  8] 75
[12  7  6  6  8  5  3  8 11  6 13  4  8] 72
[ 9  3  6  6  8  5 12  3  6  9  3  5  8] 57
[ 9  3  7  4 12 14 12  3  6  9  3  5  8] 94
[ 9  3 13  5 13 12 12  8  6  9  3  4  8] 103
[ 9  1 11  1 13  7  3  8 11 11 13  6  8] 85
[ 9  1 11  1 13  7  3  8 10  9  3 13  8] 83
[ 9  1 11  1 13  7  3  8 11  9  3  7  8] 80
[ 9  2  1  1 13  4  7  7  8  4  5  7  7] 69
[12  3  1  8 13  4  7  8 11  4 13  4  8] 82
[ 9  3  6  6  8  5  3  8 11 13  3  4  8] 85
[ 9  6  7  4 13 12 12  3 11  9  3  7  8] 83
[ 9  3 13  5 13  7  3  8  6  9  3  4  8] 95
Generation  121
Fitness  134
[ 9  1 11  5  2 12 12  8  7  7  3  4  8] 72
[ 1  3 13 10 13 12 12  8  6  9  3  4  8] 80
[ 9  3 13  5 13  7  1  8  6  9  3  5  8] 77
[ 9  4 11  6  8  5  3  3 11 13  3  4  8] 104
[ 9  3  5  6  8  5  3  8 11 11 13  6  8] 97
[10  1 11  1 13  7  3 10 11 11  3 14 14] 82
[ 9  1 11  1 13  2  3  8 11 11  3  4  8] 84
[ 9  2 11 11 13 12  6  8  6  3 13  4  8] 91
[ 9  3 13  4 12 14 12  3  6  9  1  5  8] 79
[ 9  8  7  4 12 14 12  2  6 12  3  7  9] 75
[ 9  1 11  1  7  7  3  8 11 13  3  4  6] 73
[ 5  2 11  1 13  7  3  8 11 11 13  6  8] 81
[ 9  1 11  1 13 11  3  8  5 11 10 14  8] 109
Generation  122
Fitness  134
[11  1 12  1 13  7  3  8 11 11  3 14  8] 82
[ 9  4  1  6  8  5  3  3 11 13  3  4  8] 94
[ 9  3  5  6  8 12  3  8 11  9 13  6  8] 82
[ 9  3  5  6  8 14 12  3  6  9  3  5  8] 88
[ 9  4  5  9 13 12  6  8  6  3 13  4  8] 77
[ 9  2 14  1  5  1  3  8 11 11  9 14  8] 84
[ 5  1 14  1 13  2  3  8  8 11  3 10  2] 68
[ 9  1 11  1 13  7  3 11 11 11  3 14  8] 77
[ 9  1 11  1 13 11  3  8  7 11  3 14  8] 108
[ 9  1  1  1 13 11  3  8  5 13  3  4  8] 98
[ 9  4  5  4  8  2  3  8 11 11 13  2  8] 82
[ 9  3  5  6  8 14 12  3  6  9  3  5  8] 70
[ 9  3  7  4 12 14 12  8  6  3 13  4  8] 118
[ 9  2 11 13 13  3  6  8  6  3 10 14  8] 87
[14  1 14  1 13  1 14  8 11 11  9 14  8] 94
Generation  123
Fitness  134
[ 9 12  5  1 13 11  3  8  7 11  3 14  8] 77
[ 1  5 11  1  3 11  3  8  5 13  3  4  8] 82
[ 9  1  1  1 13 11  3  8 11 11  9 14  8] 75
[ 9  4  1  6  8  5  3  3 11 13  3 13  8] 101
[ 9  4  1  6  8  5  3  3 11 13  7  5  8] 74
[ 9  8  1 13 13  1  6  8  6  4 10 14  8] 55
[ 9  2 11 13  2  3  6  8  6  3 10  9  8] 74
[ 9  1 11  4 12 14 12  8  6  3 13 12  8] 80
[ 9  3  7  4 12 14 12  5  7 11  3 14  8] 102
[ 2  1 11  1 13 11  3  9  5 13  3  4  8] 106
[ 9  2  1  1 13  1 14  8 11 11  9 14  8] 85
[14  1 14  1 13  1 14  3 11 13  3  4  8] 96
[ 9  4  1  6  8  5  3  3 11  7  3  5  3] 102
[ 9  3  5  2  8  3 12  3  6  9  3 11  8] 79
Generation  124
Fitness  134
[ 9  1 11  1 13 14 12  1  1  3 13  4  8] 82
[ 9  3  7  4 12 14 12  6  6  3 13  4  8] 86
[ 2  4  1  3  8  5  3  3 11 12  3  5  3] 88
[ 9  4  1  6  8  5  3  3 11 11  3 14  8] 81
[ 9  4  1  6  9  5  3  3 11 13  1 13  8] 90
[ 9  4  4  6 13  1 14  3 11 13  3  4  8] 85
[14  1  1  1 13  1 14  3 11 13  3  4  8] 91
[ 9 12  1  1  2  1 14  8 11 11  3 14 13] 83
[ 9  1 11  1  8  7  3  8 11 11  3  4  8] 80
[ 2 11  7  4 12 14 12  8  5  7  3  4  8] 87
[ 2  1 11  6  8  5  3  3 14  7  3 14  3] 81
[ 4  4  7  4 12 14 12  5  7  9  3 14  8] 87
[ 2  3  1  6  8  5  3  3 11 13  3 13 12] 96
[ 9  4  1  6  8  5  3  3 11 13  3  4  8] 107
[14  1 14  1  3  1 14  8 13  1  9 14  8] 103
Generation  125
Fitness  134
[ 9  4  1  6  8  5  3  3 11 13  1  4  8] 71
[ 9  4  1  6  8  1 14  8 13  1  9 14  8] 119
[14  1 14  1  3  5  3  3 11 13  3 13 10] 67
[ 2  3  1  1 13  1 14  3 11 13  3  4  8] 101
[14  1  1  1  9  5  3  3 11 13  1 13  8] 94
[ 9  4  1  6  7  5  3  3 11 13  1  5  3] 92
[ 2  4  1  3  8  5  3  5  7  9 14 14  7] 95
[ 7  4  7  3 12 14 12  5  7  9  3 14 11] 69
[ 9  4  1 13  8  5  3  3 11 13  8  4  8] 59
[ 9  4  1  6  8  6 14  3 11  1  9 14  8] 67
[14  1  7  1  3 14 14  3 11 13  3 13 12] 89
[ 2 13  1  6  8  5  3  3 14 13  3  4  8] 72
[14  1  1  1 13  6 14  3 11 12  1 13  8] 74
[ 9  4  1  7  8  5  3 11 11 12  3  5  7] 100
[ 2  4  7  4 12 14 12  5  7  9  3  5  4] 75
Generation  126
Fitness  134
[ 9  4 12  6  8  1 14  8 13  1  9 14  8] 109
[ 9  4  1  6  8  1 14  3 11 13  3  4  8] 89
[ 2  3  1  1  8  5  3 11 11 12 11  5  7] 95
[ 9  4  1  1  6  5  3  5  7  9  7 14  3] 73
[14  1  1  1  9  5  6  3  1  6  1 13  8] 88
[14  1  1  1  9  5  3  3 11 13  1  5  3] 93
[ 9  1  7  1  1  2 14  3 11 13  3 13 12] 80
[14  1  7 12  3 14 14  8 11 11  3 14  8] 86
[ 9  1 11  1  8  1 14  8  7  1  9 14  7] 85
[ 9  4  1  6 13 12 14  3 11 13  3  4  8] 79
[ 9  4  1  7  8  5  3 11 11 12  3  8 13] 87
[ 9  4  1  7  8  5  3 11 11 12 14 14  7] 104
[ 2  4  1 14  8  5  3  5  7  9 14 13  8] 98
[14  1  3  1  9  5  3  3 11 13  1  5  3] 93
[ 9  4  1 12  7  5  3  3 11 13  3 13 12] 81
Generation  127
Fitness  134
[ 9  4 12  6 14  1 14  8 13  1 11 14  8] 76
[ 8  4  3  7  8  5  3 11  2  8 14 14  7] 77
[ 9  4  1 14  8  5  3  6  7  9  7 13  8] 86
[ 2  4  1  1  8  4  3 11 11  7 11  5  7] 104
[ 2  3  1  1  8  5  3 11 11  3 11  5 14] 81
[ 8  1  7  6  8  1 14  3 12 13  3  4  8] 71
[ 9  4  1  6  8  1 14  3 13 12  3 14  8] 89
[ 4 11 11  1 13  7  3  8 11 11  9 14 13] 103
[ 9  4 12  6  8  1 14  6 13  6  9 12  7] 102
[ 9  4  1 10  8  5  3 11  7  9 14 13  8] 92
[ 2  3  1  1  9  5  3  3 11 13  1  5  3] 100
[14  1  1  1  9  5  8  3 11 13  1  5  3] 106
[ 9  8  1  6  8  1 14  3 11 13  3  4  8] 82
Generation  128
Fitness  134
[13  1 11  1 13  7  3  8 11 11  1  1  3] 81
[14  1  1  1  8  6  3  2 11  7 11  5  7] 60
[ 2  4  1  1 13  3  3 13 11 11  9 14 13] 103
[ 4 11 11  1  6  7  3  8 11 11  9 14  7] 89
[ 9  4  1 10  9  5  3  3  3 13  1 10  3] 94
[ 2 12  1  1  9  5  3  3 11 12 11  5  7] 74
[13  1  3 14  9  5 12  3 11 13  1  5  3] 102
[14  1  3  1  9 12  3  3 11  8  1  5  3] 71
[ 9  1 11  1  9  5  8  3 11 13  1  5  3] 104
[ 1  4  1 11  8  4  3 11  4  7 11 14  7] 81
[ 2  4  1  1  8  4  3 11 11 11  9 14 13] 80
[ 4 11 11  2 13  7  3  3 11  6  9 12  7] 93
[ 9  4 12 14  9  5  3  3 11 13  1  5  3] 83
[13  3 13  1  2  5  3 11 11 12 11  5 10] 92
[10  3  1 14 14  5  3 11 11 10 11  5  3] 70
Generation  129
Fitness  134
[ 9  1 11  1  5  5  8  3 11 13  1  5  3] 99
[ 9  1 11  1  9  7  6 13 11 11  9 14 13] 98
[14  4  1  1 13  2  3 13 11 11  9 14  3] 88
[13  1  3 14  9  7  3  3 14 13  1 10 11] 113
[ 9  5 11  2 13  7  5 10 11  6  9  5 11] 81
[ 4 11 11 14  2  5  3 11 11 12 11  5 10] 98
[14  3 13  1  2  7  3  8 11 11  9 14  7] 82
[ 4 11 11  1  4 13  3  8 11 11  3 14  8] 75
[ 9 10 11  1 13  5  8  6 11 13  1  5  3] 53
[11  4  1  1 13  3  3 13 11 11  5 14 13] 92
[ 2  4  7 10 13  3  3 13 11 11  1  5  3] 85
[ 4  1  3 14  9  5 12  3  3 13  1 10  3] 68
[ 9  4  1  2 13  7  3  3 13  6  9 12  7] 61
[ 7 11 11 12  2  5  3 11 11  5 11  5 10] 95
[13  3  5 11  2  5  3 11 11 12 11 14  7] 102
Generation  130
Fitness  134
[ 9  1 11  1 13  7  3  8  4 13  1 10 11] 75
[13  1  3 14  9 13  3  3 14 12 11 14  7] 61
[13  3  5 11  2  5  3  3 11 13  1  5  3] 104
[ 9  1 11  1  5  7  8 10 11 13 10  5 10] 93
[ 4  1 11  6  9  7  1 13  9 11  9 14 13] 76
[14  1 11 12  2  5  3 11 11  3 11 14 10] 111
[11 11  1  1 13  3  8 13 11 11  5 14  7] 84
[11  4 10  1 13  3  3  8 11 11  7 13  8] 95
[ 4  1 11 14  9  7  3  3 14 13  1 10 11] 96
[13  1 10 14  9  7  3  3 14  1 11 14 10] 77
[14  3  3 10  2  5  3  3 11 13  1  5  3] 102
[13  1 11  1  5  5  8  3 11  4 11  5 11] 106
[ 4 11 11 14  2  5  3 11 11 11 14 14 13] 91
[14  1 11  1  9  7  6 11 10  5 11  5 10] 59
[ 7 11 11 12  2  5  6 11  4  5 11  5 13] 90
Generation  131
Fitness  134
[ 9  1 11  8 13  7  3  2 11  1 11  8 10] 73
[14  1 11 12  2  5  8  3 11  4 11  5 11] 90
[13  1 11  7  5  5  3  3  9 13  5  5 10] 85
[13  3 11 11  2  5  3  3 12 13  1  5  3] 70
[14  3  3 10  2  5  3  3  9 13  1  7 11] 83
[ 4  1 11 14  9  7  3  3 14 13  1 10  8] 85
[11  4 10  1 13  3  3  8 11 13 10  5 10] 80
[ 9  1 10  4 13 13  3  8 11 12  3 14  8] 52
[ 9  1 11  1 13  7  3  8 11  3 11 14 10] 76
[14  1 11 12  3  5  3 11 11  4 11  5 11] 88
[11  1 11  1  5  5  8  3 11  4  1  5  3] 81
[13  7  5 11  2  5  3  3 11 13 11 14  3] 65
[14  3  3 10  2  5  6  3 14 13  1 10 11] 109
[10  1 11 13  9  7  3  3 14 11  7 13  8] 102
[11  4 10  1 13  3  4  8 11 11  9  5 10] 96
Generation  132
Fitness  134
[14  3  3 10  2  7  3  4 14 11  7 13  8] 78
[11  4 10  1 13  2  4  8 11 11  9  5 10] 89
[11  4 10  1 13  3  4  3 11  4 11  5 11] 74
[14  1 11 12  2  5  8  3 12  4 11  5  4] 80
[ 4  1 11  5  9  7  3  3 14 13  1 10  8] 84
[13  1 11  7  5  5  6  3  9 13  5  5  9] 97
[13  1  2  1 13  7  9  4 11 11  3 14  8] 82
[ 9 10 13  1 13  1  3  8 14 13  1  5 11] 91
[14  3  3 10  2  5  6  3 14 13  1 13  8] 84
[10  1 10  1 13  3  4  5 11  2  9  4 10] 82
[ 3  4 10 14 13  3  4  8 11 11  9  5 12] 79
[14  1 10 12  3  5  3 11  7  4 11  5 11] 77
[14  1 11 12  9  7  3  3 14 13  1 11  8] 100
[ 4  1 11 12  9  5  2 14  9 13  5  6 10] 97
Generation  133
Fitness  134
[ 9  1  5  1 13  7  6  3 14 13  9 10 11] 95
[14  3  3 10  2  9  6  3 14 13  1 10 13] 73
[14  1 11  3  9  5  2 14  6 13  5  6 10] 88
[ 4  1 11 12  9  5  6  3  9 13  5  5  9] 88
[ 8  2 13  1 13  5  3  8 14  2  7  5 11] 112
[ 9 10  2  1 13  1  3  8  9 13  1  5 10] 100
[11  3  3 10  2  5  6  9 14 13  1 13  8] 83
[14  3  3 10 13 14  8  8 11  6  3 14  8] 102
[ 9  1 11  1 13  7  3  8 11 11  1  6 11] 71
[14  3 14 10  9  7  3  3 14 13  1 11  8] 85
[14  1 11 12  6  8  2 14 10 13  5  6 10] 76
[ 4  1 11 12  9  5  2 14  9 13  5  6  9] 64
[13  1 11  1  7  1  3  8 14 13  1  5 11] 76
[ 9 10 13  1 10  1  3  8 14 13  9  5 10] 60
[11  4 10  1 13  2  6  3 14 13  1 13  8] 74
Generation  134
Fitness  134
[ 9  1 11  1  7  7  3  8 11 11  3  5 11] 90
[ 8  3  5  5 13 14 13  9 11  6  3 14  8] 82
[ 9 11 14  1 13  1  3  8  9 10  9  5 10] 87
[ 9 10  2  1 13  7  6  3  9 13  9 10 11] 72
[ 9  1  5  1  9  5  6 10  9 13  5  5  9] 89
[ 3  9 12 12  9  5  6  3  9 13  5  5 10] 61
[14  9  9  3  9  5  3  3 14 13 12 11  8] 68
[14  3 14 10  9  7  3  3 14 13  3 14 10] 80
[12  1 11  1 13  7  3  6 11 11  7  5 11] 63
[ 8  2 13  1 13  5  8  8 11  6  3 14  8] 94
[ 3 10  4  1  6  1  3  8  9 13  1  5 10] 62
[ 9  5  2  1 13 10  3  8  9 10  4 11 11] 85
[ 9  1 11 12  9  5  1  3  9  2  5  5  3] 65
[ 4  1 11 12  9  5  2 14  6 13  5  6 10] 77
[14  3  2 10  9  7  3 12 14  4  1 11  8] 68
Generation  135
Fitness  134
[ 9  1 11  1 13  7  3  6 11 11  3 14  8] 97
[ 8  2 13  1 13  3  8  8 11  7  3  5 11] 61
[12  1  6  1 11  7  8  8 11 13  5  9  2] 108
[10  1  5  1  9  5  6 10  9 11  5  5 10] 75
[ 9 11 14  1 13  1  3  8  5 10  4  4  6] 79
[ 9  5  2  1  6 10  5  8  9  6  3 14  8] 80
[ 8  3  5  5 13 14  3  3 14 13  3 14 10] 90
[14  3 11  1 13  7  3  2 12 11  3 14  8] 102
[ 9  1 11  1 13  7  3  8 11 14  3 12  8] 89
[ 4  2 13  6 13  5  3  8 11 11  3  5 11] 84
[ 9  1  8  1  7  7  3 10  9 13 11  5  9] 100
[ 9 13  5  1  9  1  3  8 10 10  9 14 10] 77
[ 9 11  9  1 13 10 14  8  9 10  4 11 11] 78
[ 9  5  4  1 13 14 13  9 11  6  3 14  8] 64
[ 8  3  2  5 13 11 13  9 11  6  3 14 10] 83
Generation  136
Fitness  134
[ 9  1 11  1 13  7  3  6 11 11  3  9  3] 81
[12  1  3  1 11  6  8  8 11 11  3 14  8] 90
[14 12 11  1  7  7  3 10  9 13 11  1  9] 91
[ 9  1 14  1 13  7 13  6 11 11  3 14  8] 71
[ 9  1 11  1 13 14  3  3 14 13  3 14 10] 69
[ 8  3  5  5 13  7  3  8  5 14  3 12  8] 102
[ 9  1  3  6  1  5  3 13 11 11  3  5 11] 97
[ 4  2 13  6 13  7  3  8 11 11  3 14  8] 86
[ 9  1 11  1 13  7  2  8  5 13  5  9  2] 65
[12  1  5 12  4  7  3  2 12  7  3 14 11] 101
[14  3 11  5 13  7  3 10 11 13 11  5 11] 84
[ 9  1  8  1 13 10  3  6 11 11  3 14  8] 81
[ 9  1 14  5 13 14  3  3 14 13  3 14 10] 62
[ 8  3 11 12  3  7  3  8 11  3  3 12  8] 83
[ 9  1 11  1 13 10  3  6 11 14  3 12 11] 89
Generation  137
Fitness  134
[ 9  1 12  1 13  7  3  4  5 13  3 12  8] 100
[ 8  3  5  5  4  7  3 13 12  7  3 14 11] 76
[12  1  5 12  4  7  3 13 11  3  9  5 11] 84
[ 9  1  3  1  7  7  3 10  9 13 11 10  9] 79
[14 12 11  1  7  7 12  8 11 11  3 14  8] 84
[12  1  3  1 11  7  8  8 11 11  3 12 11] 105
[ 9  1 11  1 13 10  3  6 11 14  3 14  1] 103
[ 4  1 11  1 13  7 14  8 11 11  3 14  8] 75
[ 9  1 11 14 13  7  3  8 11 14 13 12  8] 87
[ 8  3  5  5 13 12  3  8  5 14  3 12 11] 87
[12  1  5 12  4  7  3  2 12 11 14  1 11] 97
[ 9  1  3  6  1  5  3  8 11 11 11  1  9] 114
[14 12 11  1  8  6  8  8 11 11  3 14  8] 71
[12  1  3  1 13 10  3  7 11 14  3 12  2] 86
[ 4  2 13  6 13  7  3 10 11 11  3 14  8] 77
Generation  138
Fitness  134
[ 9  1 11  6  1  5  3  8 11 11 11  1 11] 84
[ 9  1  3  6  1  5  3  1 11 11  3 12 11] 76
[ 9  1  9  1 13  8  3  6 11 14  3 14  7] 81
[ 9  1 11  1 13 10 14  6  5 13  3 12  8] 105
[ 9  1 12  1 13  7  3  4  1 13  3  1 11] 65
[12  3  5  5 13 12  3  8 13 14  3 12 11] 98
[ 2  3  5  5 13 12  3  8  5  4 13 12  8] 73
[ 9 11 11  9 13  7  3  8 11 11  3 14  8] 68
[ 9  1 11  1 13  5  3  8 11 11 11  1  9] 79
[ 9  1  3  6  1  5  3  8 11  8  3 12 11] 78
[12  1  3  1 11 10  3  7 11 14  3 14  1] 113
[ 9  1 13 11 13 10  3  6 11 14  3 14  8] 82
[ 9  1 12 14 13  7  3  4  5 13  3  1 11] 109
[12  1  5 12 13 12  6  8  5 14  3 12 11] 90
[13  3  6  5 13 12  3  8 12 14 13 12  8] 70
Generation  139
Fitness  134
[ 9  1  3  1 13 10  3  4 11 14  3 14  1] 88
[12  1  3  1 11 10  3  4  5 13  3 11 11] 84
[ 9  1 12 14 13  7 14  4  9 13  3 12  8] 44
[ 9  1 11  1 13 12  3  8 13 14  3 12 11] 86
[12  3  5  5 13 12  3  8 13 14  3 12 14] 85
[10  1  4 12 13 12  6  8 11 11 11  6 11] 72
[ 9  1 13 11 13 10  3  6 11  7  3 14  8] 97
[ 9  1 13  2 13 10  3  6 11 14  3 14  8] 73
[ 1  1 11  8 13  7  3  8 11 11  3 14  1] 87
[12  1 14 14 13  7  3  4  5 13  3  1 11] 110
[ 9  1  1 14 13  7  3  4  5 13  3  1  8] 72
[ 7  1  8  1 11  8 14  6  5 13  9 12 11] 92
[ 7  3  5  5 13 12  3  8  9 14  5 10 11] 96
[12  1  5 12 13 12  6 10  5 14 11  1 11] 85
[ 9  1 13 11 13 10  3  6  3 14  3 14  8] 65
Generation  140
Fitness  134
[ 9  1 11 14 13  7  3  4 13 13  3  1 11] 72
[ 1  1 14 14  3  7  3  4  5  2  3 14  8] 90
[ 9  1 13 11  1 12  3  1  9 14  5 10 11] 91
[ 7  3  5  5 13 12  6  6  5  2  9  3 11] 95
[ 7 12  8  1 11  8 14  6  5 13  9 12  1] 79
[ 9  1  3  1 13 10  3  8 11 11  9 14  1] 81
[ 1  1 11  8 13  3  3  8 11 11  3 14  7] 89
[ 9  1 11  1 13  7  2  4  5 13  3  1 11] 82
[ 2  1 13 11  8 10  3  2 11  7  3 14  5] 96
[ 7  1 13 11 13  6  3  6 11 14  5 10 11] 82
[ 7  3  5  5 13 12  3  6 10 13  9 12 11] 90
[ 7  1  8  1 13 10  3  4 11 14  3 14  1] 87
[ 9  1 11  8 13  7  3  8  8 14  3 14  1] 101
[ 1  1 11  8 13 12  3  8 13  6  3 12 11] 90
Generation  141
Fitness  134
[ 9  1 11  1 13  7 10  8 11  1  3 14  8] 90
[ 9  1 11  6 13  7  3  8  5 14  3 14  1] 96
[ 9  1 11  8 13  7  3  2 11  4  3 14  5] 82
[ 2  1  5  5 13 12  6  6  5  2 10  3  6] 95
[ 9  1  3 11 13 12  3  7 13  1  3 14 11] 84
[ 1  1 11 12 13  4 11  8 13  6 10 12 11] 82
[ 7  1 11  1 13  7  3  8 11 11  3 14  2] 80
[ 9  1 11  1 13  7  3  7 11 11  3 14  8] 82
[ 9 11 13 11  8 10  3 13 11  1  3  7  5] 101
[ 2  9  5  5 13 12  6  6  5 12  9  3 11] 104
[ 9  1  2 11  1 12  8  1  9  6  3 12 11] 83
[ 5  3  5  5 13 12  3  6  9 13  9 12 11] 96
Generation  142
Fitness  134
[ 9  1 11  1 13  7  3  8 11  2  6  3  1] 67
[ 2  9  5  5 13 11  6 13 12  1 11  7  5] 90
[ 9 11 13 11 13  7  3  8  8 14  3 14  1] 73
[ 9  1 11 13 13  7  3  8  8 14  2 12 11] 110
[ 5  3  5  5 13 12  3  6  9 13  9 14  1] 89
[ 9  1  4  5 13 12  6  6  5  2  9  3 11] 80
[ 2  3  5  5 13 12  6  6  5  2  9  3 11] 87
[ 9  3 11  5 13  7  3  8 11 11  3 14 11] 98
[ 9  1 11  1 13 12  6  1  5 12  9 12 11] 95
[ 2 11 13 11  8 10  3 13 11  1  3  7  5] 84
[ 9 11 13 11  8 10  3 13 11 14  3 14  1] 98
[ 9  6 11 12 13  7  3  8  8 13  9 12 11] 91
[ 5  3  5  5 13 12  3  4 10 10  3 14  1] 100
[ 9  1 11  9 13 12  1  6  5  2  9  3 11] 79
[ 7  3  5  5 13 12  6  6  5  2  9  3 14] 79
Generation  143
Fitness  134
[ 9  1 11  1 13  7  3  8 11 11  3 11 11] 72
[ 9  1 11 13 13  1  6  4 10 10 13 14  1] 67
[ 5  3  4 13 13 12  3  4 10 14  3 14  1] 96
[ 9 11 13 11  8 11  3 13 11 11  3 14 11] 62
[ 9  3 11  5 13  7  6  1  5 12  9 12 11] 99
[ 9  1 11  1 13 12  6  1  5 13  9 12 11] 99
[ 9  6 11  2 13  6  3  8  8 13  9  6  5] 99
[ 2  9  5  1 13  7  3  8 13  8  3 14  8] 90
[ 9  1 11  1  5  7  3  8 11 11  3 14 11] 95
[ 5 10 11 13 13  7  3  8  8  8  2 12  1] 86
[ 5  3  5  5 13 12  3 13 11 14  3 13  1] 70
[ 9 11 13 11  8 10  3 13 11 14  3 14 11] 85
[ 9  3  5  3 13 12 10  1  5 12  9 12  2] 59
[ 8  1 11  8 13  7  3  8  8 13  1 12 11] 80
[ 9  6 11 12 13  1  6 13 12  9 11  7  5] 74
Generation  144
Fitness  134
[ 9  5 11  2 13  6  3  8 12 13  8  6  5] 73
[ 9  6 11  6 13  6  3  1  5 13  9 12 11] 94
[ 9  1 11  1 13 12  6  1  5 12  9  8 11] 88
[ 9  3 11  5 13  7  6  1 10 14  3 14  1] 74
[ 5  3  4 13 13 12  3  4  2 10 10  3 11] 90
[ 9  1 11  1  5  7  2  8 13  8 11 14  8] 104
[ 5 10 11 13 13  7  3  8  8  8  2  1  1] 69
[ 5 10  4 13 13  7  3  8 11 11  3 14  8] 70
[ 2  1 11  1 13  7  3  8 11 11  3 14  5] 92
[ 9  6 11  2 13  6 10  1  5 13 14 12 11] 89
[ 9  1 11  1 13  7  6  1  5 12  9 12 11] 76
[11  3 11  5 13  7  6  1  5 12  2 14  1] 105
[ 5  3  4 13  9 12  3  6  2 14 11 14  3] 97
[ 9  7 11  1  5  7  3  8 13  8  3  8  8] 69
[ 5 10 11  1 13  7  3  8 12  8  2 12  1] 74
Generation  145
Fitness  134
[ 9  1  4  1  2  7 13  8  5 12  2  7  1] 87
[11  3 11  5 13  7  6  1  5  5  2 14  8] 93
[ 5  3  4 13  9 12  3  6  2  8 11 14  3] 89
[ 5  3  4 13  9 12  3  2  2 14 11 14 11] 82
[ 9  6 11  6 13  6  3  1  5 13  2 14  5] 101
[ 2  1 11  1 14 12 11  4  2 10 10  3 11] 76
[ 3  3  4  2 13  2 10  1 10 13 14 12 12] 113
[ 9  6 11  2 13  6 10  6  5 13  3 14  8] 90
[ 9  1 11  1  3  7  2  8  4 11  3  9  1] 89
[11  3 11  4 13  7  6  1 12 12  9 14  8] 63
[ 9  3  4 13  9 12  2  6  2 14 11 14  3] 81
[ 2  3  4 13  9 12  2  6  2 14 11 12 11] 63
[ 9  6 11  6 13  6  3  1  5 13  9 14  5] 96
[ 2  1 11  1 10  7  3  4  2 10 10  3 11] 78
[ 5  3  4 13 13 12  3  4  2 10 10 12 11] 79
Generation  146
Fitness  134
[ 9  1 11  1 13  7  3  8 11 13 14 12 12] 77
[ 3  3 14  2 13  6  3  4  5 12  2 14  5] 62
[ 9  3  5  4 13  6  3  1 13 13  9 14  5] 88
[ 9  6 11  6 13  6  3  1  5 13  9  5 14] 82
[11  3 11  2 13  6 10  6  5  9  3 14  8] 82
[ 9  6 11  8 13  6 10  6  5 13  3 11  1] 74
[ 9  1  4 13  9 12  3  6  8 11 11  2  3] 61
[ 9  1 11  1 13  7  3  8 11 11  2 14  8] 87
[ 9  3  4  2  5  2 10  1 10 13 14 12 12] 89
[11  3  4  6 13  6 11  1 10 13  2 14  5] 93
[ 9  6 11  6 13  6  1  1  7 13  9 14  5] 81
[ 9  6 11  4 13  6 10  6  5 13  3 14  9] 69
[ 9  6 11  2 13  6 10  6  5 13  3  9  1] 75
[ 9  1 11  1 10  7  7  3  4  4 11 14  3] 90
Generation  147
Fitness  134
[ 9  1 11  1 13  7  3 14 11 11  2 14  8] 86
[11  3  4  6 13  6 11  3  4 13 11 14  3] 79
[ 9  1 11  1 10  7  7  3  4  4 11 14  7] 111
[ 9  3  4  2  5  6  3  1 13 13  9 14  5] 106
[ 9  3  5  4 13  6  3  8 11  1  2 14  8] 110
[ 9  1 11  2 13 12  3  8  5  1  3 14 10] 91
[11  3 11  2  3  7  3  8  7 11  3  5  8] 85
[ 9  3  5  7 13  5  8  1  5  5  2 14 11] 73
[11  3  4  6 13  6 11  1 12 13  2 14  5] 93
[11  1 11  1 10 11  7  3  4  4 14 14  3] 77
[ 9  1 11  8  5  2 10  1 10 13 14 12 12] 69
[ 9  3  4  2  8  6  3  1 13 13  9 10  5] 74
[ 9  3  5  4 13  7  3  8 11 11  2 14  8] 103
[13  1 11  2 13  6  6  6  5  9  3 14  8] 84
Generation  148
Fitness  134
[ 9  3  5  4 13  6  3  8 11  2  2 14 13] 70
[ 9  3  5  4 13  7  3  8 11 11  2 10  8] 96
[ 9  4  5  4 13  7  3  8 11 11  5 14  5] 78
[11  3  4  6 13 12  9  9  5  1  3 14 10] 72
[ 9  1 11  4 13  6  3  8 11  1  2 14  8] 84
[ 9  3  5  4  5  6  3  7 13 10  9  8 10] 86
[ 9  3  4  2 13  7  3  1 11 11  2 14  8] 81
[ 2  3  5  6  5  6 11  1 12 10  2 14  5] 95
[13  3  4  6 13  6 11  1 10 13  2 12 10] 91
Generation  149
Fitness  134
[13  1 11  1 13  7  3  2 11 11  3 14  8] 73
[ 9  1 10  1 10  7  7  3  4  4 11 14 12] 94
[ 9  1 11  1  9  7  9  3  4  4  5 14  5] 76
[ 2  6  5  6  5  1  1 13 12  1  2 14  5] 95
[11  3  4  6 13  7  3  8 11 11  3 14  8] 91
[ 9  1 11  1 13  1  3  8 11 11  3 14  8] 99
[11  1 11  1 14  7  7  3  4  2 11 14  7] 97
[ 3  1 11  1 10 14  7  3  4 13 11  7  7] 93
[ 9  3  8  2  3  6  3  1 13 13  9 14  2] 85
[ 9  3  5  9 13  7  5  8 11 11  2 10  8] 69
[ 9  3  5  4 13  7  3 14  8 11  2 14  5] 100
[ 1  3  9  6  5  6 11  1 10 12  2 14  5] 104
Generation  150
Fitness  134
[ 9  1 11  1 13  7 14  3  3  4 11 10  7] 91
[ 9  1  2  1 10 13  7  4 10 12  2 14  5] 79
[ 1  3  9  4 13  7 10 14  8 11  2 14  8] 114
[ 9  3 11  1 13  1  3  8 11 11  3 14  8] 108
[ 9  1 11  1  1  7  7  3  4  2 11 14 14] 88
[11  1 11  1 14  7  7  8 11 11  2 10  8] 99
[ 9  3  5  4 13  7 10 14 11 11  2 14  5] 91
[ 2  6  5  6 13  7  3  8 11 11  3 14  8] 99
[ 9  1 14 10 13  7  3  8  2 14 11 14  7] 72
[ 1  3  9  6  5  6 11 14 10 12  2 14  5] 91
[ 1  3  9  6  5  6 11  1 10 11  2 14  6] 66
[ 3  3 11  4 13  7  3 14  8 11  2  9  8] 96
[ 9  1 11  4 14  7 14  4  4  2 11  3  7] 82
[ 7  1 11  1 14  7 10  3 11 10  2 10  8] 78
[ 9  3  5  4 13  1  1 13 12  1  2 14  5] 96
Generation  151
Fitness  134
[ 9  1 11  1 13  7  3  8 11 11  2  9  8] 103
[ 9  3 13  1 13  1  3  7 11 11  3 14  8] 98
[ 9  3 11  1 13  7  3  8 11 11  3 14  8] 103
[11  6  5  6 13  7 12  8 11 11  3 14  8] 102
[ 9  1  5  4 13  1  1 13  5  1  8 14  1] 73
[ 7  3  9 10 10  6 11 14 10 12  2 14 12] 88
[ 1  1 11  1 13  7  3  8 11 11  3 14  8] 84
[ 9  1 11  1 13  7  3  8  8 11  2 14  8] 79
[ 6  3  9  4  1  7 10 14  8 11  2 11  8] 90
[ 9  3  7  6 13  7 14  8 11 13  3 14  8] 102
[ 2  6  5  6 13  7  7  8 11 11  2 10  8] 80
[11  1 11  1 14  1 13  8 11 12  2 10  5] 83
[10  2  5  4 13  7  1 13  8 11  2  9  8] 73
[ 9  3  9  6  5  6 11 14 10 12  2 14  5] 70
Generation  152
Fitness  134
[ 9  1 11  1 13 13  3  8  4  4  3 14  8] 81
[ 9  3 11  1 13  7  5  8 11 14  2  9  8] 95
[ 9  3  7  6 13  7  8  8  8 13  3 14  8] 77
[ 9  3  7  6 13  7 12  8 11 11  3 14  8] 75
[11  5  2  1 13  1  3  7 11  2  3 14  8] 94
[ 9  3 13  1 13  1  1 13 12 14 10 14  5] 95
[ 9  3  9  4  1  7 10  6  8 11  2 11  8] 94
[ 9  1 11  1  2  7  9  8 11 11  3 14  8] 98
[ 5  3 10  1 13  7  3  8 11 11  3  9  8] 98
[ 9  5  7  6 13  7 14  8 11 13  3 14  8] 88
[ 4  6  5 10 13  7 12  8 11 11  3 14  8] 75
[ 9  3 10 14 13  1  3 13 12  1  2 14  5] 89
[ 9  3  7  4 13  1  1 13 12  1  2 11  8] 54
Generation  153
Fitness  134
[ 9  1 11  1 13  7  3  8 11  9  3 14  8] 102
[ 8  3  7  1 13  7  5  7 11 11  3  9  8] 80
[ 5  3 10  1 13  7  3  8 11  6  3 14  8] 80
[ 9  1 11  1  2  1  1  7 12 14 10 11  4] 72
[ 9  3 11  1 13  7  5  8 11  8 13 13  8] 80
[ 9  3 11  1 13  7  5  8 11 14  2 11  8] 75
[ 9  3  9  4  1  7 10  6  8  8  3 14  8] 80
[ 9  1 11  1 13  7  3  8 11 11  3 14 10] 116
[ 9  1  7  6 13  7  6  8  7 13  3 14  3] 94
[ 9  3  7  6 13  7 14  8 11 13  3  9  8] 84
[ 5  5 10  1 13  7  9  7 11 11  3 14  8] 84
[ 9  1 11  1  2  7 11  8 10 11  3 14 10] 91
[ 9  3 13  1 13  1  1 13 12 14 10  8  8] 89
[ 9  3 14  4  9  7 10 11  8 11  2 11  8] 107
Generation  154
Fitness  134
[ 9  2 11  1 13  7  3  1 11 11  3 14  6] 85
[ 9  7  2  1 13  7  3  7 11 11  3 14 10] 109
[10  1 11  1 13  7  3  8  2  3  2 11  8] 103
[ 9  3  9  4  9  7 10  4 11  9  5 14  8] 86
[ 9  1 11  1 13  7  6  8  7 13  3 14  3] 75
[ 9  1  7  6 13  7  6  8 10 11  3 14 12] 94
[ 9  1 11  1 13  1  8 13 12 14 10  8 10] 88
[ 9  3 11  1 13  7  3  8 11 11  3 14  8] 71
[ 9  5 11  1 13  7  3  8 11 11  3 14  8] 93
[ 4  1 11  1 13  3  3  8 11 11  3 14 10] 86
[14  1 14  4  9  7 10 11  8 11  2  5  8] 68
[ 9  3 14  4  9  7  3  8  1 10  3 14  8] 84
[ 9  1 11  1 13  7  3  8  7 13  3 14  8] 78
[ 9  1  7  6 13  7 11  8 10  8  3 14 10] 39
[ 9  1 11  1  2  7 11 13  4 14 10  8  8] 52
Generation  155
Fitness  134
[ 9  1 11  1 13  7  3  8  9 11  3  4 10] 85
[ 9  7  2  1 13  7  3  7  1 11  2 11  8] 86
[10  1 11 10  2  7  3  8  2 11  3 14 12] 87
[ 9  1  7  7 13  7  3  8 14 11  3 14  8] 94
[ 9  1 11  1 13  1  8 13  2 14 10  8 10] 96
[ 9  1 11  1 13  6  8 13 11 11  3 14 10] 68
[ 4  1 11  1 13  3 10  4 11  9  5 14  8] 69
[ 9  3  9  6  9  7  7  3 11 11  3 14  5] 89
[ 9  7  2  1  1  6  3  9 11 11  3  9  2] 70
[ 9  7  2 12 13  7  3  8  2  3  2  6  8] 71
[10  1 11  1 13  7  6  8 10  2  3 14 12] 93
[ 9  1  7  6 13  7  3  8 11 11  3 14  8] 90
[ 9  1 11 14 13  1  8 13 12 14  4  8 10] 91
[ 9  1  6  1 13  3  3  8 11 11  3 14 10] 87
[ 2  1 11  1 13  7 10 12 11 11  5 14  8] 75
Generation  156
Fitness  134
[ 9  2 13  1 13  1  8 13  2 14 10  8 10] 113
[ 9  1  7  7 13  7  5  8 14 10  3 14  8] 81
[ 9  1  7  7 13  7  3 12 14  2  3 14  4] 77
[10  3 11  1  2  7  6  8 10  2  4  8 10] 97
[ 9  1 11 11 13  1  8 13 12 14  4 14  8] 94
[ 9  1  7 13 13  7  7  3 11 11  3 14  5] 75
[ 8  3  9  6  9  3  3  8 11 11  3 14 10] 79
[ 9  1  6  3 13  7  3  1 11 10  3 14  8] 80
[ 9  1 11  1 13  1  8  3  2 14 10  8 10] 58
[ 9  1 11  7 13  7  3  8  2 10  3 14  8] 82
[ 9  1  7 11 13  7  3  8 14  2  3 11 12] 95
[10  1 11 13 13  7  6  8 10  2 10  2 10] 89
[ 9  1 11 14 13  1  8 13  4 11  3 14  8] 72
[ 9  1  7  6 13  7  3  8 11  8  3 14  5] 79
[ 9  3  9  6  9  7  7  3  5 11  3  1 10] 65
Generation  157
Fitness  134
[ 9  1 11  1 13  1  8 13  2 14 10  8 10] 73
[ 9  2 13  1 13 10  8 13  2 14 10  8 10] 81
[10  3 11  1 13  4  3  8 14  2 11 11 12] 99
[ 9 13 11  3 13  1  8 13 12 14  4 14  8] 106
[ 9  1 11 13 13  7  6  8  6  2 10  2 10] 89
[10  1 11 13 13  7  6  8  2 10  3 14  4] 85
[ 9  1 11  7 13  7  3  6  2 10  3 14  8] 68
[ 9  1  7  7 14  7  3  8 11 11  3 14  8] 78
[ 9  1 11  1 13 12  3  1 11 11  3  8 10] 58
[ 9  2 13  1 14 11  6  8  6 14  4  8 10] 70
[10  3 11  1  2  7  6  8 10  2  8  8 12] 96
[ 9 13  7 11 13  7  3  8 12 14  4  3  8] 74
[ 9  1 11 11  3  1 11 10 10  2 10  2 10] 84
[10  1 11 13 13  7  6  8 10  2 10 14  7] 89
[ 9  1 11  7  3  7  3  8 14 10  3 14 13] 101
Generation  158
Fitness  134
[ 9  1 11  1 13  7  3  8 13 11  4 14  8] 108
[ 9 13 11  3 13  5  3  8 14 10 14 14 13] 89
[ 9  1 11  1 13  4  3  8 14  2 11 11 12] 79
[ 8  3 11  1 13  7  6  8 10  2  8  8 13] 93
[14  3 11 13  2  7  6  8 10  7 10 14  7] 72
[10 13 11 13 13  7  6  8 10  1 10  2 10] 84
[ 9  1 11 13 13  7  6  8  2 10  3 14  4] 100
[10  1 11  1 13  7  3  8 11 11  3 14  8] 82
[ 9  1 11  3 13  1  8 13 12  5  4 14  8] 92
[ 9 13 11  4 13  1  3  8 14 10  3 14 13] 74
[12  1 11  1 13  4  3  8 14 12 11 11  7] 89
[10  3  9  3  2  7  6  8 10  2  8  8 12] 89
[10  1 11 13  5 10  6  8 10  8 10 14  7] 84
[10  1 11 13 13  7  6  8  6  2 10  2 10] 97
[ 9  1 11 13 12  7  6  8  6 10  3 14  4] 103
Generation  159
Fitness  134
[ 6  1 11  9 13  7  3  8 11 11  3 14 12] 102
[ 9  1 11 13 12  7  7  8  6 10  3 14  4] 78
[ 9  1 11 13 13  3  6  8  2 10  3 14  4] 95
[ 9  1 11 13 13  6  6  8  2  2  9  2 10] 90
[10  1 11 13 13  7  6  8  6  2 14  2 13] 106
[ 8  3 11  3 13  1  8 13 12  7  4  3  7] 86
[ 9  1 11  3 13  1  6 13 12  5  8  6 12] 86
[10  3  9  3  2  7  6  8  5  2  8 11  8] 105
[ 9  1 11  1  4  7  3  9 13 11  4 14  8] 94
[ 2 11 11 13 13  7  6  8  5 10  3 14  4] 88
[ 9  1 11 13 13 14  6  8  2  2 10  2 10] 61
[10  1  4 13 13  7  6  8 10  2  3  9 13] 99
[ 8 13 11  3 13  1  8 13 12  5  4 14 11] 70
[ 9 11 11  3 13  1  6 11 10  2  2  8 12] 94
Generation  160
Fitness  134
[ 9  1 11  1 10  7  6  9  5  2 14  2 13] 91
[10  1 11 13 13  9  6  8  6  2 14  2  8] 72
[10  3  9 14  2 10  6  8  5  2  8  2  4] 82
[ 6  1 11  9 13  7  3  8 11 11  3  9 13] 101
[10  1  4 13 13  7  6  8 10  2  3  9  4] 65
[11 11 11  3 13  1  6 11 10  2  2  8  4] 100
[ 9 11 11  6 13  1 10 11 10 11  3 14  8] 84
[ 9  1 11 14 13  7  3  8  6  2 14  2 13] 107
[10  1 11  9 13  7  6  8  8  2  8 11  8] 64
[10  3 11 13 12  7  6  8 13 10  3 14  4] 92
[ 9  1 11  5 13  7 11  8 11 11  3 14 12] 57
[10  1  6 13 13  7 14  8  7  2  3  9 13] 68
[10  1  4 13 13  7  6  8 10  2  3 14  4] 63
[ 9  1  3 13 13  3  6 11 10  2  2  8 12] 88
Generation  161
Fitness  134
[ 9  1 11  1 13  7  3  8  6  2 14  2 13] 86
[ 9  1 11 14 13  7  5  8 11 11  3 14 12] 98
[ 6  1 13  9 13  7  3  8 11  3  3  9 13] 81
[ 6  1  6  3 13  1  1 11 10  2  2  8  4] 60
[11 11 11  3 13  1  5 11 10  2  3 14  4] 70
[10  3 11 13 12  7  6  9  5  2 14  2 13] 64
[ 9  8 11  1 10  7  6 11 10  2  2 12  3] 84
[ 5 10  3 13 13  3  6  8 11 11  3 14  8] 99
[14  1 11  1 13  7  7  8  6  2 14  2 13] 68
[ 9  1 12 14 14  7  3 14 11 11  3 14  9] 45
[ 6  1 11  9 13  7  3  8 11 11  3  9 10] 81
[ 6  1 11  2 13  7  3  8 10  2  2  8 12] 87
[10  3 11 13 12  7  2  5 13 10  3 14 13] 108
[ 9  1 11  1 10 14  6 11 10  2  2  8 12] 83
Generation  162
Fitness  134
[ 9  1 11  1 13  7  3  8 11 11  3 14 13] 89
[13  3 11 13 12  7  2  5 10  9  2  8  4] 70
[11 11 11  3 13  1  6 11 10  2  2  8  8] 71
[ 5  1 11  5 13  7  5  8  1  5  3 14 12] 79
[ 9  1 11  2 13  8  3  8 10  2  2  8 12] 88
[10  1 11  2 13  7  3  8 10  2 14  2 13] 75
[ 9  1  6  1 13 13  6 11 10  2  2 12  3] 69
[ 9  8 11  5 10  7  3  8 10  8  3 14  8] 92
[10 11 11 13 12  7  2  5 13 10  3  9 13] 80
[10  3 11 13 13  8  6 11 14  2  2  8  4] 89
[11 11  1  3 13  1  6  8 11 11  3 14  8] 66
[ 5 10  9 13 13  7  5  8 11 11  3 14 12] 87
[ 6  1 13  2 13  7  3  8 10  2 13  2 13] 58
[ 9 14 11  1 10  7  8  3 10  2  2 12  3] 82
Generation  163
Fitness  134
[ 9  1 11  1 10  7  3 14 10  8  3 14  8] 70
[10  3 11  1 13  1  3  8  3 11  3 14 13] 95
[ 9  1 11  1 13  8  3  8 10  2  2  8  2] 101
[ 9  1 11 13 13  8  3  8 10  2  2  8 12] 81
[ 6  1 11  2 13  7  3  8 10  2  4 14 12] 87
[ 5 10  3 13 13 11 13  8 11 11  3 12  3] 110
[ 9 14 11  1 10  7  8  8  5  5  3 14 11] 86
[ 9  2 11  1 13 13 14  8 10  8  3 14  8] 95
[ 9  8 11 13 13 13  6 11 14  2  2  8  4] 97
[10  3 11 13 13  8  6  7 14  9  2 14 13] 86
[ 3  1 11  2 13  8  3  8 10  2 14  8  9] 102
[ 2  1 11  2 13  7  3  8 10 11 13  8 12] 100
[ 6  1 11 14 10  7  3  5  8 11  3 14 12] 90
[ 5 10  9 12 10  7  8  3 10  2  2 12  3] 88
Generation  164
Fitness  134
[ 9  1 11  1 13  7  3 14 11 11  3 12  3] 74
[ 5 10  3 13 11 11 13  8 11 11  3  8  9] 95
[ 3  1 11  2 13  8  8  8 10  2  2  8  2] 76
[ 9  1 11 10 13  7  3  8 10 11 13  8 12] 86
[ 9  1 11  2 14 13  6 11 14  2  2  8  4] 53
[ 9  8 11 13 13 13  6 11 14  2  2  8  8] 88
[ 9  2 11  1 13 13 14  8 10 11  3 14 13] 80
[10  3 11  1 13  7  3  8 11 11  3  2  8] 84
[ 9  1 11  1 13  7  3  8 11 11  3 12  3] 78
[ 5 10  3 11 13 11 13  9 10  2 14  8  9] 80
[ 3  5 11  2 13  8  3  8 10  2  2  8  2] 71
[ 4  1 11  1  4  8  3  8 10  2  2  8 12] 70
[ 2 12 11  2 13  1  3  8 11 11  2  8  4] 63
[ 9  8 11 13 13 13  6 11 14  2  2 10  8] 84
[ 9  6  5  1 13 13  1  4 10 11  3 14 13] 90
Generation  165
Fitness  134
[ 9  1 11  5 13 11 13  8 11 11  3  8 12] 96
[ 5 10  3  1  8 13  1  4 10 11  3 14 13] 61
[ 9  6  5  1 13 13  6 11 14  2  2  8  8] 87
[ 9  1 11 10 13  7  3  8 10 11 13  8 13] 60
[ 9  1 11 10 13  7  3  8 10  2  2  5  8] 108
[ 9  8 11  1  7  7  3  8 11 11  3  2  8] 102
[10  3 11  5 13 11 13  9 10  2 14  8  9] 80
[ 5  5  3 11 13  7  3 13 11 11  8  3  8] 88
[13  6 11  1 13  7  2  1 11 11  3  8  5] 92
[ 5  2  5  1 10 13  1  6 10 11  3 14 13] 112
[ 9  6 13  1 13 13  1  4 10 11  2  8  8] 86
[ 9  8 11 13 13  4  6 11 10 13 13  8 12] 99
[ 9  1 11 10 13  7 10  8 10 11 13  8  8] 67
[ 9 14 11 13 13  7  7  8  5 11  3 13  8] 88
[10  3 11  1 13 11 13  9 10  2 14  8  9] 103
Generation  166
Fitness  134
[ 4  1 11  1 13  7  3  8 10 11  3 14 13] 88
[ 5  1 11  8 13  7 12  8 10  2  2  5 14] 67
[10  3 12  1 13 11 13  9 10  2 14  8  9] 68
[10  3 11  1 13  7  3  8 11  4  3  2 13] 79
[ 9  8 11  1  7  7  3  6 11 11  3  2 12] 70
[ 9  8 11 13 13  4  6 11 11 11  3  8 12] 74
[ 9  1  9  1 13  7  2  3 11 12  3  2  5] 96
[13  6 11  1 11  7  3  8 11 11  3 14  8] 96
[ 9 14  5  1 10  6  1  6 10 11  3  2 13] 68
[ 5  2  5 10 10 13  1  8 10  2  2  5  8] 93
[ 9  1 11 10 13  7  3  9 10  2 14  8  9] 83
[10  3 11  7 13 11 13  9 10 11  3  2  8] 89
[ 9  8 11  5  7  7  6 11 10 13 13  8 12] 63
[ 9  1 11  5 13 11 13  9 12 13  3  8 12] 73
[ 9 13 11  1 13  7  2  1  7 11  1  8  5] 60
Generation  167
Fitness  134
[ 9  1 11  1 13  7  3  8  1 11  3 14  3] 84
[13  6 11  1  7  7  3  8 11 11  3  2  5] 68
[ 9  9  9  6 13 11  1  8 10  2  2  5  4] 80
[ 5  1 11  7 12 11 13  9 10 11  3  2  8] 68
[10  3 13  7 13  7  3  2  6 11  3 14 13] 74
[ 4  1 11  1 13  7  3  9 10  2 14 13 10] 105
[ 9  1 11 10 13  7  3  8 11  4  3  2  7] 94
[10  3 11  1 13  7  3  8 11 11  3  3  8] 85
[ 9  1 11  1 13  1  3  8 11 11  3 14  8] 95
[13  6 12  4 11  7  3  8 11 11  3  2  5] 96
[ 9 10  9  1 10 13  1  8 10  2  2  5  8] 72
[ 2  7  5 10 10 13  1  8 10  4  3  2  8] 107
[10  3 11  1 13  7  3  8 10 11  6  8 13] 102
[ 4  1 11  1 13  7  3  9 10  2  3  8  9] 90
[ 9 10 11  1 10  7  3  8 11  4  3  2 13] 80
Generation  168
Fitness  134
[ 2  7  5 10 10 13  1 12 10  4 10  2  8] 104
[ 2  7  5 10 10 13  1  8  1  2 14 13 10] 86
[ 4  1 11 11  5  5  5  8 10 11  6  8 13] 74
[ 3  3 11  7 13  9  3  8 11 11  3  2  5] 89
[13  6 12  4 11  7  3  8 11 11  3 14  8] 81
[ 9  1 11  3 14 11  3  8  5  4  3  8  1] 76
[ 9  1 11 10 13  7  3  8 12  4  3  2  9] 97
[ 4  1 11  1 13  7  3  1 11 11  3  4  8] 93
[ 9  1  5  1 13  7  3  8 11  4  3  4  8] 96
[ 4  1 11  1 13  7  3  9  1  2  1  7  7] 96
[10  3 12  1 13  7  3  8 10 11  6  8 13] 101
[10  3 12  1 13  7  3  8 10 11  6  2  5] 109
[13  6 12  4 11  7  3  8 11 11  6 14  8] 78
[ 9  1 11  1 13  1  3  8 11 11  3 14  7] 90
[ 9  8 11  1 13  7  3  9 10  2  3  8  9] 92
Generation  169
Fitness  134
[ 9  1 11  1 13  7  3  8 10 11 10  5  3] 95
[10  3 13  1 13  7  3  8  8  4 10  2  8] 62
[ 2  6  5 10 10 13  1 12  8  4  6  8 13] 83
[10  3 12  1 13  7  3  8 12  4 10  2 10] 92
[ 9  1 11 10 13  7  3  8 12  2  3  2  7] 63
[13  1  2  1 12  7  8  9  1  2  1  7  7] 73
[ 9  2  5  1  9  7  3  1 11 11  3  4  8] 78
[ 4  1 11  1 13  7  3  1 11 11  3 14  8] 84
[ 9  1 11  1 13  7  3  8 10 11  6  2  5] 69
[10  3  7  1 13  7  3  8 10 11  6  2  8] 92
[ 1  7  6 10 11 13 10 12 10 10  6  8 13] 86
[10  3 12  1 13  7  3  8 10 11  6  2  2] 71
[ 4 12 11  1 13  7  3  9  7  2  1  7  7] 103
[ 4  1 11  1 11 14  3  8 11  4  3 12  8] 103
[ 9  1  5  1 13  7  3  1 11 11  3  4  8] 92
Generation  170
Fitness  134
[ 8  1 11  1 13  7 11  8 11 11  3 12  8] 97
[ 4  1 12  4 11 14  3  9  7  2  1  7 14] 69
[ 4 12 11  1 13  7  3  8 10 11  9  7  3] 84
[ 9  1 11  1  6  7  3  1 11  6  3  4  8] 62
[ 9  1  5  1 13  7  3  8 10 13 12  2  8] 98
[10  3  7  1 13  7  3  8 10 11  6  2 10] 72
[10  3 12  1 13  7  7 10 12  4 10  2 12] 84
[ 1  7  6 10 11 13 10  6 10 10  6  8  8] 76
[ 9  4 11  9 13  7  3  8 11  4  1 12  8] 81
[ 4  1 11  1  1 14  3  8  7  2  1  7  7] 92
[ 4 12 11  1 13 13  3 10 10 11  9  5  3] 100
[ 9  1 11  1 13  7  3  1 11 11  3  4  8] 90
[10  3  7  1 13  7  3  8 10 11  6  1  8] 98
[10 11  7  1 13  7  3  8 12  3 10  2 10] 69
Generation  171
Fitness  134
[ 9  5 11  1 13  7  3  8 11 11  9  5  3] 89
[ 4 12 11  1 13 13  3  8 10 11  6  1  8] 59
[ 9  1  5  1 13  7  3 13 14 11 12 12  8] 104
[ 8  3 11  1 14  4 11  8 14 14  1  7  7] 84
[ 5  1 11  1 13  7  3  1 13 11  3  4  8] 87
[ 9  1 11  1 13  2  3  1 10  4  6 11 13] 92
[ 1  7  6 14 11 10 10 12 10 11  3 14  8] 94
[ 8  1 14  1 13  7  3  8 10 11  9  5 14] 79
[ 4 12  6  1 13 13  3 10 10 11  6  1  8] 89
[10  3  5  1 13  7  3  8 10 13 12  2  6] 74
[ 9  5  5  1 13  7 14  8 10 13  5  2  8] 97
[ 8  1 11  1 13  7  3  8  7  2  8  7  7] 68
[ 4  1 11  1  1 14  3  8  7  2  3  4  8] 104
[ 9  2 11  1 13  7  3  1 11 11 14  8 13] 82
Generation  172
Fitness  134
[ 9  1 11  1 13  7  3  8 11  2  3  4  8] 94
[ 2  1 11  1  1 14  3  8  7  2  9  4  8] 101
[ 9  1  5  1 13  7  3 13 14 11  4  2  8] 86
[ 9  1  5  1  3  7 14  8 10 13  5  2  8] 71
[ 9  5  5  1 13 10 10 12 10 11  3 14  8] 86
[ 1  7  6 14  9 10 10  4 10  4 10  8 13] 85
[ 9  1  5 14 13 13  3 10 10 11  6  8  8] 85
[ 4  1 11  1 13  7  3  8 11  2  3 14  8] 74
[ 9  2 11  1 13  7  3  8 11  2  3  4 13] 76
[ 9  1  5  1 13  7  3 13 14 11 12 12 10] 83
[ 9  1 11  2 13 11  3  8 10 13 12  2  8] 92
[ 9  1  5  1 13  7  3  9 10 13  5  2  8] 105
[ 9  5 10  1 13  7 10 12 10 11  3 14  8] 60
[ 1  7  6  2 11 10 10 12 10 11  7 14  4] 76
[ 9  1 11  1 13  2  8  1 10  4  6  4  8] 106
Generation  173
Fitness  134
[ 9  1 11  1 13  7  3  1 10  4  9  4  8] 92
[10  2 11  1 13  9  8  3 10  4  1  4  8] 69
[ 9  1 11  1  1 14 13  8  7  2  9  4  8] 89
[ 2  1 11  1 13  7  3  8 11  2  3 12  8] 87
[ 9  1 11  2 13 11  7  8  2  5  8  2  8] 99
[ 9  9 11  2 13 10 10 12 10  8  3 14  8] 81
[ 9  5  5  1 13  7  3 13 14 11  4  2  8] 92
[ 9  1  5 13 10  7  3  8 11 11  3 14  8] 92
[ 9  8 11  1 13  2  8  1  2  4  6  4  8] 80
[ 9 14  5  1 13  7  3  9 10 13  5  2  8] 81
[ 9  1 11  5  1 14  9  8  7  2  9  4  8] 76
[ 2  1 11  1 13  7  5  8  9  6  3  4  8] 93
[ 9  1  5  1  3 10 10 12 10 11  3 14  8] 75
[ 9  4  5  7 13 11  3 13 14 11  4  2  8] 101
Generation  174
Fitness  134
[ 6  1 11  1 13  7  7  8 14 11  4  1  8] 85
[ 9  4  5  7 13  6  7  8  2  5  8  2  8] 83
[ 9  1  4  2 13  7  3  8 10  2  5  4 11] 78
[ 2  1 11  1 13  7  3  8 11  2  3  4  8] 76
[ 2  1 11  1  6  7  3  8 11 11  3 14  8] 94
[ 9  1  5 13 10  7  3  8 11 11  5  2  8] 102
[ 9  5  5  1 13  5  3 13 14 10  9  4  8] 104
[ 9  1 11  4  5  2  3  1  4  4 12 14  8] 112
[ 9  4 11  1 13  7  3 13 14 11  4  2  8] 75
[ 9  1 11  2 13 11  7  8  2  5 10  2 12] 77
[ 9  1 11 13  9 11  7  8 11  2  3  4  8] 81
[ 9  1 11 11 13  7  3  8 11  2  3  6  8] 88
[ 2  1  5 13 10  7  3  8 11 11  3 14  8] 79
[ 9  1  5 13 10  8  3  8 14 11  4  2  8] 84
[ 9  5  5  1 13  7  3  4 10  4  9  4  1] 84
Generation  175
Fitness  134
[12  1  2  4  5  2  3  2  4  4 12 10  8] 90
[ 2  1  5  1 13  5  3 13 14 10  9 12  8] 60
[ 9  5  5  1 13  5  9 13 14  5  9  2  8] 86
[ 9  1  5 13 10  5  3  8 11 11 12 14  8] 100
[ 2  1 11  1  6  7  3  8 11  2  3  9  8] 94
[ 9  1 11  1  7 14  7  8 14  9  2  1  8] 87
[13 13 11  1 13  7  7  8 14 11  4  4  1] 90
[ 9  5  5  1 10  7  3  8 11  7  8 14  8] 88
[ 9 11  8  1 13  2  3  1  4 14  5 14  8] 90
[ 9  1 11  1 13  5  9 13 14 10  9 10  8] 85
[ 9  1  5  8 10  9  3  8 11 11  5  2  8] 71
[ 9  1  5 13 10  7  3  8 11 11  3 14  3] 107
[ 9  1 11 11 13  7  3  8  4  2  3  6 10] 71
[ 9  1 11  1 13  7  8  8 14 11  6  1  8] 100
[ 6  1 11  1 13  7  7  8 14 11  9  4  1] 93
Generation  176
Fitness  134
[ 9  1 11  1  8  7  2  8 11  9  3 14  3] 98
[ 9  7  5 13 10 12  3  8 11 11  3 14  8] 94
[ 9  1 11  1  1  5  3  8 11 11 12 14  8] 59
[ 9  7 11  1  6  7  3  8 11  2 14  9  8] 83
[ 2  1 11  1  6  7  3  8 13  3 14  4  1] 79
[13 13  8  1 13  2  3  1  4 14  5 14  8] 76
[11 11  8  1 13  7  7  8 14  4  4  4  1] 85
[13 13 11  1 13  7  6  8 11 11  9 14  8] 95
[ 9  1 11  4 14  7  3  8 11 11  3 14  3] 78
[ 9  2  5 13 10  7  4  8 14 11  6  1  8] 80
[ 9  4 11  1 13  7 12  8 11 11 12 14  8] 83
[ 9  1  5 13 10  5  3  8 11  8  3  9  8] 87
[ 2  1  9  1  6  7  3  8 11  2  3  4  1] 85
[ 6  1 13  1 13  7  7  8 14 14  5 14  8] 109
[ 9  5  8  1 13  2  3  1 14 11  4  4  1] 85
Generation  177
Fitness  134
[ 9 13 11  8 13  7  6  8 14 14  3 14  8] 106
[12  1 13  8 13  7  2 12 11  1  3  2  3] 96
[ 9  1 11  1  8  7  2  8 11 11  9 14  8] 87
[13 13 11  1 10 12  3  8 11 11  3 14  8] 96
[ 9  1  5  1 13  2  3  1 14 11  4  4  1] 85
[ 9  5  8  1  6  7  3  8 11  2  3  4  1] 95
[12  1  9  1  6  7  3  8 11 11 11 14  8] 106
[ 2  1 13  1 13  1  7  8 14 14 13 14  8] 109
[ 6  1  8  1 13  7  7  8 11 14  3 14  3] 100
[ 9  1 12  1  8  7  2  9 14  9 11 14  8] 80
[13  7  5 13 10 12  3  8 11 11  3  1  8] 87
[ 9  7  5 13 10  5  3  8 11 13  3  9  8] 75
[ 9  5  8  1 13  2  3  1 14 11  3  4  1] 83
[ 9  5  8  1 13  2  3  8 11  2  3  4  9] 76
Generation  178
Fitness  134
[ 9  1 13  1 13  1  7  8 14 14 13 14  8] 74
[ 2  1  9  1  6  7  3  8 11 11 11 14  8] 78
[ 1 13 11  8 13  7  6  8 14 14  3  4  8] 96
[13  1  8  1 13  7  7  8 11 14  3 14  4] 82
[ 6  1  8  1 10 12  3  8  8 11  4 14  8] 81
[13 13 11  1 10 12  3  8 11 11  3 14  3] 98
[ 6  1  1  8 13  1  2 12 11  2  3  4  1] 96
[ 9  5 11  1 13  7  5  8 11  1  3 14  8] 94
[ 9  6 11  1 13  7  7  8 14 14 13 14  8] 60
[ 2  1 13  1 13  2  3  8 11 11 11 14  8] 88
[12  1  9  1 13  7  6  8 14 14  3 14  8] 123
[ 9 13 11  8  9  7  7  8 11 14  4 14  3] 74
[ 6  1  8  1 13  7  7  8 11 14 13 14  8] 85
[11  2 11  1 10 12  3 12 11  1  3  2  3] 80
[12 13  8  1  5  7  3  4 13  2  3  1 13] 96
Generation  179
Fitness  134
[ 9  1 11  1 13  7  3  7  5 14  3 14  8] 100
[12 13 11  1 10 12  3  8 11 11  3 14  3] 64
[13 13 11  1 10 12  3  8 11  2  3  1 13] 73
[12 13  8  1  5  7  3  9 11  2  3  4  1] 88
[ 6 13 11  8 13  7  6  8 14 14  3  4  8] 91
[ 1  5 11  1 13 10  5  8 11  1  3 14  8] 113
[ 9  5 11  1 13  2  3  8 11 11 11 14  8] 90
[ 2  4 10  1 13  7  3  8 11 11  3 14  8] 75
[12  1 11  1 13  6  4  8 14  2  3 14  8] 96
[12  1  9  1 13  7 13  8 14  4  8 14  3] 96
[13 13 11  1 10 12  3  8 11 11  3  1 13] 62
[12 13  4  1 13  1  2 12 11  2  3 14  1] 100
[ 6  1  1  8 13  9  7  5 11  2  3  4  8] 73
[ 1 13  9  8 13  7  7  8 11  1  3 14  8] 89
[ 9  5 11  1 13  2  3  4 11 11 11 14 10] 91
Generation  180
Fitness  134
[ 9  1 11  1 13  7 10  8 11  6  3 14  8] 80
[ 1  5 11  1 13  3  6  8 11  1  3 14  5] 104
[12 13  9  1  1  7  3  9  5  5  3 14  8] 59
[ 9  1 11 13 13  9  3  7  5 14  8 14  3] 105
[12  1 12  1 13  6  4  8 14  2  3 14  8] 99
[13  1 11  1 13  2  3  4 11 11 11 14 10] 89
[ 2  5 11  1  6  4  3  8 14 14  3 14  8] 95
[ 6  4 11  3 13  7  3  8 11 11  3 14  8] 112
[ 1  5 11  1  9 10  5  8 11  1  3 14  8] 65
[ 1 13  4  1 13  1  2 12 11  2  3 14  1] 50
[ 9  1 11  1 13  7  3  8  5  5  3 14  8] 60
[12  1  9  1 13  7  5  8 14  4  8 13  3] 100
[12  1  9  1 13  7 13  8 14  3  8 14  8] 78
[12  1 11  1  9  2  3  4 11 11 11 14 10] 94
[ 6  5 11  8 13 11  6  8 14 14  3  4  8] 85
Generation  181
Fitness  134
[ 9  1 11  1 13  7  3  8 11  9  3 12  8] 72
[ 6  4  3 13  6  9  3  7  5  9  8  9  3] 71
[ 9  1 11 10 13  9  3  7 11  1  3 14  5] 89
[ 1  5 13  1 13  3  7  8 14  4  8 13  3] 81
[12  1 12  1 13  6  1  8 14 12  3 14  8] 79
[13 13 11  1  6  4  3  8 14 14  3 14  8] 74
[ 2  5 11  1  6  4  1  8 14 14  3 14 10] 95
[12  1 11  1  9  2  3  8 12  4  3 14  8] 79
[ 9  1 11  3 13  7  3  8 11 11  3 14  8] 94
[ 6  4 11  3 13  7  3  8 11 11  3 14  3] 101
[ 9  5 11  1 13  3  6  8  7  1  3 14  5] 87
[ 1  5 14  1 13  3  6  8 11  1  3  2  3] 115
[11  1  9  1 13  7  8  1 14  2  3 14  8] 70
[12  1  6  1  6  4  3  8 14  4  3 14  8] 86
[ 2  5 11  1  9  2  3  4 11 11 11 10 10] 84
Generation  182
Fitness  134
[ 9 14 11  1 13  7  3  8 11 11  3 14 14] 86
[ 1  5 14  1 13  4  5  2  2 11 10 14  3] 77
[ 6  4 11  7  6  4  1  8 14 14  3 14 10] 99
[ 2  1 11  3 13  7  3  7 11 11  3 14  8] 97
[ 9  1 11  3 13 12  3  7 11  1  3 14  5] 91
[ 9  1 11  1 13  3  6  8  7  4  3 14  5] 70
[ 9 14 11  1 13  3  3  8 14  4  3 14  8] 71
[12  1  6  1  6  4  1  8 14 11  3 14  8] 72
[ 9  1 11  1 13  3  6  8 11  1  3  2  3] 76
[ 1  5 14  1 13  3 12  8 11  9  3  2  3] 67
[ 6  4 11  3 13  7  3  8 11 11  3  4 10] 91
[ 2  5 11  1  6  4  3  8 11 11  3 14  8] 66
[ 9  1 11 10 13  8  3  7 11  1  3 14  5] 94
[ 9  1 11 10 13 14  6  8  7  1  3 14  5] 84
[12  1  6  1  6  4  3 12 14  4  3 14  8] 99
Generation  183
Fitness  134
[ 1  1  5  1 13  7  6  8 11  4  3 14  8] 77
[12  1  6  1  6  4  3 12 14  4  3 14 10] 108
[ 8  4 11  7 13  4  1  8 14 11  3 14  8] 84
[ 2 13 11  3 13  7  3  7 11 11  3 14  5] 105
[ 9  1 11 10 13  8  3  7 11 11  3  4 10] 82
[ 6  4 11  3 13 12  3  7  1  1 14  5  5] 79
[ 9  1 11  3 13 12  4  7 11  5  3 14 14] 98
[ 9 14 11  1 13  7  3  5  6 11  3 14  8] 85
[ 8 11  3  1  6  4 11  7 14 14  3 14 10] 90
[ 6  4 11  7  6  4  5  8 14 14  3 14  8] 89
[14  1 11 10 13  4  3  7 11  1  3 14  5] 77
[ 9 12 11  3 13  7  3  8 11 11  3  5 13] 70
[12  1 11  3 10  5  3  8 11 11  3 14 14] 86
Generation  184
Fitness  134
[ 9  1 11  1 13  8  3  8 14 11  3 12  6] 86
[12  9  6  1 13  4  3 12  6  4  3 14  5] 73
[ 2 13 11  3 13 12  4  7  7  5  3 14 14] 96
[ 9  1  8  3 13  6  4  7 11  5  3 14  5] 87
[ 9  1  5  3  3 12  3  7 11 14  3 14 10] 69
[ 8 11  3  7  6  4  5  8 14 14  3 14  8] 100
[ 6  4 11  1 13  7  2  8  8 11  3 14  8] 83
[ 9  1  7 14 13  7 12  8  9 13  3 14  8] 86
[ 2  1 11  1  6  4  6 12 14  4  6 14 10] 88
[12  1  6  1  6  4  3  7 11 11  7  7  5] 76
[ 2 13 11  3 13  7  3  7 11 11  3 14 14] 111
[ 9  1 11  3 13 12  3  7 11  1  8 14  5] 81
[ 9 11  3  1  6  4 11  7 14 14  3 14 10] 82
[ 8  6  3  2  6  4 11  7 14 14  3 14  8] 85
Generation  185
Fitness  134
[ 9  1 11  1 13  1  3  8 11 11  3 14  8] 64
[ 4  1 11  1 13  7  3  6 11 11  3 14 14] 60
[ 2 13 11  7  6  4  5  8 14 14  3 14  8] 70
[ 8 11  3  7  6  4  4  8  3 14  3 14 14] 106
[ 1 13 11  1  6  4  6  7 14  4  6 14 10] 82
[ 2  4 11  1  6  4  6 12 14  5  3 14  5] 57
[ 9  1  8  3 13  6  4  7 11  5  3 14 11] 94
[ 9  1  7 14  7  7 12  8  9 13  3 14  8] 96
[ 9  1 11  1 13  7  3  8 11  7  3 14  8] 89
[ 9 13 11  3 13  7  3  7 11  5  3 14 14] 74
[ 8 11  3  7  6  4  5  8 14 10 14 14  8] 74
[ 8  1  3  7  6 12  4  7  7 10  3 14  4] 74
[13 13  5 10  9  4  6 12 14  4  6 11 10] 88
[ 2  1 11  1  6  4  6 12 14  4  3 14  5] 73
[ 9  9  7 14 13  7 12  8  9 13  3  8  8] 88
Generation  186
Fitness  134
[ 9  1 11  4 13  4  4  2  3 14  3  4 14] 80
[10 11  3  7  7  7 12  8  9 13  3 14  8] 93
[ 9  1  7 14  7  7 12  8  1 13  3 14 11] 97
[ 9  1  8  3 13  6  4  7 11  7  8 14  7] 89
[ 9  1  3 14 13  7 12  8  9 13  3  8  8] 91
[ 9  9  7 14 12  7  6 12 14  4  6  3 10] 76
[13 13 11  1  6  4  1  7 11  4  6  3 10] 84
[ 9  1 11  7  6  4  4  8  3 14  3 13 14] 75
[ 8 11  3  7  6  4  4 11  3  5  6 14  8] 95
[ 9  1  1 14  7  7 12  8  9 12  3 14 11] 89
[ 2  1  8  3 13 10  4  7 11  7  3 14  1] 111
[ 9  1 11  1 13  1  3  8  6  7  3  8  8] 89
[ 9  9  2 14 13  9 12  8 14  5  6  6 10] 97
[13 13  5 10  9  4  6 12 14  4  6 14 13] 76
Generation  187
Fitness  134
[ 9  1 11  1 13  5  3  8 13 11  3 14 12] 107
[10  1  8  3 13 10  4 11 11  7  3 14  1] 86
[ 3  9  2 14 13  7 12 10 14  8  6  6 10] 98
[ 9  9  2 14 13  9 12  8  1 14  3 14  9] 105
[ 8 11  3  7  6  4  4  9  9 13  3 14  8] 103
[10 11  3  7  7  7 12  8  9 13  3  8  4] 73
[ 9  1  3 12 13  7  3  8 11 11 11 14  9] 82
[ 9  1 12  1 13  7  3  8 11 11  3 14  8] 85
[ 9  1 11  3 13  7  9  8 11 11  3 14  1] 84
[ 2  1  8  4 13 10  4  7 14  5 14  6 10] 61
[ 9  9 13 14 13  7 12  8  1 13  3 14 11] 89
[ 9  3  7 14  7  7 12  8  1  5  6 14  8] 97
[ 8 11  3  7  6  4  4  8  9 13  3 14  8] 93
[10 14  3 14  3  7 12  8  9 14 10  8  8] 113
Generation  188
Fitness  134
[ 9  1 11  1  3  7 12  8  9 14 10  8  8] 84
[10  1  3  1  4  5  3  8 13 11 14 14 12] 89
[ 9  9  2 14 13  9 12  8  1 14  3  5  8] 90
[ 9  9  2 14 13  4  4 14  9 13  3 11  8] 65
[ 8 11 14  7 11  4  4 13 14  8  6  6 10] 86
[11  9  2  5 13  7 12  8  1  5  6 10  8] 105
[ 9  3  7 14  7  7 12  8  3  5  6 14  8] 109
[14 11  3  1 13  7  8  8 11 11  8 14  8] 114
[ 9  1 11  1 13  7  3  8 11 14 10  8  8] 97
[10  1 11  6 13  5  3  8 13 11  3 14 12] 71
[ 9  1 11  1 13  5  3  8 13 14  3 14  9] 84
[ 9  9  3 14 13  9 12  9  9 13  3 14  8] 78
[ 8 11  3  7  6  8  4  9 14 13  3 14 10] 93
[ 3  2  2 14 13  7 12 10  6  9  6 14  8] 80
[ 9  3  7 14  7  7 12  8 11  5  6 14  8] 77
Generation  189
Fitness  134
[ 9  1 11  1 13  7  8  8 11 11  8  9  8] 89
[14  1  3  6 13  7  8 13 11 11  4 14  9] 85
[ 9  3  7  5 13  7  6  8  1 10  6 14  8] 94
[11  9  2  5 13  7  5  6  1  5 10  8  8] 70
[ 9  1 11  1 13  7  3  8 11 13  3 14 13] 84
[ 8 11  2 14 13  9 12  8 13  6  3  5  8] 92
[ 9 11  2 14 13  5  3  8 13 11 14  1 12] 94
[10  1  3  1  4  5  3  8 11  5  3 14  8] 98
[ 9 11 12  1 13  7  8  8 11 11  8  8  8] 70
[14 11  7 14  7  7 12  8  3  5  6 14  8] 74
[ 9  3  7 14  7  7 12  8  3  5  6  5  8] 99
[11  1 11  1 13  7  3  8 11  4 10  8  8] 67
[ 9  1 11  1 13  8  4  9 14  7  7 14 10] 106
[ 8 11  3  7  6  8  4  9 14 13  3  5  8] 96
[ 9  9  2 14  4  7  3  3 13 11 14 14 12] 83
Generation  190
Fitness  134
[ 9  1 14  1 13  8  4  9 14  7  7 14 10] 86
[ 9  1 11  1 13  2 12  8  3  5  6  5  8] 82
[ 9  3  7 14  7  7 12  8  3  5  2 14  8] 99
[10  1  3  1  4  5  3  3 11  5  3 12  8] 96
[13 11  2 14 13  5  3  8 13 11 14  1 12] 97
[ 9 11  3 14 13  5  3  8 13 11 14  1  8] 99
[ 9  3  7  5 13  7  6  8  1 10  6  5  8] 99
[ 8  9  2 14 13  7  3  8 11 11  3 14  2] 101
[ 9  1 11  1 13  5  3 13 14  6  7 14 10] 88
[ 9  1 11  9 13  8 12  8  3  5  3  5  8] 104
[ 9  3  3  6  4  5  3  5 11  5  3 14  8] 66
[10  1  3  1  4 13  3  8  2  5  3  5  8] 101
[ 9 11  2 14 13  5  3  7 13 11 14  1 12] 85
[ 9 11  2 14 13  5  6  8 13 10  6 14  8] 65
[ 9  3  7  5  4  7  6  8 13  6  8  5  8] 67
Generation  191
Fitness  134
[ 9  1 11  1 13  7  3  8 11  5  3  4  8] 92
[ 9  1 11  9  4  8 12  8  2  8  3  1  8] 87
[ 2  9  2 14 13  7 13  8  6 11  3 14  2] 98
[ 8  9  2  5 13  7  6  8  4 10  6  5  8] 93
[ 9  3 14  5 11  7  6  8  1 10  6  1 12] 86
[ 9 11  3 14 13  5  3  8 13  5  2  1  8] 61
[14  3  7 14  3 14 12  1 13 11 14  1 12] 77
[13 11  2 14  7  5  3  8 13 11 14  1  8] 70
[ 9  1 11  1 13  7  3  8 11  5  3  5  8] 88
[ 9  1  3  1  4 13  3  8  2  5  3  5  8] 71
[10  1  3  1  4  9  3  8 11 11  3  9  2] 60
[ 8  9  2 14  5  7  6  8  1 10  6  3  8] 87
[ 9  3 13  5 13  7  6  8 13  4  6  5  8] 80
[ 9 11  3 14 13  5 11  8  3  5  2  1  8] 86
[ 9 11  2 14 13  7  3  8 13 11 14  1 12] 86
Generation  192
Fitness  134
[ 4  1 11 14 13  7 13  8  2 11  3 14  2] 94
[ 2  9  2 14 13  7 13  8  4 10 13  5  7] 97
[ 8  9  2  5 13 14  6  8  4 10  3  4  2] 104
[ 9  1  9  1 13  7  3 14 11  5  3  5  8] 91
[ 9 14 11 10  5  7 10  8  1 10  6  5  8] 104
[ 8  9  2 14  5  7  6  8  1  8 13  1  8] 92
[ 9  1 11  9  6  8 12  8  2  8  3  1 12] 76
[ 9 11  2  1 13  7  3  8 11 14  3 14  8] 91
[ 9  1 11  1 13  2 13  8  6 11  3 14  2] 100
[ 2  9  6 10 13  7 13  8  6 11  6  4  8] 84
[ 8  9  2  5 13  7  6  8  4  5  3 10  8] 98
[ 9  1  3  1 13  7  3  8  9  5  3  4  8] 89
[ 9  1 11 14  5  7  6  1 13 10  6  3  8] 93
[ 8  9  2 14  5  8 12  8  8  8  1  1  8] 67
[ 9  1 11  9  4  8 12  8  2  8  3  1 12] 88
Generation  193
Fitness  134
[ 9  1 11  1 13  7  3  8  1 10  6  5  8] 92
[ 1 14  2  5 13 14  4  8  4  9  3  4  2] 93
[ 8  9  2  7 13  2 10  8  3 11  3 14  2] 78
[ 9  1 11  1 13  3 13 10  6  9  3 10  8] 76
[ 8  9  2 14 13  7 13  8  4 10 11 14  7] 64
[ 2 13  2 14 13  7 13  8  2 11  3  9  2] 103
[ 4  1 11 14 13  2 12  1  7 10  6  9  8] 70
[ 9  1 11 14  5  7  6  1 13 10  6 14  8] 108
[ 9  1 11  1 13  7 10  8  1 10  6  5  8] 71
[ 9 12 11 10 13 14  6  8  4 10  3  4  2] 101
[14  9 11  1 13  2 13  8  6 11  3 14  2] 93
[ 9  1  9  1 13  2 13 14  6  5  3 10 11] 88
[ 8  9  2 14 13  7 13  8  4 10 13 12  7] 77
[ 2  9  2  1 13  7 11  8  2 11  3 14  2] 101
[ 4  1 11 14 13  7 13  1 13 13  7  3  8] 98
Generation  194
Fitness  134
[ 9  1  4  1 13  7  6  1 13 10  6 14  8] 87
[ 9  1 11 14  5  7 14  4 13  1  3  9  2] 89
[ 2  9  2  1  8  7 11  8  2 11  3 14  7] 88
[ 2  9  2  1 13 10 11  8  2 11  3 14  7] 76
[ 9 12 11 10 13 14  6  8  4 10  3  4 10] 92
[ 8  1 11 14 13  2 13  8  9 11  3  5  2] 65
[14  9 11  5 13 14  4  8  4  9  3 13  2] 79
[14  1 11  1 13  1  3  8 11 11  3 14  8] 83
[ 9  1 11  1  2  7 14  8 11 11  3 14  8] 93
[ 9  1  2 14 13  7 13  8  2 11  3  9  2] 70
[ 2  2 13 14  5 10 13  8  2 11  3 14  2] 91
[ 2  9  2 10 13 14  6  8  4 10  3  4  2] 77
[ 9 12 11 10 13 14 11  6 13  1  7  3  8] 89
[ 3  1 11  1 13  2 13 11  6 11  3 14  2] 94
[ 8 14  2  5 13 14  4  8  4  9  3  4  2] 83
Generation  195
Fitness  134
[ 9 13 11 11 13  7 13 11  6 11  3 14  2] 89
[ 3  1 11  1 13  2 13 11  6 11  3 14  8] 85
[ 9  1 11  1  2  7 14  8  4 10  3  4 10] 97
[ 9 12 11 10 13 10  6  8  2  5  3 14 11] 79
[ 2  2 11 10 13 14 13  6 13 11  7 11  8] 80
[ 9 12 10 10 13 14 11  6 13  1  7  9  2] 80
[ 9  1 11 14  5  7 11  8  2 11  3 14  7] 99
[ 2  9 11  1 13  7  3 13 11 10 12 14  8] 104
[ 7  1 11  1 13  4 13 11  6 11  3  2  2] 96
[ 9  1 11  1 10  9  6  8  4 10  3  4 10] 104
[ 9 12 11 10  8 14  6 14  4 11  3 14  2] 94
[ 2  2  2 14  5 10 13  8  6 11  3  3  8] 93
[ 9 12 11 10 13 14 11  6 13  1  7  3  2] 93
[ 9  9  2  1 14  7 11  8  8 11  3 14  7] 94
Generation  196
Fitness  134
[ 9  1 11  1  6  9  6  8  4 10  3  4 10] 89
[ 9  1 11  1 13  7  3  9 11 10 12 14  8] 73
[ 9  1 11 14  5  2 11  8  2 11  3 13  7] 78
[ 9  1  1 14  5  7 11 13  2 11  3 14 10] 82
[ 9  1 13  1  2  7 14  8  4 10  3  4  5] 83
[ 7  1 11  1  9  4 13  8  8 11  4 14 10] 81
[ 9  9  2  1 14  7 11  8  8  3  3 14  2] 75
[ 9 12 11 10  8 14  6 14 11 11  9 14  8] 90
[ 9  1 11  1 14  7  3  8 13 10  3  4 10] 72
[ 9  3 11  1 10  9  6 13 11 10 12 10  8] 67
[ 2  9 11  1 13  7  3 13 11 10 12 14  7] 88
[ 9  1 11 13  5  7 11 14 10 10  3  6 10] 72
[ 9  1 11  1  2  7 14 11  6 11  3  3  2] 88
[ 7  1 11  1 13  4 11  8  8  3  5  4  7] 95
[ 9  9  9  4 14  7 11  8  8 11  3 14  2] 98
Generation  197
Fitness  134
[ 9  1 11  1 13  7  3  8 10 11  9 13  2] 103
[ 9  6 11  1 13  4 11 12  8  3  5 10  7] 102
[ 7  1 11  1 13  4  3  8  8  3  9 10  8] 66
[ 9 12 11  1  6  9  4  8  4 10  3  4 10] 72
[ 9  1 11  1  6  9  6  9  4 11  3  3  5] 93
[ 9  1 11  1  2  7 14 12  6 11 12 14  7] 73
[12  9 11  1 13  3  3 13  4 10  3  4  5] 80
[ 9  1 13  1  2  7 14  8  4 10  7 14  8] 88
[ 9  1 10  1 13  7 11  8  8 11  3 14  2] 73
[ 9  9  9  4 14  7 11  8 10 11  5 11  7] 74
[ 7  8 11 10 13 14  6 14 11 11  9 14  8] 99
[ 9  2 11 10  8 14  6 14 11 11  9 14 10] 67
[ 9  1 11  1  2  7 13 11  6 11  3  3 14] 76
[ 8  1 11  1  2  7 14 11  6 11  3  3  7] 74
[ 2  9 13  1  2  7  2  8  4 10 10  6  5] 69
Generation  198
Fitness  134
[ 4  1  3  1 13  7  3  8 10 11  9 13  8] 91
[ 9  6 11  4  3  4 11 14 13 11  9 14  8] 79
[ 7  8 11 10 13 14  6 14 11 11  3 12  5] 100
[ 9  1 11  4  6  9  6 10  4 11  3  3 11] 69
[ 9  1 13  1  2  7 14  8  4 10  7 14  5] 91
[ 9  1 11  3  2  7 13 11  6 11  3  3 14] 77
[ 9  1 11  1 13  7  3  8 11 11  3 14 11] 104
[ 9 12 11  1 13  7  3  8 11 11  3 13  2] 60
[ 9  1 11  1 13  2 11 12  8  3  5 10  7] 85
[11  6 11  1 13  4 11 12  8 11  9 14  9] 78
[ 7  8 11 10 13  3  6  9 14 11  3  3  5] 81
[ 9  1 11  1  6  9  6  9 14  2  7  6  8] 76
[12  9 11  1  2  7 13 11  6 11  3  9 14] 92
Generation  199
Fitness  134
[ 9  1 11  1 13  7  3  5 11 11  7 14 11] 91
[ 9  8 11  2 14  7 11  8  8  3  5 10  2] 82
[ 9  6 11  1 13  4 11 12  8  3  3 12  5] 86
[ 7  8 10 10  2  9 13 11  6 10  7  9  9] 79
[12  9 11  4  2  7 13  2  4 10  7  7  5] 78
[ 4  1  3  1 13  7  3  8 10 11  5 13  8] 96
[ 4  1  3  1 13  2 11 12  8 14  5 10  7] 88
[ 9  7 11  1 13  7  3  8 11 11  3 14  8] 86
[13  1 11  1 13  7  3  8  6 11  3 14 11] 94
[ 9  1  4  1 13  7  3  8 11  3  5  1  7] 102
[ 9  5 11  1 13 14  6 14 11 11  3 12  5] 84
[ 7  8 11 10 13 14 11 14 10 11  3  7  8] 90
[12 11 13  7  2  7 14 10  4 10 13 14  5] 94
[ 9  1 11  1  2  7  3 11 10  3  9 13  8] 82
[ 4  1 11 12 13  2 11 12  8 10  5 12  7] 95
Generation  200
Fitness  134
/home/juniper/Documents/Schoo/IIA/my_virtual_env/lib/python3.13/site-packages/pygad/visualize/plot.py:120: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  matplt.legend()

NameError: name 'ga_instance' is not defined

In [65]:
# input = [ 0.52787664, -1.80397515, -0.84931827,  3.95207083,  1.8429896,  -0.03634803,
#   0.21389263,  4.0924443,  -0.7545396,   2.81553299,  0.61459585] 

input = [ 4.27781311, 4.38457935, 4.08930862, 3.88120389, -1.8686245, -0.71927776, 4.38365042, -0.69951014, -0.55374408, -0.47743206, -1.1693262 ] 

run_tournament(input)

Profundidade 1
8
Profundidade 2
9
Profundidade 3


KeyboardInterrupt: 